In [2]:
import pandas as pd
from openpyxl import load_workbook
import numpy as np
import openpyxl
import csv
import math
import datetime
from datetime import datetime
from dateutil import rrule
from datetime import date
from datetime import datetime,timedelta
import time
import json
from collections import OrderedDict
from PIL import Image
import io
import requests
import os
import sys
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from numpy import log as ln

# 전류가 흐르지않는 상태(지게차 휴식 상태) -> 0 (+- 2A 이하)
# 방전 상태 - 1 -> 0 -> 2 음수 전류
# 충전 상태 - -1 -> 1 -> 1 양수 전류


def preprocess(csvname):
    
    df = pd.read_csv('f008d1709732.csv')  #받은 파일
    df1 = df.dropna() #NAN 값 제거
    df1['regDate']=df1['regDate'].astype(str) #지수값 변환
    df2 = df1.reset_index()

    global batteryCapacity
    batteryCapacity = 565.0
    
    D = []
    year = []
    month = []
    day = []
    hour = []
    minute = []
    for i in range(len(df2)) :
        D.append(datetime.strptime(df2['regDate'][i],'%Y-%m-%dT%H:%M:%S.000Z')-timedelta(hours=9)) #시간 변환
#         D.append(datetime.strptime(df2['regDate'][i],'%Y-%m-%dT%H:%M:%S.000Z')) #시간 변환
        year.append(D[i].strftime('%Y'))
        month.append(D[i].strftime('%m'))
        day.append(D[i].strftime('%d'))
        hour.append(D[i].strftime('%H'))
        minute.append(D[i].strftime('%M'))
        
    df1['year']= year
    df1['month']= month
    df1['day']= day
    df1['date'] = df1['year']+df1['month']+df1['day']
    
    df1['hour']= hour
    df1['minute']= minute #날짜 분리
        
    del df1['year']
    del df1['month']
    del df1['day']
    del df1['_id']

    del df1['regDate'] 
    del df1['Unnamed: 0']
    #불필요한 부분 지우기  
    
    df1.sort_values(by =['date','hour','minute'],inplace = True)
    df1.reset_index(drop=True, inplace=True)

    chargeDischarge_list = []
    for i in range(len(df1)):
        if i == 0:
            chargeDischarge_list.append(df1['chargeDischarge'][i])
        else :
            if df1['chargeDischarge'][i] == 0:
                if chargeDischarge_list[-1] == 1:
                    chargeDischarge_list.append(1)
                elif chargeDischarge_list[-1] == 2:
                    chargeDischarge_list.append(2)
            elif df1['chargeDischarge'][i] == 1:
                chargeDischarge_list.append(1)
            elif df1['chargeDischarge'][i] == 2:
                chargeDischarge_list.append(2)

    df1['new_chargeDischarge'] = chargeDischarge_list

#     rmd = df1[(df1['voltaic'] < 1) & (df1['voltaic'] > -1)].index #전류값이 -1에서 1일때의 값들 삭제 
#     rmdf = df1.drop(rmd) #오류 값 제거

#     bdf = rmdf.reset_index() #인덱스 값 새로 설정  

#     bdf1 = bdf.sort_values(by =['date','hour','minute'],inplace = False)
#     bdf2 = bdf1.reset_index()
#     del bdf2['level_0']

    bdf1 = df1.sort_values(by =['date','hour','minute'],inplace = False) # 날짜 정렬
    bdf2 = bdf1.reset_index()

    rm_charge_soc_list = [] #충전 SOC 리스트
    rm_discharge_soc_list = [] #방전 SOC 리스트
    i = 0
    if bdf2['chargeDischarge'][i] == 1: #충전
        if(bdf2['voltage'][i] >= 51.1):
            rm_charge_soc_list.append(100)
        elif(bdf2['voltage'][i] >= 50.9 and bdf2['voltage'][i] < 51.1) :
            rm_charge_soc_list.append(95)
        elif(bdf2['voltage'][i] >= 50.6 and bdf2['voltage'][i] < 50.9):
            rm_charge_soc_list.append(90)
        elif(bdf2['voltage'][i] >= 50.4 and bdf2['voltage'][i] < 50.6) :
            rm_charge_soc_list.append(85)
        elif(bdf2['voltage'][i] >= 50.2 and bdf2['voltage'][i] < 50.4):
            rm_charge_soc_list.append(80)
        elif(bdf2['voltage'][i] >= 49.9 and bdf2['voltage'][i] < 50.2) :
            rm_charge_soc_list.append(75)
        elif(bdf2['voltage'][i] >= 49.7 and bdf2['voltage'][i] < 49.9):
            rm_charge_soc_list.append(70)
        elif(bdf2['voltage'][i] >= 49.4 and bdf2['voltage'][i] < 49.7) :
            rm_charge_soc_list.append(65)
        elif(bdf2['voltage'][i] >= 49.2 and bdf2['voltage'][i] < 49.4):
            rm_charge_soc_list.append(60)
        elif(bdf2['voltage'][i] >= 49.0 and bdf2['voltage'][i] < 49.2) :
            rm_charge_soc_list.append(55)
        elif(bdf2['voltage'][i] >= 48.7 and bdf2['voltage'][i] < 49.0):
            rm_charge_soc_list.append(50)
        elif(bdf2['voltage'][i] >= 48.5 and bdf2['voltage'][i] < 48.7) :
            rm_charge_soc_list.append(45)
        elif(bdf2['voltage'][i] >= 48.2 and bdf2['voltage'][i] < 48.5):
            rm_charge_soc_list.append(40)
        elif(bdf2['voltage'][i] >= 48.0 and bdf2['voltage'][i] < 48.2) :
            rm_charge_soc_list.append(35)
        elif(bdf2['voltage'][i] >= 47.8 and bdf2['voltage'][i] < 48.0):
            rm_charge_soc_list.append(30)
        elif(bdf2['voltage'][i] >= 47.5 and bdf2['voltage'][i] < 47.8) :
            rm_charge_soc_list.append(25)
        elif(bdf2['voltage'][i] >= 47.3 and bdf2['voltage'][i] < 47.5):
            rm_charge_soc_list.append(20)
        elif(bdf2['voltage'][i] >= 47.0 and bdf2['voltage'][i] < 47.3) :
            rm_charge_soc_list.append(15)
        elif(bdf2['voltage'][i] >= 46.8 and bdf2['voltage'][i] < 47.0):
            rm_charge_soc_list.append(10)
        elif(bdf2['voltage'][i] >= 46.6 and bdf2['voltage'][i] < 46.8) :
            rm_charge_soc_list.append(5)
        elif(bdf2['voltage'][i] < 46.6):
            rm_charge_soc_list.append(0)
        else:
            pass
    elif bdf2['chargeDischarge'][i] == 2: #방전
        if(bdf2['voltage'][i] >= 51.1):
            rm_discharge_soc_list.append(100)
        elif(bdf2['voltage'][i] >= 50.9 and bdf2['voltage'][i] < 51.1) :
            rm_discharge_soc_list.append(95)
        elif(bdf2['voltage'][i] >= 50.6 and bdf2['voltage'][i] < 50.9):
            rm_discharge_soc_list.append(90)
        elif(bdf2['voltage'][i] >= 50.4 and bdf2['voltage'][i] < 50.6) :
            rm_discharge_soc_list.append(85)
        elif(bdf2['voltage'][i] >= 50.2 and bdf2['voltage'][i] < 50.4):
            rm_discharge_soc_list.append(80)
        elif(bdf2['voltage'][i] >= 49.9 and bdf2['voltage'][i] < 50.2) :
            rm_discharge_soc_list.append(75)
        elif(bdf2['voltage'][i] >= 49.7 and bdf2['voltage'][i] < 49.9):
            rm_discharge_soc_list.append(70)
        elif(bdf2['voltage'][i] >= 49.4 and bdf2['voltage'][i] < 49.7) :
            rm_discharge_soc_list.append(65)
        elif(bdf2['voltage'][i] >= 49.2 and bdf2['voltage'][i] < 49.4):
            rm_discharge_soc_list.append(60)
        elif(bdf2['voltage'][i] >= 49.0 and bdf2['voltage'][i] < 49.2) :
            rm_discharge_soc_list.append(55)
        elif(bdf2['voltage'][i] >= 48.7 and bdf2['voltage'][i] < 49.0):
            rm_discharge_soc_list.append(50)
        elif(bdf2['voltage'][i] >= 48.5 and bdf2['voltage'][i] < 48.7) :
            rm_discharge_soc_list.append(45)
        elif(bdf2['voltage'][i] >= 48.2 and bdf2['voltage'][i] < 48.5):
            rm_discharge_soc_list.append(40)
        elif(bdf2['voltage'][i] >= 48.0 and bdf2['voltage'][i] < 48.2) :
            rm_discharge_soc_list.append(35)
        elif(bdf2['voltage'][i] >= 47.8 and bdf2['voltage'][i] < 48.0):
            rm_discharge_soc_list.append(30)
        elif(bdf2['voltage'][i] >= 47.5 and bdf2['voltage'][i] < 47.8) :
            rm_discharge_soc_list.append(25)
        elif(bdf2['voltage'][i] >= 47.3 and bdf2['voltage'][i] < 47.5):
            rm_discharge_soc_list.append(20)
        elif(bdf2['voltage'][i] >= 47.0 and bdf2['voltage'][i] < 47.3) :
            rm_discharge_soc_list.append(15)
        elif(bdf2['voltage'][i] >= 46.8 and bdf2['voltage'][i] < 47.0):
            rm_discharge_soc_list.append(10)
        elif(bdf2['voltage'][i] >= 46.6 and bdf2['voltage'][i] < 46.8) :
            rm_discharge_soc_list.append(5)
        elif(bdf2['voltage'][i] < 46.6):
            rm_discharge_soc_list.append(0)
        else:
            pass
    
    charge_soc = [] # 충전 soc
    discharge_soc = [] # 방전 soc
    # 1 -> 충전, 2 -> 방전
    for i in range(len(bdf2)-1):
        if i == 0:
            if bdf2['new_chargeDischarge'][i] == 1:
                charge_soc.append(rm_charge_soc_list[0])
            elif bdf2['new_chargeDischarge'][i] == 2:
                discharge_soc.append(rm_discharge_soc_list[0])

        elif i != 0:
            if bdf2['new_chargeDischarge'][i] == bdf2['new_chargeDischarge'][i-1] :
                if bdf2['new_chargeDischarge'][i] == 1:
                    charge_soc.append(charge_soc[-1] + ((-0.0007 * charge_soc[-1] + 1.00716) * (abs(bdf2['voltaic'][i]) * 60)/(batteryCapacity*3600))*100)
                elif bdf2['new_chargeDischarge'][i] == 2:
                    discharge_soc.append(discharge_soc[-1] + ((bdf2['voltaic'][i] * 60)/(batteryCapacity*3600))*100)
            elif bdf2['new_chargeDischarge'][i] != bdf2['new_chargeDischarge'][i-1]:
                if bdf2['new_chargeDischarge'][i] == 1:
                    charge_soc.append(discharge_soc[-1] + ((-0.0007 * discharge_soc[-1] + 1.00716) * (abs(bdf2['voltaic'][i]) * 60)/(batteryCapacity*3600))*100)
                elif bdf2['new_chargeDischarge'][i] == 2:
                    discharge_soc.append(charge_soc[-1] + ((bdf2['voltaic'][i] * 60)/(batteryCapacity*3600))*100) 


    add_soc = [] # 충전 soc와 방전 soc를 합친 리스트
    charge_soc_count = 0
    discharge_soc_count = 0
    for i in range(len(bdf2)-1):
        if bdf2['new_chargeDischarge'][i] == 2:
            add_soc.append(discharge_soc[discharge_soc_count])
            discharge_soc_count = discharge_soc_count +1

        elif bdf2['new_chargeDischarge'][i] == 1:
            add_soc.append(charge_soc[charge_soc_count])
            charge_soc_count = charge_soc_count + 1

    bdf3 = bdf2.drop(index=[len(bdf2)-1],inplace=False)
    bdf3['soc']=add_soc

    new_charge_soc =[] # 충전 soc
    new_discharge_soc =[] # 방전 soc
    modify_count = 0
    voltage_count = 0
    # 1 -> 충전, 2 -> 방전
    for i in range(len(bdf3)):
        if i == 0:
            if bdf3['new_chargeDischarge'][i] == 1:
                new_charge_soc.append(bdf3['soc'][i])
            elif bdf3['new_chargeDischarge'][i] == 2:
                new_discharge_soc.append(bdf3['soc'][i]) # 초기값 설정             
        elif i != 0:
            if bdf3['new_chargeDischarge'][i] == bdf3['new_chargeDischarge'][i-1] :
                if bdf3['new_chargeDischarge'][i] == 1 and bdf3['chargeDischarge'][i] == 1:
                    if bdf3['voltage'][i] > 51.2 :
                        voltage_count = voltage_count +1
                        if voltage_count > 30 :
                            new_charge_soc.append(100)
                        else :
                            modify_count = 0
                            new_charge_soc.append(new_charge_soc[-1] + ((-0.0007 * new_charge_soc[-1] + 1.00716) * (abs(bdf3['voltaic'][i]) * 60)/(batteryCapacity*3600))*100)
                    else :
                        voltage_count = 0
                        new_charge_soc.append(new_charge_soc[-1] + ((-0.0007 * new_charge_soc[-1] + 1.00716) * (abs(bdf3['voltaic'][i]) * 60)/(batteryCapacity*3600))*100)

                elif bdf3['new_chargeDischarge'][i] == 1 and bdf3['chargeDischarge'][i] == 0:
                    modify_count = modify_count +1
                    voltage_count = 0
                    if modify_count == 31 :
                        if(bdf3['voltage'][i] >= 51.1):
                            new_charge_soc.append(100)
                        elif(bdf3['voltage'][i] >= 50.6 and bdf3['voltage'][i] < 51.1):
                            new_charge_soc.append(90)
                        elif(bdf3['voltage'][i] >= 50.2 and bdf3['voltage'][i] < 50.6):
                            new_charge_soc.append(80)
                        elif(bdf3['voltage'][i] >= 49.7 and bdf3['voltage'][i] < 50.2):
                            new_charge_soc.append(70)
                        elif(bdf3['voltage'][i] >= 49.2 and bdf3['voltage'][i] < 49.7):
                            new_charge_soc.append(60)
                        elif(bdf3['voltage'][i] >= 48.7 and bdf3['voltage'][i] < 49.2):
                            new_charge_soc.append(50)
                        elif(bdf3['voltage'][i] >= 48.2 and bdf3['voltage'][i] < 48.7):
                            new_charge_soc.append(40)
                        elif(bdf3['voltage'][i] >= 47.8 and bdf3['voltage'][i] < 48.2):
                            new_charge_soc.append(30)
                        elif(bdf3['voltage'][i] >= 47.3 and bdf3['voltage'][i] < 47.8):
                            new_charge_soc.append(20)
                        elif(bdf3['voltage'][i] >= 46.8 and bdf3['voltage'][i] < 47.3):
                            new_charge_soc.append(10)
                        elif(bdf3['voltage'][i] < 46.8):
                            new_charge_soc.append(0)
                    elif modify_count > 31 :
                        new_charge_soc.append(new_charge_soc[-1])

                    else :
                        new_charge_soc.append(new_charge_soc[-1] + ((-0.0007 * new_charge_soc[-1] + 1.00716) * (abs(bdf3['voltaic'][i]) * 60)/(batteryCapacity*3600))*100)

                elif bdf3['new_chargeDischarge'][i] == 2 and bdf3['chargeDischarge'][i] == 2:
                    modify_count = 0
                    voltage_count = 0
                    if new_discharge_soc[-1] + ((bdf3['voltaic'][i] * 60)/(batteryCapacity*3600))*100 < 0:
                        new_discharge_soc.append(0)
        
                    else :
                        new_discharge_soc.append(new_discharge_soc[-1] + ((bdf3['voltaic'][i] * 60)/(batteryCapacity*3600))*100)
        
                elif bdf3['new_chargeDischarge'][i] == 2 and bdf3['chargeDischarge'][i] == 0:
                    modify_count = modify_count + 1
                    voltage_count = 0
                    if modify_count == 31 :
                        if(bdf3['voltage'][i] >= 51.1):
                            new_discharge_soc.append(100)
                        elif(bdf3['voltage'][i] >= 50.6 and bdf3['voltage'][i] < 51.1):
                            new_discharge_soc.append(90)
                        elif(bdf3['voltage'][i] >= 50.2 and bdf3['voltage'][i] < 50.6):
                            new_discharge_soc.append(80)
                        elif(bdf3['voltage'][i] >= 49.7 and bdf3['voltage'][i] < 50.2):
                            new_discharge_soc.append(70)
                        elif(bdf3['voltage'][i] >= 49.2 and bdf3['voltage'][i] < 49.7):
                            new_discharge_soc.append(60)
                        elif(bdf3['voltage'][i] >= 48.7 and bdf3['voltage'][i] < 49.2):
                            new_discharge_soc.append(50)
                        elif(bdf3['voltage'][i] >= 48.2 and bdf3['voltage'][i] < 48.7):
                            new_discharge_soc.append(40)
                        elif(bdf3['voltage'][i] >= 47.8 and bdf3['voltage'][i] < 48.2):
                            new_discharge_soc.append(30)
                        elif(bdf3['voltage'][i] >= 47.3 and bdf3['voltage'][i] < 47.8):
                            new_discharge_soc.append(20)
                        elif(bdf3['voltage'][i] >= 46.8 and bdf3['voltage'][i] < 47.3):
                            new_discharge_soc.append(10)
                        elif(bdf3['voltage'][i] < 46.8):
                            new_discharge_soc.append(0)
                    elif modify_count > 31 :
                        new_discharge_soc.append(new_discharge_soc[-1])

                    else :
                        new_discharge_soc.append(new_discharge_soc[-1] + ((bdf3['voltaic'][i] * 60)/(batteryCapacity*3600))*100)

            elif bdf3['new_chargeDischarge'][i] != bdf3['chargeDischarge'][i-1]:
                if bdf3['new_chargeDischarge'][i] == 1:
                    new_charge_soc.append(new_discharge_soc[-1] + ((-0.0007 * new_discharge_soc[-1] + 1.00716) * (abs(bdf3['voltaic'][i]) * 60)/(batteryCapacity*3600))*100)
                elif bdf3['new_chargeDischarge'][i] == 2:
                    new_discharge_soc.append(new_charge_soc[-1] + ((bdf3['voltaic'][i] * 60)/(batteryCapacity*3600))*100) 

            if (datetime.strptime(bdf3['date'][i]+bdf3['hour'][i]+bdf3['minute'][i],'%Y%m%d%H%M') - 
                datetime.strptime(bdf3['date'][i-1]+bdf3['hour'][i-1]+bdf3['minute'][i-1],'%Y%m%d%H%M')).seconds/60 > 30:

                if bdf3['chargeDischarge'][i] == 1:
                    if(bdf3['voltage'][i] >= 51.1):
                        new_charge_soc.append(100)
                    elif(bdf3['voltage'][i] >= 50.6 and bdf3['voltage'][i] < 51.1):
                        new_charge_soc.append(90)
                    elif(bdf3['voltage'][i] >= 50.2 and bdf3['voltage'][i] < 50.6):
                        new_charge_soc.append(80)
                    elif(bdf3['voltage'][i] >= 49.7 and bdf3['voltage'][i] < 50.2):
                        new_charge_soc.append(70)
                    elif(bdf3['voltage'][i] >= 49.2 and bdf3['voltage'][i] < 49.7):
                        new_charge_soc.append(60)
                    elif(bdf3['voltage'][i] >= 48.7 and bdf3['voltage'][i] < 49.2):
                        new_charge_soc.append(50)
                    elif(bdf3['voltage'][i] >= 48.2 and bdf3['voltage'][i] < 48.7):
                        new_charge_soc.append(40)
                    elif(bdf3['voltage'][i] >= 47.8 and bdf3['voltage'][i] < 48.2):
                        new_charge_soc.append(30)
                    elif(bdf3['voltage'][i] >= 47.3 and bdf3['voltage'][i] < 47.8):
                        new_charge_soc.append(20)
                    elif(bdf3['voltage'][i] >= 46.8 and bdf3['voltage'][i] < 47.3):
                        new_charge_soc.append(10)
                    elif(bdf3['voltage'][i] < 46.8):
                        new_charge_soc.append(0)

                elif bdf3['chargeDischarge'][i] == 2:
                    if(bdf3['voltage'][i] >= 51.1):
                        new_discharge_soc.append(100)
                    elif(bdf3['voltage'][i] >= 50.6 and bdf3['voltage'][i] < 51.1):
                        new_discharge_soc.append(90)
                    elif(bdf3['voltage'][i] >= 50.2 and bdf3['voltage'][i] < 50.6):
                        new_discharge_soc.append(80)
                    elif(bdf3['voltage'][i] >= 49.7 and bdf3['voltage'][i] < 50.2):
                        new_discharge_soc.append(70)
                    elif(bdf3['voltage'][i] >= 49.2 and bdf3['voltage'][i] < 49.7):
                        new_discharge_soc.append(60)
                    elif(bdf3['voltage'][i] >= 48.7 and bdf3['voltage'][i] < 49.2):
                        new_discharge_soc.append(50)
                    elif(bdf3['voltage'][i] >= 48.2 and bdf3['voltage'][i] < 48.7):
                        new_discharge_soc.append(40)
                    elif(bdf3['voltage'][i] >= 47.8 and bdf3['voltage'][i] < 48.2):
                        new_discharge_soc.append(30)
                    elif(bdf3['voltage'][i] >= 47.3 and bdf3['voltage'][i] < 47.8):
                        new_discharge_soc.append(20)
                    elif(bdf3['voltage'][i] >= 46.8 and bdf3['voltage'][i] < 47.3):
                        new_discharge_soc.append(10)
                    elif(bdf3['voltage'][i] < 46.8):
                        new_discharge_soc.append(0)

            else :
                pass


    new_add_soc = [] # 충전 soc와 방전 soc를 합친 리스트
    new_charge_soc_count  = 0
    new_discharge_soc_count  = 0
    for i in range(len(bdf3)):
        if bdf3['new_chargeDischarge'][i] == 2:
            new_add_soc.append(new_discharge_soc[new_discharge_soc_count])
            new_discharge_soc_count = new_discharge_soc_count + 1

        elif bdf3['new_chargeDischarge'][i] == 1:
            new_add_soc.append(new_charge_soc[new_charge_soc_count])
            new_charge_soc_count = new_charge_soc_count + 1


    bdf3['modify_soc'] = new_add_soc
    
#     overChargeCount = 0
#     overChargeState = False 
#     for i in range(len(bdf3)): # 과충전 횟수
#         if bdf3['soc'][i] > 120.0 :
#             if overChargeState == False:
#                 overChargeCount = overChargeCount + 1 
#                 overChargeState = True
#             else :
#                 pass
#         else:
#             overChargeState = False

    overChargeCount_condition_1 = 0
    overChargeCount_condition_2 = 0

    overChargeCount = 0
    for i in range(len(bdf3)): 
        if bdf3['voltage'][i] >= 58.0: # 58V이상 4시간 연속 유지시 과충전
            overChargeCount_condition_1 = overChargeCount_condition_1 + 1
            if overChargeCount_condition_1 == 240 :
                overChargeCount = overChargeCount + 1
            else :
                pass
        else:
            overChargeCount_condition_1 = 0

    for i in range(len(bdf3)):
        if bdf3['chargeDischarge'][i] == 1: # 충전시간이 10시간 연속 유지시 과충전
            overChargeCount_condition_2 = overChargeCount_condition_2 + 1
            if overChargeCount_condition_2 == 600 :
                overChargeCount = overChargeCount + 1
            else:
                pass
        else :
            overChargeCount_condition_2 = 0    

#     overdisChargeCount = 0    
#     overdisChargeState = False
#     for i in range(len(bdf3)): # 과방전 횟수
#         if bdf3['soc'][i] < 20.0:
#             if overdisChargeState == False :
#                 overdisChargeCount = overdisChargeCount + 1
#                 overdisChargeState = True
#             else :
#                 pass
#         else :
#             overdisChargeState = False


    overdisChargeCount = 0
    for i in range(len(bdf3)):
        if bdf3['chargeDischarge'][i] == 0 and bdf3['voltage'][i] <= 42.0 : # 42V이하로 떨어지는 경우 과방전 횟수
            overdisChargeCount = overdisChargeCount + 1

#     abnormalTemperatureCount = 0
#     abnormalTemperatureState = False

#     for i in range(len(bdf3)): # 온도이상 횟수
#         if bdf3['temperature'][i] >= 55.0 or bdf3['temperature'][i] <= -15.0 :
#             if abnormalTemperatureState == False :
#                 abnormalTemperatureCount = abnormalTemperatureCount + 1
#                 abnormalTemperatureState = True
#             else :
#                 pass
#         else :
#             abnormalTemperatureState = False

    abnormalTemperatureCount = 0
    abnormalTemperatureState = False

    for i in range(len(bdf3)):
        if bdf3['temperature'][i] >= 55.0  : #55도 이상이면 온도이상
            if abnormalTemperatureState == False :
                abnormalTemperatureCount = abnormalTemperatureCount + 1
                abnormalTemperatureState = True
            else :
                pass
        elif bdf3['temperature'][i] <= 45.0 :
            abnormalTemperatureState = False
            
    for i in range(len(bdf3)):
        if bdf3['temperature'][i] <= -15.0  : #-15도 이하이면 온도이상
            if abnormalTemperatureState == False :
                abnormalTemperatureCount = abnormalTemperatureCount + 1
                abnormalTemperatureState = True
            else :
                pass
        elif bdf3['temperature'][i] >= -5.0 :
            abnormalTemperatureState = False

    lackFaceValueCount = 0
    lackFaceValueState = False

    for i in range(len(bdf3)): # 액면부족 횟수
        if bdf3['faceValue'][i] == 0:
            if lackFaceValueState == False :
                lackFaceValueCount = lackFaceValueCount + 1
                lackFaceValueState = True
            else :
                pass
        else :
            lackFaceValueState = False
        
    if lackFaceValueCount > 0 :
        lackFaceValueCount = lackFaceValueCount - 1
    else :
        pass
    
    insufficient_count = 0
    for i in range(len(bdf3)-1):
        if bdf3['chargeDischarge'][i] == 1:
            if bdf3['chargeDischarge'][i+1] != 1 and bdf3['voltage'][i] <= 58.0 :
                insufficient_count = insufficient_count + 1
            else :
                pass
        else :
            pass
    

    chargeDischarge_count = 0
    chargeDischarge_cycle = 0 # 충방전 사이클
    chargeDischarge_last = bdf3['new_chargeDischarge'][0]

    for i in range(len(bdf3)-1):# 충방전 사이클 30분이상 유지시 1사이클
        if bdf3['new_chargeDischarge'][i] != bdf3['new_chargeDischarge'][i+1] :
            if chargeDischarge_count >= 30 and chargeDischarge_last != bdf3['new_chargeDischarge'][i+1] :
                chargeDischarge_cycle = chargeDischarge_cycle + 1
                chargeDischarge_count = 0 
                chargeDischarge_last = bdf3['new_chargeDischarge'][i+1]
            elif chargeDischarge_count >= 30 and chargeDischarge_last == bdf3['new_chargeDischarge'][i+1] :
                chargeDischarge_count = 0 
                chargeDischarge_last = bdf3['new_chargeDischarge'][i+1]
            elif chargeDischarge_count < 30:
                chargeDischarge_count = 0 
            else :
                pass
        elif bdf3['new_chargeDischarge'][i] == bdf3['new_chargeDischarge'][i+1] :
            chargeDischarge_count = chargeDischarge_count  + 1
        else :
            pass
        
    bdf3['overChargeCount'] = overChargeCount
    bdf3['overdisChargeCount'] = overdisChargeCount
    bdf3['abnormalTemperatureCount'] = abnormalTemperatureCount
    bdf3['lackFaceValueCount'] = lackFaceValueCount
    bdf3['insufficientChargeCount'] = insufficient_count
    bdf3['chargeDischargeCycle'] = math.floor(chargeDischarge_cycle/2)
    
    rmd = bdf3[(bdf3['voltaic'] < 1) & (bdf3['voltaic'] > -1)].index #전류값이 -1에서 1일때의 값들 삭제 -> 삭제 안되도록 변경 
    rmdf = bdf3.drop(rmd) #오류 값 제거

    bdf4 = rmdf.reset_index() #인덱스 값 새로 설정  
    
    dir_path = r'C:\Users\sm02\파이썬\battery' #생성할 폴더 위치

    global path_list
    global path_list1
    global num

    path_list = []
    num = []

    for sensor,hour in bdf4.groupby('sensorID'):
        num.append(sensor)
        big_path = dir_path +'/'+ str(sensor) +'/'
        path_list.append(big_path)

        try:
            os.mkdir(big_path) #폴더 생성
        except FileExistsError:
            pass

    path_list1 = []
    num = []

    for date,hour in bdf4.groupby(['date','sensorID']):  #파일 날짜 별로 분리 
        num.append(date)
        small_path = dir_path +'/' + date[1] +'/'+ date[0] +'/'
        path_list1.append(small_path)

        try:
            os.mkdir(small_path) #폴더 생성
        except FileExistsError:
            pass

        hour = hour.sort_values(by=['hour','minute'])
        hour.to_csv(small_path + str(date[0])+'.csv',mode = 'w')
        rm = pd.read_csv(small_path + str(date[0])+'.csv',encoding = 'cp949') #cp949 또는 utf-8-sig
        group_df = rm.groupby(['date','hour','sensorID','chargeDischarge']).mean() #시간 기준 평균값 구하기
        group_df.to_csv(small_path + 'df-'+str(date[0])+'.csv',encoding='cp949',mode = 'w')

    return
preprocess('f008d170d6ea.csv') #관제서버에서 받아온 csv입력
    
def prediction_bat(brand, capacity):
    
    if os.path.exists(path_list[0]+'Hourresult.csv') : # 중복된 파일 있으면 삭제
        os.remove(path_list[0]+'Hourresult.csv')
        time.sleep(0.5)
    else :
        pass

    for j in range(len(num)):
        
        rm = pd.read_csv(path_list1[j] + str(num[j][0]) +'.csv',encoding = 'cp949') #cp949 또는 utf-8-sig
        df3 = pd.read_csv(path_list1[j] + 'df-'+ str(num[j][0]) +'.csv',encoding='cp949')
        
        R_list = []
        DI = [] #방전전류
        DI1 = [] # 길이 맞춤 값 저장용
        DT = [] #방전온도
        DT1 = [] # 길이 맞춤 값 저장용
        CI = [] #충전전전류
        CI1 = [] # 길이 맞춤 값 저장용
        CT = [] #충전온도
        CT1 = [] # 길이 맞춤 값 저장용
        DV = [] #방전전압
        DV1 = [] # 길이 맞춤 값 저장용
        CV = [] #충전전압
        CV1 = [] # 길이 맞춤 값 저장용
        FVH_list = []
        FVD_list = []

        NQ = 1  # 국가별 품질 계수(National Quality) <- 추후 수정
#         BV = int(capacity)  # 배터리 표준용량(Battery Volume)
        BV = batteryCapacity # 배터리 표준용량(Battery Volume)
        charge_list = []
        charge_list = rm['chargeDischarge']
        n = 1
        charge_count = 0
        Discharge_time = 0 #방전시간
        Charge_time = 0 #충전시간
        Suspend_time = 0 #휴지시간
        sensor = df3['sensorID'][0] #센서번호 저장 
        
        locX = rm['locX'][0]
        locY = rm['locY'][0] # 위치좌표 저장
        
        for i in range(len(rm)):
            if( rm['chargeDischarge'][i] == 2):
                DI.append(rm['voltaic'][i]) #방전시 전류 (시)
                DT.append(rm['temperature'][i]) #방전시 온도 (시)
                DV.append(rm['voltage'][i]) #방전시 전압 (시)
                DI1.append(rm['voltaic'][i]) 
                DT1.append(rm['temperature'][i]) 
                DV1.append(rm['voltage'][i]) 

                CI1.append(0)
                CT1.append(0)
                CV1.append(0)
            elif(rm['chargeDischarge'][i] == 1):
                DI1.append(0)
                DT1.append(0)
                DV1.append(0)

                CI.append(rm['voltaic'][i]) #충전시 전류 (시)
                CT.append(rm['temperature'][i]) #충전시 온도 시(시)
                CV.append(rm['voltage'][i]) #충전시 온도 평균(시)
                CI1.append(rm['voltaic'][i]) 
                CT1.append(rm['temperature'][i]) 
                CV1.append(rm['voltage'][i]) 

            else:
                pass

        try: # 1일 평균값 저장
            DID = round(sum(DI)/len(DI),1) #방전시 전류 평균(일)

        except ZeroDivisionError: # 1일 평균값 저장
            DID = round(sum(DI)/1,1) #방전시 전류 평균(일)

        try: # 1일 평균값 저장    
            DTD = round(sum(DT)/len(DT),1) #방전시 온도 평균(일)

        except ZeroDivisionError: # 1일 평균값 저장
            DTD = round(sum(DT)/1,1) #방전시 온도 평균(일)

        try: # 1일 평균값 저장
            DVD = round(sum(DV)/len(DV),1) #방전시 전압 평균(일)

        except ZeroDivisionError: # 1일 평균값 저장
            DVD = round(sum(DV)/1,1) #방전시 전압 평균(일)

        try: # 1일 평균값 저장
            CID = round(sum(CI)/len(CI),1) #충전시 전류 평균(일)

        except ZeroDivisionError: # 1일 평균값 저장
            CID = round(sum(CI)/1,1) #충전시 전류 평균(일)

        try: # 1일 평균값 저장
            CTD = round(sum(CT)/len(CT),1) #충전시 온도 평균(일)

        except ZeroDivisionError: # 1일 평균값 저장
            CTD = round(sum(CT)/1,1) #충전시 온도 평균(일)

        try: # 1일 평균값 저장
            CVD = round(sum(CV)/len(CV),1) #충전시 전압 평균(일)

        except ZeroDivisionError: # 1일 평균값 저장
            CVD = round(sum(CV)/1,1) #충전시 전압 평균(일)

        hour_DI = [] #방전전류
        hour_DI1 = [] # 길이 맞춤 값 저장용
        hour_DT = [] #방전온도
        hour_DT1 = [] # 길이 맞춤 값 저장용
        hour_CI = [] #충전전전류
        hour_CI1 = [] # 길이 맞춤 값 저장용
        hour_CT = [] #충전온도
        hour_CT1 = [] # 길이 맞춤 값 저장용
        hour_DV = [] #방전전압
        hour_DV1 = [] # 길이 맞춤 값 저장용
        hour_CV = [] #충전전압
        hour_CV1 = [] # 길이 맞춤 값 저장용

        for i in range(len(df3)): #시간 평균의 평균
            if( df3['chargeDischarge'][i] == 2):
                hour_DI.append(df3['voltaic'][i]) #방전시 전류 (시)
                hour_DT.append(df3['temperature'][i]) #방전시 온도 (시)
                hour_DV.append(df3['voltage'][i]) #방전시 전압 (시)
                hour_DI1.append(df3['voltaic'][i]) 
                hour_DT1.append(df3['temperature'][i]) 
                hour_DV1.append(df3['voltage'][i]) 

                hour_CI1.append(0)
                hour_CT1.append(0)
                hour_CV1.append(0)
            elif(df3['chargeDischarge'][i] == 1):
                hour_DI1.append(0)
                hour_DT1.append(0)
                hour_DV1.append(0)

                hour_CI.append(df3['voltaic'][i]) #충전시 전류 (시)
                hour_CT.append(df3['temperature'][i]) #충전시 온도 시(시)
                hour_CV.append(df3['voltage'][i]) #충전시 온도 평균(시)
                hour_CI1.append(df3['voltaic'][i]) 
                hour_CT1.append(df3['temperature'][i]) 
                hour_CV1.append(df3['voltage'][i]) 
            else :
                pass
            
        try: # 1일 평균값 저장
            hour_DID = round(sum(hour_DI)/len(hour_DI),1) #방전시 전류 평균(일)

        except ZeroDivisionError: # 1일 평균값 저장
            hour_DID = round(sum(hour_DI)/1,1) #방전시 전류 평균(일)

        try: # 1일 평균값 저장    
            hour_DTD = round(sum(hour_DT)/len(hour_DT),1) #방전시 온도 평균(일)

        except ZeroDivisionError: # 1일 평균값 저장
            hour_DTD = round(sum(hour_DT)/1,1) #방전시 온도 평균(일)

        try: # 1일 평균값 저장
            hour_DVD = round(sum(hour_DV)/len(hour_DV),1) #방전시 전압 평균(일)

        except ZeroDivisionError: # 1일 평균값 저장
            hour_DVD = round(sum(hour_DV)/1,1) #방전시 전압 평균(일)

        try: # 1일 평균값 저장
            hour_CID = round(sum(hour_CI)/len(hour_CI),1) #충전시 전류 평균(일)

        except ZeroDivisionError: # 1일 평균값 저장
            hour_CID = round(sum(hour_CI)/1,1) #충전시 전류 평균(일)

        try: # 1일 평균값 저장
            hour_CTD = round(sum(hour_CT)/len(hour_CT),1) #충전시 온도 평균(일)

        except ZeroDivisionError: # 1일 평균값 저장
            hour_CTD = round(sum(hour_CT)/1,1) #충전시 온도 평균(일)

        try: # 1일 평균값 저장
            hour_CVD = round(sum(hour_CV)/len(hour_CV),1) #충전시 전압 평균(일)

        except ZeroDivisionError: # 1일 평균값 저장
            hour_CVD = round(sum(hour_CV)/1,1) #충전시 전압 평균(일)

        
        rm_date = []
        change_Date = []
        for i in range(len(rm)):
            rm_date.append(str(rm['date'][i]) + str(rm['hour'][i]) + str(rm['minute'][i]))
            change_Date.append(datetime.strptime(rm_date[i],'%Y%m%d%H%M')) # 년 월 일 데이터 변환

        OVC_list = []
        OVC_list_dup = []

        for i in range(len(rm)):
            if (rm['chargeDischarge'][i] == 1) and ( 120 < rm['soc'][i] ): #과충전
#                 OVC_list.append(str(rm['hour'][i])+'h'+ str(rm['minute'][i])) 
                OVC_list.append(change_Date[i].strftime('%y')+change_Date[i].strftime('%m')+change_Date[i].strftime('%d')+change_Date[i].strftime('%H')+change_Date[i].strftime('%M'))
            else:
                pass
            
        for i in OVC_list:
            if i not in OVC_list_dup:
                OVC_list_dup.append(i)
            else:
                pass #중복 제거 

        OC_count = sum(df3['overChargeCount'])/len(df3['overChargeCount']) # 과충전 횟수
            
        OVC = ''
        for i in range(len(rm)):
            if 0 <= len(OVC_list_dup)//60 < 10 and 0 <= len(OVC_list_dup)-((len(OVC_list_dup)//60)*60) < 10 :
                OVC = change_Date[i].strftime('%y')+'/'+change_Date[i].strftime('%m')+'/'+change_Date[i].strftime('%d')+'@'+str(0)+str(round(len(OVC_list_dup)//60))+'h'+str(0)+str(len(OVC_list_dup)-((len(OVC_list_dup)//60)*60))
            elif 0 <= len(OVC_list_dup)//60 < 10 and len(OVC_list_dup)-((len(OVC_list_dup)//60)*60) >= 10 :
                OVC = change_Date[i].strftime('%y')+'/'+change_Date[i].strftime('%m')+'/'+change_Date[i].strftime('%d')+'@'+str(0)+str(round(len(OVC_list_dup)//60))+'h'+str(len(OVC_list_dup)-((len(OVC_list_dup)//60)*60))
            elif len(OVC_list_dup)//60 > 10 and 0 < len(OVC_list_dup)-((len(OVC_list_dup)//60)*60) < 10 :
                OVC = change_Date[i].strftime('%y')+'/'+change_Date[i].strftime('%m')+'/'+change_Date[i].strftime('%d')+'@'+str(round(len(OVC_list_dup)//60))+'h'+str(0)+str(len(OVC_list_dup)-((len(OVC_list_dup)//60)*60))
            else :
                OVC = change_Date[i].strftime('%y')+'/'+change_Date[i].strftime('%m')+'/'+change_Date[i].strftime('%d')+'@'+str(round(len(OVC_list_dup)//60))+'h'+str(len(OVC_list_dup)-((len(OVC_list_dup)//60)*60))
            # 데이터양을 시간으로 변환 
            
        if OVC[9:14] == "00h00" : # 시간이 0이면 공백 
            OVC = ' '
        else :
            pass            
            
        ODC_list = []
        ODC_list_dup = []

        for i in range(len(rm)):
            if (rm['chargeDischarge'][i] == 2) and (rm['soc'][i] < 20): #과방전
                ODC_list.append(str(rm['hour'][i])+'h'+ str(rm['minute'][i]))
            else:
                pass
            
        for i in ODC_list:
            if i not in ODC_list_dup:
                ODC_list_dup.append(i)
            else:
                pass #중복 제거

        ODC_count = sum(df3['overdisChargeCount'])/len(df3['overdisChargeCount'])

        ODC = ''
        for i in range(len(rm)):
            if 0 <= len(ODC_list_dup)//60 < 10 and 0 <= len(ODC_list_dup)-((len(ODC_list_dup)//60)*60) < 10 :
                ODC = change_Date[i].strftime('%y')+'/'+change_Date[i].strftime('%m')+'/'+change_Date[i].strftime('%d')+'@'+str(0)+str(round(len(ODC_list_dup)//60))+'h'+str(0)+str(len(ODC_list_dup)-((len(ODC_list_dup)//60)*60))
            elif 0 <= len(ODC_list_dup)//60 < 10 and len(ODC_list_dup)-((len(ODC_list_dup)//60)*60) >= 10 :
                ODC = change_Date[i].strftime('%y')+'/'+change_Date[i].strftime('%m')+'/'+change_Date[i].strftime('%d')+'@'+str(0)+str(round(len(ODC_list_dup)//60))+'h'+str(len(ODC_list_dup)-((len(ODC_list_dup)//60)*60))
            elif len(ODC_list_dup)//60 > 10 and 0 < len(ODC_list_dup)-((len(ODC_list_dup)//60)*60) < 10 :
                ODC = change_Date[i].strftime('%y')+'/'+change_Date[i].strftime('%m')+'/'+change_Date[i].strftime('%d')+'@'+str(round(len(ODC_list_dup)//60))+'h'+str(0)+str(len(ODC_list_dup)-((len(ODC_list_dup)//60)*60))
            else :
                ODC = change_Date[i].strftime('%y')+'/'+change_Date[i].strftime('%m')+'/'+change_Date[i].strftime('%d')+'@'+str(round(len(ODC_list_dup)//60))+'h'+str(len(ODC_list_dup)-((len(ODC_list_dup)//60)*60))
            # 데이터양을 시간으로 변환 
                
        if ODC[9:14] == "00h00" : # 시간이 0이면 공백 
            ODC = ' '
        else :
            pass

        AT_list = []
        AT_list_dup=[]

        for i in range(len(rm)):
            if (rm['temperature'][i] >= 55) or (rm['temperature'][i] <= -15): #온도이상
                AT_list.append(str(rm['hour'][i])+'h'+ str(rm['minute'][i]))
            else:
                pass
            
        for i in AT_list:
            if i not in AT_list_dup:
                AT_list_dup.append(i)
            else:
                pass # 중복 제거 
        
        AT_count = sum(df3['abnormalTemperatureCount'])/len(df3['abnormalTemperatureCount'])
        
        AT = ''
        for i in range(len(rm)):
            if 0 <= len(AT_list_dup)//60 < 10 and 0 <= len(AT_list_dup)-((len(AT_list_dup)//60)*60) < 10 :
                AT = change_Date[i].strftime('%y')+'/'+change_Date[i].strftime('%m')+'/'+change_Date[i].strftime('%d')+'@'+str(0)+str(round(len(AT_list_dup)//60))+'h'+str(0)+str(len(AT_list_dup)-((len(AT_list_dup)//60)*60))
            elif 0 <= len(AT_list_dup)//60 < 10 and len(AT_list_dup)-((len(AT_list_dup)//60)*60) >= 10 :
                AT = change_Date[i].strftime('%y')+'/'+change_Date[i].strftime('%m')+'/'+change_Date[i].strftime('%d')+'@'+str(0)+str(round(len(AT_list_dup)//60))+'h'+str(len(AT_list_dup)-((len(AT_list_dup)//60)*60))
            elif len(AT_list_dup)//60 > 10 and 0 < len(AT_list_dup)-((len(AT_list_dup)//60)*60) < 10 :
                AT = change_Date[i].strftime('%y')+'/'+change_Date[i].strftime('%m')+'/'+change_Date[i].strftime('%d')+'@'+str(round(len(AT_list_dup)//60))+'h'+str(0)+str(len(AT_list_dup)-((len(AT_list_dup)//60)*60))
            else :
                AT = change_Date[i].strftime('%y')+'/'+change_Date[i].strftime('%m')+'/'+change_Date[i].strftime('%d')+'@'+str(round(len(AT_list_dup)//60))+'h'+str(len(AT_list_dup)-((len(AT_list_dup)//60)*60))
         # 데이터양을 시간으로 변환 
        
        if AT[9:14] == "00h00" : # 시간이 0이면 공백 
            AT = ' '
        else :
            pass
    
        insufficient_time = 0 
        insufficient_list = []
        for i in range(len(rm)-1): #부족 충전 시간
            if rm['chargeDischarge'][i] == rm['chargeDischarge'][i+1] : 
                insufficient_time = insufficient_time + 1
                if rm['chargeDischarge'][i] == 1 and insufficient_time == 300 and rm['soc'][i] < 90: # 부족 충전 조건 비교
                    insufficient_list.append(str(rm['hour'][i])+'h'+str(rm['minute'][i]))
            elif rm['chargeDischarge'][i] != rm['chargeDischarge'][i+1] :
                insufficient_time = 0

        insufficient_count = sum(df3['insufficientChargeCount'])/len(df3['insufficientChargeCount'])        

        for i in range(len(rm)):
            if rm['chargeDischarge'][i] == 1: # 충전시간
                Charge_time = Charge_time + 1
            elif rm['chargeDischarge'][i] == 2: # 방전시간
                Discharge_time = Discharge_time + 1

        Ctime = round(Charge_time / 60,5)
        Dtime = round(Discharge_time / 60,5)

        Stime = 24 - ( Dtime + Ctime )

    #         OR = (Dtime * 64.5) / 5 #운행율 기존
        OperationRate = (Dtime / 5) * 100 #운행율

        BU = abs(DID * Dtime) #방전량 (방전 전류 * 방전시간)
        C = abs(CID *  Ctime)
            
#         DA = BU /BV  # 용량 계수 (방전량/배터리 용량) 기존
        DA = abs(DID) / BV # CRate 계수(방전전류 / 배터리 용량)
        
        if DA < 0.3:
            DAC = 1.0
        elif DA >= 0.3 and DA < 0.4:
            DAC = 0.75
        elif DA >= 0.4 and DA < 0.6:
            DAC = 0.60
        elif DA >= 0.6 and DA < 0.8:
            DAC = 0.50
        elif DA >= 0.8 and DA < 1.0:
            DAC = 0.45
        elif DA >= 1.0:
            DAC = 0.40
        else:
            pass

#         FVH = 1 #len(FVH_list)    
        LFV_list = []
        LFV_list_dup = []
        LFV_count = 0
        for i in range(len(rm)):
            if (rm['faceValue'][i] == 1):
                FVD_list.append(rm['faceValue'][i]) # 액면 만족시간 (일)
            elif(rm['faceValue'][i] == 0):
                LFV_list.append(str(rm['hour'][i])+'h'+ str(rm['minute'][i]))
            else:
                pass

        for i in LFV_list:
            if i not in LFV_list_dup:
                LFV_list_dup.append(i)
            else:
                pass # 중복 제거
            
        LFV_count = sum(df3['lackFaceValueCount'])/len(df3['lackFaceValueCount'])
    
        LFV = ''
        for i in range(len(rm)):
            if 0 <= len(LFV_list_dup)//60 < 10 and 0 <= len(LFV_list_dup)-((len(LFV_list_dup)//60)*60) < 10 :
                LFV = change_Date[i].strftime('%y')+'/'+change_Date[i].strftime('%m')+'/'+change_Date[i].strftime('%d')+'@'+str(0)+str(round(len(LFV_list_dup)//60))+'h'+str(0)+str(len(LFV_list_dup)-((len(LFV_list_dup)//60)*60))
            elif 0 <= len(LFV_list_dup)//60 < 10 and len(LFV_list_dup)-((len(LFV_list_dup)//60)*60) >= 10 :
                LFV = change_Date[i].strftime('%y')+'/'+change_Date[i].strftime('%m')+'/'+change_Date[i].strftime('%d')+'@'+str(0)+str(round(len(LFV_list_dup)//60))+'h'+str(len(LFV_list_dup)-((len(LFV_list_dup)//60)*60))
            elif len(LFV_list_dup)//60 > 10 and 0 < len(LFV_list_dup)-((len(LFV_list_dup)//60)*60) < 10 :
                LFV = change_Date[i].strftime('%y')+'/'+change_Date[i].strftime('%m')+'/'+change_Date[i].strftime('%d')+'@'+str(round(len(LFV_list_dup)//60))+'h'+str(0)+str(len(LFV_list_dup)-((len(LFV_list_dup)//60)*60))
            else :
                LFV = change_Date[i].strftime('%y')+'/'+change_Date[i].strftime('%m')+'/'+change_Date[i].strftime('%d')+'@'+str(round(len(LFV_list_dup)//60))+'h'+str(len(LFV_list_dup)-((len(LFV_list_dup)//60)*60))
        # 데이터양을 시간으로 변환 
                
        if LFV[9:14] == "00h00" : # 시간이 0이면 공백 
            LFV = ' '
        else :
            pass
    
#         FVD = 1 #1440 - len(IT_list) # 1일 = 1440분
    
        ATM = []
        for i in range(len(df3)):
            if (df3['temperature'][i] >= 55) or (df3['temperature'][i] <= -15): #온도이상(일)
                ATM.append(str(df3['hour'][i]))
            else:
                pass

        BC = []
        for i in range(len(df3)):
            if(df3['faceValue'][i] != 1) or (df3['faceValue'][i] == 1 and ((len(ATM) >= 3) or len(OVC)>=180)):
                BC.append('Check')
            else:
                BC.append('Normal')
                pass
            
        ET = [] #온도
        ETC = [] #온도 계수
        for i in range(len(df3)):
            if(df3['chargeDischarge'][i] == 2):
                ET.append(df3['temperature'][i]) #방전시 온도
            else:
                ET.append(0)

            if ET[i] < 10:
                ETC.append(0.75)
            elif ET[i] >= 10 and ET[i] < 20:
                ETC.append(0.84)
            elif ET[i] >= 20 and ET[i] < 30:
                ETC.append(1.0)
            elif ET[i] >= 30 and ET[i] < 40:
                ETC.append(0.98)
            elif ET[i] >= 40 and ET[i] < 50:
                ETC.append(0.86)
            elif ET[i] >= 50 and ET[i] < 60:
                ETC.append(0.75)
            elif ET[i] >= 60:
                ETC.append(0.68)
            else:
                pass

        TVC = [] #온도 계수
        if DTD < -30:
            TVC.append(0.40)
        elif DTD >= -30 and DTD < -20:
            TVC.append(0.50)
        elif DTD >= -20 and DTD < -10:
            TVC.append(0.66)
        elif DTD >= -10 and DTD < 0:
            TVC.append(0.79)
        elif DTD >= 0 and DTD < 10:
            TVC.append(0.88)
        elif DTD >= 10 and DTD < 20:
            TVC.append(0.95)
        elif DTD >= 20 and DTD < 30:
            TVC.append(1.0)
        elif DTD >= 30 and DTD < 40:
            TVC.append(1.05)
        elif DTD >= 40 and DTD < 50:
            TVC.append(1.09)
        elif DTD >= 50 :
            TVC.append(1.12)


        DD = [] #DOD
        DDC = [] #DOD 계수

        for i in range(len(df3)): 
            DOD = ((BU*DAC*TVC[0])/BV) * 100 # ((방전량*용량계수*온도계수)/배터리 용량) * 100
            
            DD.append(round(DOD,1))

            if DD[i] < 40:
                DDC.append(1.5)
            elif DD[i] >= 40 and DD[i] < 50:
                DDC.append(1.42)
            elif DD[i] >= 50 and DD[i] < 60:
                DDC.append(1.25)
            elif DD[i] >= 60 and DD[i] < 70:
                DDC.append(1.08)
            elif DD[i] >= 70 and DD[i] < 80:
                DDC.append(0.92)
            elif DD[i] >= 80 and DD[i] < 90:
                DDC.append(0.75)
            elif DD[i] >= 90 : # and DD <= 100 
                DDC.append(0.58)
            else:
                pass

        OC = [] #제조사 공식 사이클
        #제조사 공식 사이클 1800 ->1200
        if brand == 'ROCKET1' or brand == 'ROCKET2':
            for i in range(len(df3)):
                if DD[i] < 60: #DOD 값의 따른 제조사공식 사이클
#                     OC.append(1800)
                    OC.append(1200)
                elif DD[i] >= 60 and DD[i] < 70:
#                     OC.append(1600)
                    OC.append(1200)
                elif DD[i] >= 70 and DD[i] < 80:
#                     OC.append(1400)
                    OC.append(1200)
                elif DD[i] >= 80 and DD[i] < 90:
                    OC.append(1200)
                elif DD[i] >= 90 and DD[i] < 100:
                    OC.append(1000)
                elif DD[i] >= 100:
                    OC.append(800)
                else:
                    pass
        elif brand == 'YUASA':
            for i in range(len(df3)):
                if  DD[i] < 60: #DOD 값의 따른 제조사공식 사이클
#                     OC.append(1800)
                    OC.append(1200)
                elif DD[i] >= 60 and DD[i] < 70:
#                     OC.append(1500)
                    OC.append(1200)
                elif DD[i] >= 70 and DD[i] < 80:
#                     OC.append(1300)
                    OC.append(1200)
                elif DD[i] >= 80 and DD[i] < 90:
                    OC.append(1100)
                elif DD[i] >= 90 and DD[i] < 100:
                    OC.append(900)
                elif DD[i] >= 100:
                    OC.append(700)
                else:
                    pass
        elif brand == 'GK':
            for i in range(len(df3)):
                if  DD[i] < 60: #DOD 값의 따른 제조사공식 사이클
#                     OC.append(2100)
                    OC.append(1200)
                elif DD[i] >= 60 and DD[i] < 70:
#                     OC.append(1700)
                    OC.append(1200)
                elif DD[i] >= 70 and DD[i] < 80:
#                     OC.append(1500)
                    OC.append(1200)
                elif DD[i] >= 80 and DD[i] < 90:
#                     OC.append(1300)
                    OC.append(1200)
                elif DD[i] >= 90 and DD[i] < 100:
                    OC.append(1100)
                elif DD[i] >= 100:
                    OC.append(800)
                else:
                    pass
        elif brand == 'KTB':
            for i in range(len(df3)):
                if  DD[i] < 60: #DOD 값의 따른 제조사공식 사이클
#                     OC.append(2100)
                    OC.append(1200)
                elif DD[i] >= 60 and DD[i] < 70:
#                     OC.append(1700)
                    OC.append(1200)
                elif DD[i] >= 70 and DD[i] < 80:
#                     OC.append(1500)
                    OC.append(1200)
                elif DD[i] >= 80 and DD[i] < 90:
#                     OC.append(1300)
                    OC.append(1200)
                elif DD[i] >= 90 and DD[i] < 100:
                    OC.append(1100)
                elif DD[i] >= 100:
                    OC.append(800)
                else:
                    pass
        else:
            pass

        f = 1 #보액상태
        if f == 1:
            DS = 1.0
        elif f == 0:
            DS = 0.9
        else:
            pass

        CR = C / BV #충전전류 <- 충전량/배터리 표준용량

        if CR < 90:
            CRC = 0.85
        elif CR >= 90 and CR < 100:
            CRC = 0.89
        elif CR >= 100 and CR < 105:
            CRC = 0.93
        elif CR >= 105 and CR < 110:
            CRC = 0.97
        elif CR >= 110 and CR < 120:
            CRC = 1.0
        elif CR >= 120 and CR < 125:
            CRC = 0.98
        elif CR >= 125 and CR < 130:
            CRC = 0.95
        elif CR >= 130 and CR < 135:
            CRC = 0.91
        else:
            pass

        reg_list = []
#         date = []
        for i in range(len(df3)):
            reg_list.append(str(df3['date'][i])+str(df3['hour'][i]))
#             date.append(str(datetime.strptime(reg_list[i],"%Y%m%d%H"))) #날짜 변환 작업

        SOC =list(map(int,df3['chargeDischarge'])) # SOC값 충방전값 int형으로 변환
        
        charge_soc_list = [] #충전 SOC 리스트
        discharge_soc_list = [] #방전 SOC 리스트
        for i in range(len(df3)):
            if df3['chargeDischarge'][i] == 1: #충전
                if(df3['voltage'][i] >= 51.1):
                    charge_soc_list.append(100)
                elif(df3['voltage'][i] >= 50.6 and df3['voltage'][i] < 51.1):
                    charge_soc_list.append(90)
                elif(df3['voltage'][i] >= 50.2 and df3['voltage'][i] < 50.6):
                    charge_soc_list.append(80)
                elif(df3['voltage'][i] >= 49.7 and df3['voltage'][i] < 50.2):
                    charge_soc_list.append(70)
                elif(df3['voltage'][i] >= 49.2 and df3['voltage'][i] < 49.7):
                    charge_soc_list.append(60)
                elif(df3['voltage'][i] >= 48.7 and df3['voltage'][i] < 49.2):
                    charge_soc_list.append(50)
                elif(df3['voltage'][i] >= 48.2 and df3['voltage'][i] < 48.7):
                    charge_soc_list.append(40)
                elif(df3['voltage'][i] >= 47.8 and df3['voltage'][i] < 48.2):
                    charge_soc_list.append(30)
                elif(df3['voltage'][i] >= 47.3 and df3['voltage'][i] < 47.8):
                    charge_soc_list.append(20)
                elif(df3['voltage'][i] >= 46.8 and df3['voltage'][i] < 47.3):
                    charge_soc_list.append(10)
                elif(df3['voltage'][i] < 46.8):
                    charge_soc_list.append(0)
                else:
                    pass
            elif df3['chargeDischarge'][i] == 2: #방전
                if(df3['voltage'][i] >= 51.1):
                    discharge_soc_list.append(100)
                elif(df3['voltage'][i] >= 50.6 and df3['voltage'][i] < 51.1):
                    discharge_soc_list.append(90)
                elif(df3['voltage'][i] >= 50.2 and df3['voltage'][i] < 50.6):
                    discharge_soc_list.append(80)
                elif(df3['voltage'][i] >= 49.7 and df3['voltage'][i] < 50.2):
                    discharge_soc_list.append(70)
                elif(df3['voltage'][i] >= 49.2 and df3['voltage'][i] < 49.7):
                    discharge_soc_list.append(60)
                elif(df3['voltage'][i] >= 48.7 and df3['voltage'][i] < 49.2):
                    discharge_soc_list.append(50)
                elif(df3['voltage'][i] >= 48.2 and df3['voltage'][i] < 48.7):
                    discharge_soc_list.append(40)
                elif(df3['voltage'][i] >= 47.8 and df3['voltage'][i] < 48.2):
                    discharge_soc_list.append(30)
                elif(df3['voltage'][i] >= 47.3 and df3['voltage'][i] < 47.8):
                    discharge_soc_list.append(20)
                elif(df3['voltage'][i] >= 46.8 and df3['voltage'][i] < 47.3):
                    discharge_soc_list.append(10)
                elif(df3['voltage'][i] < 46.8):
                    discharge_soc_list.append(0)
                else:
                    pass
                
        charge_soc = [] #충전 SOC    
        discharge_soc = [] #방전 SOC
        for i in range(len(discharge_soc_list)):
            if i == 0:        
                discharge_soc.append(np.round(discharge_soc_list[0] + (((DI[i]*-1) * 60)/(BV*3600))*100,1))
            else :
                discharge_soc.append(np.round(discharge_soc[i-1] + (((DI[i]*-1) * 60)/(BV*3600))*100,1))
        if len(discharge_soc) != 0:
            for i in range(len(charge_soc_list)):
                if i == 0:        
                    charge_soc.append(np.round(discharge_soc[-1] + ((-0.0007 * discharge_soc[-1] + 1.000716) * (abs(CI[i]) * 60)/(BV*3600))*100,1))
                else :
                    charge_soc.append(np.round(charge_soc[i-1] + ((-0.0007 * charge_soc[i-1] + 1.00716) * (abs(CI[i]) * 60)/(BV*3600))*100,1))
        elif len(discharge_soc) == 0:
            for i in range(len(charge_soc_list)):
                if i == 0:        
                    charge_soc.append(np.round(charge_soc_list[0] + ((-0.0007 * charge_soc_list[0] + 1.000716) * (abs(CI[i]) * 60)/(BV*3600))*100,1))
                else :
                    charge_soc.append(np.round(charge_soc[i-1] + ((-0.0007 * charge_soc[i-1] + 1.00716) * (abs(CI[i]) * 60)/(BV*3600))*100,1))
        else :
            pass 

        add_soc_list = []
        charge_soc_count  = 0
        discharge_soc_count  = 0
        for i in range(len(df3)):
            if df3['chargeDischarge'][i] == 2:
                add_soc_list.append(discharge_soc[discharge_soc_count])
                discharge_soc_count  = discharge_soc_count +1
            elif df3['chargeDischarge'][i] == 1:
                add_soc_list.append(charge_soc[charge_soc_count])
                charge_soc_count = charge_soc_count + 1                # 충방전 SOC 합치기
        
        add_soc = sum(df3['modify_soc'])/len(df3['modify_soc']) 
        
        cycle_count = 0
        CYCLE = 0
        for i in range(len(rm)-1): #사이클수 구하기 30분이상 충방전 했을때 1사이클
            if i == 0 :
                if rm['chargeDischarge'][i] == rm['chargeDischarge'][i+1]:
                    cycle_count = cycle_count + 1
            else :
                if rm['chargeDischarge'][i] == rm['chargeDischarge'][i-1]:
                    cycle_count = cycle_count + 1 
                elif rm['chargeDischarge'][i] != rm['chargeDischarge'][i-1]:
                    if cycle_count > 30 :
                        CYCLE = CYCLE + 1 
                        cycle_count = 0
                    else :
                        cycle_count = 0
                else :
                    pass

        CYCLE = CYCLE / 2
    
        global R
        
        
        if CYCLE < 1:
            R = 1
        else:
            R = math.trunc(CYCLE)

        R_list.append(float(R))

        Max = [] # MAX 예상수명 월
        Max_cycle = [] # MAX 예상수명 CYCLE
        for i in range(len(df3)):
            Max.append(40)
            Max_cycle.append(1200) # 현대 요구사항

#             Max.append(round((OC[i]*30*12/(30*(1+(7/30)*0)*1*220))*NQ,0))
#             Max_cycle.append(round(OC[i]*30/(30*(1+(7/30)*0))*NQ, 0))
        
        try:
            work_temperature = sum(DT)/len(DT) # 작업시 온도
            average_current = round(ln(abs(sum(DI)/len(DI))/BV * 5),1) # 방전 전류
        except ZeroDivisionError :
            work_temperature = sum(DT)/1
            average_current = round(ln(abs(sum(DI)/1)/BV * 5),1)
        
        if sum(DI) == 0 or average_current < 0 :
            average_current = 0

        if work_temperature < 30 :
            work_temperature = 30
        else :
            pass

        charging_rate = [] # 충전 효율
        for i in range(len(charge_soc_list)):
            if i == 0:
                charging_rate.append(-0.0007 * charge_soc_list[0] + 1.000716)
            else :
                charging_rate.append(-0.0007 * charge_soc[i-1] + 1.00716)
        
        charging_rate_C = [] # 충전 효율 계수
        for i in range(len(charge_soc_list)):
            if charging_rate[i] >= 135 :
                charging_rate_C.append(0.88)
            elif charging_rate[i] >= 130 and charging_rate[i] < 135:
                charging_rate_C.append(0.91)
            elif charging_rate[i] >= 125 and charging_rate[i] < 130:
                charging_rate_C.append(0.95)
            elif charging_rate[i] >= 120 and charging_rate[i] < 125:
                charging_rate_C.append(0.98)
            elif charging_rate[i] >= 115 and charging_rate[i] < 120:
                charging_rate_C.append(1.00)
            elif charging_rate[i] >= 110 and charging_rate[i] < 115:
                charging_rate_C.append(0.98)
            elif charging_rate[i] >= 105 and charging_rate[i] < 110:
                charging_rate_C.append(0.97)
            elif charging_rate[i] >= 100 and charging_rate[i] < 105:
                charging_rate_C.append(0.93)
            elif charging_rate[i] >= 90 and charging_rate[i] < 100:
                charging_rate_C.append(0.89)
            elif charging_rate[i] < 90:
                charging_rate_C.append(0.85)
                
        facevalue_C = 1 -(0 * DS) # 보액 상태 계수
    
        dod_cycle_month = []
        dod_cycle = []
        for i in range(len(charge_soc_list)):
            dod_cycle_month.append(round((((1200*sum(DDC)/len(DDC))*30*12/(work_temperature*(1+(7/30)*average_current)*R*240))*1*charging_rate_C[i]*facevalue_C),0))
            dod_cycle.append(round((((1200*sum(DDC)/len(DDC))*30/(work_temperature*(1+(7/30)*average_current)))*1*charging_rate_C[i]*facevalue_C),0))

        count = 0
        lack = 0 # 부족 충전(시간 단위)
        for i in range(len(df3)-1):
            if df3['chargeDischarge'][i] == df3['chargeDischarge'][i+1] :
                count = count + 1
                if count == 5:
                    lack = i+1
            else : 
                count = 0

        insufficient = 0 #부족 충전
        if add_soc_list[lack] < 90 :
            insufficient = insufficient + 1
        else : 
            insufficient = 0
            
        FVH = []
        for i in range(len(df3)):
            FVH.append(df3['faceValue'][i])

        FVD = sum(FVH)/len(FVH)

        totalCycle = []
        totalCycle.append(sum(df3['chargeDischargeCycle'])/len(df3['chargeDischargeCycle'])) #누적 사이클
    
        FN = [] #∫n
        for i in range(len(df3)):
            BL = OC[i] * NQ  # 기초수명
            FN.append(BL * DDC[i] * CRC * ETC[i] * DS)  # * (T/24)

        for i in range(len(df3)):    
            with open(path_list1[j] +'sample1.txt', 'a',encoding = "UTF-8") as f:
                f.write(str(FN[i]) + '\n')  # ∫n값 저장

        with open(path_list1[j] + 'sample1.txt', 'r',encoding = "UTF-8") as f:
            lines = f.readlines()  # TXT파일 읽기

        result = 0
        for i in lines:
            result += float(i) #∫n값 누적시기

        col_name = ['sensorID'] #첫 열 생성
    
        sensor_ID = []
        
        hour_date = []
        for i in range(len(reg_list)):
            hour_date.append(str(datetime.strptime(str(reg_list[i]),"%Y%m%d%H"))) # 날자 변환
            hour_date[i] = hour_date[i].replace(' ','T')    

        for i in range(len(df3)):
            sensor_ID.append(sensor)

        hour_df = pd.DataFrame(list(sensor_ID), columns = col_name) #데이터 프레임 생성

        hour_df['date'] = hour_date
        hour_df['chargedTime'] = np.round(Ctime,1)
        hour_df['dischargedTime'] = np.round(Dtime,1)
        hour_df['voltaicCharged'] = np.round(hour_CI1,1)
        hour_df['voltaicDischarged'] = np.round(hour_DI1,1)
        hour_df['voltageCharged'] = np.round(hour_CV1,1)
        hour_df['voltageDischarged'] = np.round(hour_DV1,1)
        hour_df['temperatureCharged'] = np.round(hour_CT1,1)
        hour_df['temperatureDischarged'] = np.round(hour_DT1,1)
        hour_df['liquidStatus'] = FVH
        hour_df['dod'] = DD

        if os.path.exists(path_list[0]+'Hourresult.csv'):
            hour_df.to_csv(path_list[0]+'Hourresult.csv',index=False, mode = 'a',header = False )
        else :
            hour_df.to_csv(path_list[0]+'Hourresult.csv',index=False, mode = 'w')    #기존의 있는 csv와 합쳐서 저장 
        
#         if not os.path.exists(path_list[0]+'Hourresult.csv'):
#             hour_df.to_csv(path_list[0]+'Hourresult.csv')
#         elif os.path.exists(path_list[0]+'Hourresult.csv'):
#             hour_df1 = pd.read_csv(path_list[0] + 'Hourresult.csv',encoding = 'cp949')
#             Hdf = pd.concat([hour_df,hour_df1],ignore_index=True,sort=True)
#             del Hdf['Unnamed: 0']
#             os.remove(path_list[0] + 'Hourresult.csv')
#             Hdf.to_csv(path_list[0] + 'Hourresult.csv')
#         else :
#             pass #기존의 있는 csv와 합쳐서 저장 
        
        if len(dod_cycle) == 0:
            dod_cycle.append(1)
        if len(dod_cycle_month) == 0:
            dod_cycle_month.append(1)

        if len(df3) == 1:
            for i in range(len(df3)):
                if(SOC[i] == 2): #방전시 
                    with open(path_list1[j] + 'sample3.txt', 'a',encoding = "UTF-8") as h:
                        h.write(str(sensor) +'\t'+ reg_list[i] +'\t'+ reg_list[i] +'\t'+ str(int(SOC[i])) +'\t'+ str(Dtime) +'\t'+ str(add_soc) +'\t'+ str(Ctime) +'\t'+ str(Stime) +'\t'+
                        str(round(DI1[i],2)) +'\t'+ str(round(DID,2)) +'\t'+ str(round(DV1[i],2)) +'\t'+ str(DVD) +'\t'+str(round(DT1[i],2))+'\t'+str(DTD) +'\t'+ str(FVH) +'\t'+
                        str(FVD) +'\t'+ str(DD[i]) +'\t'+  str(round(sum(dod_cycle)/len(dod_cycle),0)) +'\t'+ str(R) +'\t'+ str(round(sum(dod_cycle_month)/len(dod_cycle_month),0)) +'\t'+
                        str(round(OperationRate,0)) +'\t'+ str(BL) +'\t'+ str(OVC) +'\t'+ str(len(OVC_list_dup)) +'\t'+ str(ODC) +'\t'+ str(len(ODC_list_dup)) +'\t'+ str(AT) +'\t'+str(len(AT_list_dup))+'\t'+
                        str(LFV) +'\t'+ str(len(LFV_list)) +'\t'+ str(locX) +'\t'+ str(locY) +'\t'+ str(BC[i]) +'\t'+str(insufficient)+'\t'+str(len(insufficient_list))+'\t'+
                        str(OC_count) +'\t'+ str(ODC_count) +'\t'+ str(AT_count) +'\t'+ str(LFV_count) +'\t'+ str(insufficient_count) +'\t'+ str(Max[i])+'\t'+str(Max_cycle[i])+'\t'+ 
                        str(totalCycle[0]) +'\n') #JSON변환용 값 저장

                elif(SOC[i] == 1): #충전시
                    with open(path_list1[j] + 'sample3.txt', 'a',encoding = "UTF-8") as h:
                        h.write(str(sensor) +'\t'+ reg_list[i] +'\t'+ reg_list[i] +'\t'+ str(int(SOC[i])) +'\t'+ str(Dtime) +'\t'+ str(add_soc) +'\t'+ str(Ctime) +'\t'+  str(Stime) +'\t'+
                        str(round(CI1[i],2)) +'\t'+ str(round(CID,2)) +'\t'+ str(round(CV1[i],2)) +'\t'+ str(CVD) +'\t'+str(round(CT1[i],2))+'\t'+str(CTD) +'\t'+ str(FVH) +'\t'+
                        str(FVD) +'\t'+ str(DD[i]) +'\t'+ str(round(sum(dod_cycle)/len(dod_cycle),0)) +'\t'+ str(R) +'\t'+ str(round(sum(dod_cycle_month)/len(dod_cycle_month),0)) +'\t'+
                        str(round(OperationRate,0)) +'\t'+str(BL) +'\t'+str(OVC) +'\t'+ str(len(OVC_list_dup)) +'\t'+ str(ODC) +'\t'+ str(len(ODC_list_dup)) +'\t'+ str(AT) +'\t'+ str(len(AT_list_dup))+'\t'+
                        str(LFV) +'\t'+ str(len(LFV_list)) +'\t'+ str(locX) +'\t'+ str(locY) +'\t'+ str(BC[i]) +'\t'+str(insufficient)+'\t'+str(len(insufficient_list))+'\t'+
                        str(OC_count) +'\t'+ str(ODC_count) +'\t'+ str(AT_count) +'\t'+ str(LFV_count) +'\t'+ str(insufficient_count) +'\t'+ str(Max[i])+'\t'+str(Max_cycle[i])+'\t'+ 
                        str(totalCycle[0]) +'\n') #JSON변환용 값 저장
                else:
                    pass
        else :
            for i in range(len(df3)-1):
                if(SOC[i] == 2): #방전시 
                    with open(path_list1[j] + 'sample3.txt', 'a',encoding = "UTF-8") as h:
                        h.write(str(sensor) +'\t'+ reg_list[i] +'\t'+ reg_list[i+1] +'\t'+ str(int(SOC[i])) +'\t'+ str(Dtime) +'\t'+ str(add_soc) +'\t'+ str(Ctime) +'\t'+ str(Stime) +'\t'+
                        str(round(DI1[i],2)) +'\t'+ str(round(DID,2)) +'\t'+ str(round(DV1[i],2)) +'\t'+ str(DVD) +'\t'+str(round(DT1[i],2))+'\t'+str(DTD) +'\t'+ str(FVH) +'\t'+
                        str(FVD) +'\t'+ str(DD[i]) +'\t'+ str(round(sum(dod_cycle)/len(dod_cycle),0)) +'\t'+ str(R) +'\t'+ str(round(sum(dod_cycle_month)/len(dod_cycle_month),0)) +'\t'+
                        str(round(OperationRate,0)) +'\t'+ str(BL) +'\t'+ str(OVC) +'\t'+ str(len(OVC_list_dup)) +'\t'+ str(ODC) +'\t'+ str(len(ODC_list_dup)) +'\t'+ str(AT) +'\t'+str(len(AT_list_dup))+'\t'+
                        str(LFV) +'\t'+ str(len(LFV_list)) +'\t'+ str(locX) +'\t'+ str(locY) +'\t'+ str(BC[i]) +'\t'+str(insufficient)+'\t'+str(len(insufficient_list))+'\t'+
                        str(OC_count) +'\t'+ str(ODC_count) +'\t'+ str(AT_count) +'\t'+ str(LFV_count) +'\t'+ str(insufficient_count) +'\t'+ str(Max[i])+'\t'+str(Max_cycle[i])+ '\t'+ 
                        str(totalCycle[0]) +'\n') #JSON변환용 값 저장

                elif(SOC[i] == 1): #충전시
                    with open(path_list1[j] + 'sample3.txt', 'a',encoding = "UTF-8") as h:
                        h.write(str(sensor) +'\t'+ reg_list[i] +'\t'+ reg_list[i+1] +'\t'+ str(int(SOC[i])) +'\t'+ str(Dtime) +'\t'+ str(add_soc) +'\t'+ str(Ctime) +'\t'+  str(Stime) +'\t'+
                        str(round(CI1[i],2)) +'\t'+ str(round(CID,2)) +'\t'+ str(round(CV1[i],2)) +'\t'+ str(CVD) +'\t'+str(round(CT1[i],2))+'\t'+str(CTD) +'\t'+ str(FVH) +'\t'+
                        str(FVD) +'\t'+ str(DD[i]) +'\t'+ str(round(sum(dod_cycle)/len(dod_cycle),0)) +'\t'+ str(R) +'\t'+ str(round(sum(dod_cycle_month)/len(dod_cycle_month),0)) +'\t'+
                        str(round(OperationRate,0)) +'\t'+str(BL) +'\t'+str(OVC) +'\t'+ str(len(OVC_list_dup)) +'\t'+ str(ODC) +'\t'+ str(len(ODC_list_dup)) +'\t'+ str(AT) +'\t'+ str(len(AT_list_dup))+'\t'+
                        str(LFV) +'\t'+ str(len(LFV_list)) +'\t'+ str(locX) +'\t'+ str(locY) +'\t'+ str(BC[i]) +'\t'+str(insufficient)+'\t'+str(len(insufficient_list))+'\t'+
                        str(OC_count) +'\t'+ str(ODC_count) +'\t'+ str(AT_count) +'\t'+ str(LFV_count) +'\t'+ str(insufficient_count) +'\t'+ str(Max[i])+'\t'+str(Max_cycle[i])+'\t'+ 
                        str(totalCycle[0]) +'\n') #JSON변환용 값 저장
                else:
                    pass
            
    return

prediction_bat('ROCKET2', '565') #배터리 명판 파라미터 입력


def el1(txtname):
    
    DIMC = []
    CIMC = []
    DVMC = []
    CVMC = []
    DTMC = []
    CTMC = []
    FVMC = [] 
    DODMC = [] 
    CYCMC = []
    CYCSMC = [] 
    DTimeMC = []
    CTimeMC = [] #계산용 리스트 
    STimeMC = []
    
#     batteryCapacity = 390.0
    
    path_list.sort()
    path_list1.sort()

    if(len(path_list) != len(path_list1)):
        for i in range(len(path_list)):
            for j in range(len(next(os.walk(path_list[i]))[1])-1):
                path_list.append(path_list[i])

    path_list.sort()
    path_list1.sort()
    
    if os.path.exists(path_list[0]+'sum.csv') and os.path.exists(path_list[0]+'tempresult.csv'):
        os.remove(path_list[0]+'sum.csv')
        os.remove(path_list[0]+'tempresult.csv')
        time.sleep(0.5)
    else :
        pass # 중복파일 있으면 제거 
    
    for j in range(len(num)):
#         try:
        df4 = pd.read_csv(path_list1[j] + 'sample3.txt',header=None,delimiter='\t',
              names=['sensorID','date1','date2','soc','dischargedTime','bsoc','chargedTime','suspendTime','voltaicH','voltaicD','voltageH','voltageD','temperatureH'
                     ,'temperatureD','faceValueH','faceValueD','dodD','elc','meancycleD','el','operationRate','BL','overCharge','overChargeTime'
                     ,'overdisCharge','overdisChargeTime','abnormalTemperature','abnormalTemperatureTime','lackFaceValue','lackFaceValueTime'
                     ,'locX','locY','batteryCheck','insufficientCharge','insufficientChargeTime'
                     ,'overChargeCount','overdisChargeCount','abnormalTemperatureCount','lackFaceValueCount','insufficientChargeCount'
                     ,'Max','Max_cycle','totalCycle']) #txt를 csv로 변환

        date1 = []
        date2 = []
        for i in range(len(df4)):
            date1.append(str(datetime.strptime(str(df4['date1'][i]),"%Y%m%d%H")))
            date2.append(str(datetime.strptime(str(df4['date2'][i]),"%Y%m%d%H")))
            date1[i]=date1[i].replace(' ','T')
            date2[i]=date2[i].replace(' ','T') #날짜 형식 맞추기 

        DIM_list = [] #방전전류 
        CIM_list = [] #충전전류 
        DVM_list = [] #방전전압 
        CVM_list = [] #충전전압 
        DTM_list = [] #방전온도 
        CTM_list = [] #충전온도 
        FVM_list = [] #액면 부족시간(월)
        DODM_list = [] #DOD(월)
        CYCM_list = [] #CYCLE(월)
        DTimeM_list = [] #배터리 방전시간(월)
        CTimeM_list = [] #배터리 충전시간(월)
        STimeM_list = [] #배터리 휴지시간(월)
        
        for i in range(len(df4)):
            if(df4['soc'][i] == 2):
                DIM_list.append(df4['voltaicD'][i])
                DVM_list.append(df4['voltageD'][i])
                DTM_list.append(df4['temperatureD'][i])
            elif(df4['soc'][i] == 1):   
                CIM_list.append(df4['voltaicD'][i])
                CVM_list.append(df4['voltageD'][i])
                CTM_list.append(df4['temperatureD'][i])

        for i in range(len(df4)):
            FVM_list.append(df4['faceValueD'][i])
            DODM_list.append(df4['dodD'][i])
            CYCM_list.append(df4['meancycleD'][i])
            DTimeM_list.append(df4['dischargedTime'][i])
            CTimeM_list.append(df4['chargedTime'][i])
            STimeM_list.append(df4['suspendTime'][i])
    
        try: # 1일 평균값 저장
            DIMC.append(round(sum(DIM_list) / len(DIM_list),1))

        except ZeroDivisionError: # 1일 평균값 저장
            DIMC.append(round(sum(DIM_list) / 1,1))

        try: # 1일 평균값 저장
            CIMC.append(round(sum(CIM_list) / len(CIM_list),1))

        except ZeroDivisionError: # 1일 평균값 저장
            CIMC.append(round(sum(CIM_list) / 1,1))

        try: # 1일 평균값 저장
            DVMC.append(round(sum(DVM_list) / len(DVM_list),1))

        except ZeroDivisionError: # 1일 평균값 저장
            DVMC.append(round(sum(DVM_list) / 1,1))

        try: # 1일 평균값 저장
            CVMC.append(round(sum(CVM_list) / len(CVM_list),1))

        except ZeroDivisionError: # 1일 평균값 저장
            CVMC.append(round(sum(CVM_list) / 1,1))

        try: # 1일 평균값 저장
            DTMC.append(round(sum(DTM_list) / len(DTM_list),1))

        except ZeroDivisionError: # 1일 평균값 저장
            DTMC.append(round(sum(DTM_list) / 1,1))

        try: # 1일 평균값 저장
            CTMC.append(round(sum(CTM_list) / len(CTM_list),1))

        except ZeroDivisionError: # 1일 평균값 저장
            CTMC.append(round(sum(CTM_list) / 1,1))

        try: # 1일 평균값 저장
            FVMC.append(round(sum(FVM_list) / len(FVM_list),1))

        except ZeroDivisionError: # 1일 평균값 저장
            FVMC.append(round(sum(FVM_list) / 1,1))

        try: # 1일 평균값 저장
            DODMC.append(round(sum(DODM_list) / len(DODM_list),1))

        except ZeroDivisionError: # 1일 평균값 저장
            DODMC.append(round(sum(DODM_list) / 1,1))

        try: # 1일 평균값 저장
            CYCMC.append(round(sum(CYCM_list) / len(CYCM_list),1))

        except ZeroDivisionError: # 1일 평균값 저장
            CYCMC.append(round(sum(CYCM_list) / 1,1))

        try: # 1일 평균값 저장
            DTimeMC.append(round(sum(DTimeM_list) / len(DTimeM_list),5))

        except ZeroDivisionError: # 1일 평균값 저장
            DTimeMC.append(round(sum(DTimeM_list) / len(DTimeM_list),5))

        try: # 1일 평균값 저장
            CTimeMC.append(round(sum(CTimeM_list) / len(CTimeM_list),5))

        except ZeroDivisionError: # 1일 평균값 저장
            CTimeMC.append(round(sum(CTimeM_list) / len(CTimeM_list),5))

        try: # 1일 평균값 저장
            STimeMC.append(round(sum(STimeM_list) / len(STimeM_list),5))

        except ZeroDivisionError: # 1일 평균값 저장
            STimeMC.append(round(sum(STimeM_list) / len(STimeM_list),5)) # 기존
        
        df4['date1'] = date1
        df4['date2'] = date2
        
        if not os.path.exists(path_list[j] +'sum.csv'):
            df4.to_csv(path_list[j] +'sum.csv',index=False, mode = 'w') #csv로 저장
        else: 
            df4.to_csv(path_list[j] +'sum.csv',index=False, mode = 'a',header = False) #csv로 누적 저장
#             df4.to_csv(path_list[j] +'sum.csv',index=False, mode = 'w') #csv로 누적 저장
    
        df5 = pd.read_csv(path_list[j] + 'sum.csv',encoding = 'cp949')

        sdate = df5.date1[0]  # date 첫 값
        edate = df5.date1[len(df5.date1) - 1]  # date 마지막 값

#         sdate = datetime.strptime(str(sdate)[:-4], '%Y%m%d%H')
#         edate = datetime.strptime(str(edate)[:-4], '%Y%m%d%H')  # str을 date로 변환

        sdate = datetime.strptime(str(sdate)[:-6], '%Y-%m-%dT%H')
        edate = datetime.strptime(str(edate)[:-6], '%Y-%m-%dT%H')  # str을 date로 변환

        diff_months_list = list(rrule.rrule(rrule.MONTHLY, dtstart=sdate, until=edate))
        Mdays = len(diff_months_list)  # 엑셀 첫날과 마지막날 비교해서 개월수 구하기

        y = df5['date1'].astype(str) 
        Ydays = int(y.str.slice(0,4,1)[len(y.str.slice(0,4,1))-1]) - int(y.str.slice(0,4,1)[0]) + 1 #첫날과 마지막날 비교해서 년수 구하기

        IM = [] #전류(월)
        VM = [] #방전전압(월)
        TM = [] #방전온도(월)
        FVM = [] #액면 부족시간(월)
        DODM = [] #DOD(월)p
        CYCM = [] #사이클(월)
        CYCSM = [] #누적 사이클
        TDTime = [] #배터리 누적 방전시간
        TCTime= [] #배터리 누적 충전시간
        TSTime = [] #배터리 누적 휴지시간
        IY = [] #전류(년)
        VY = [] #전압(년)
        TY = []  #온도(년)
        FVY = [] #액면부족시간(년)
        DODY = [] #DOD(년)
        CYCY = [] #사이클(년)
        BS_list = [] #배터리 상태
    
        EL_list = []
        UL_list = []
        ULC_list = []
        RL_list = []
        RLC_list = []
        U_list = []
        D_list = []

        days = len(next(os.walk(path_list[0]))[1]) #파일개수로 날짜 파악

        for i in range(len(df5)):
            if(df5['soc'][i] == 2):
                IM.append(round(sum(DIMC)/days,1))
                VM.append(round(sum(DVMC)/days,1))
                TM.append(round(sum(DTMC)/days,1))
            elif(df5['soc'][i] == 1):   
                IM.append(round(sum(CIMC)/days,1))
                VM.append(round(sum(CVMC)/days,1))
                TM.append(round(sum(CTMC)/days,1))
            else:
                pass

        for i in range(len(df5)):
            FVM.append(round(sum(FVMC)/days,1)) #액면 부족시간 (월)
            DODM.append(round(sum(DODMC)/days,1)) #DOD (월)
            CYCM.append(round(sum(CYCMC)/days,1)) #CYCLE (월)
            CYCSM.append(round(sum(CYCMC),1)) #누적 CYCLE
            TDTime.append(sum(DTimeMC))
            TCTime.append(sum(CTimeMC))
            TSTime.append(sum(STimeMC))
            EL_list.append(df5['elc'][i]/Mdays)
            IY.append(round(IM[i]/Ydays,1)) #전류(년)
            VY.append(round(VM[i]/Ydays,1)) #전압(년)
            TY.append(round(TM[i]/Ydays,1)) #온도(년)
            FVY.append(round(FVM[i]/Ydays,1)) #액면상태(년)
            DODY.append(round(DODM[i]/Ydays,1)) #DOD(년)
            CYCY.append(round(CYCM[i]/Ydays,1)) #사이클(년)

        df5['voltaicM'] = IM
        df5['voltaicY'] = IY
        df5['voltageM'] = VM
        df5['voltageY'] = VY
        df5['temperatureM'] = TM
        df5['temperatureY'] = TY
        df5['faceValueM'] = FVM 
        df5['faceValueY'] = FVY
        df5['dodM'] = DODM
        df5['dodY'] = DODY
        df5['meancycleM'] = CYCM
        df5['meancycleY'] = CYCY
#         df5['cycle'] = CYCSM
        df5['cycle'] = sum(df5['totalCycle'])/len(df5['totalCycle'])
        df5['chargeduseTime'] = TCTime
        df5['dischargeduseTime'] = TDTime
        df5['suspendTimeTotal'] = TSTime
        df5['chargedCount'] = df5['meancycleD'] #데이터프레임 추가

        for i in range(len(df5)):
            U_list.append(round(df5['cycle'][i]/EL_list[i] * 100,1)) #사용수명율 
            D_list.append(round(100-U_list[i],1)) #잔존수명율 
            UL_list.append(round((U_list[i]/100 * EL_list[i])/df5['meancycleM'][i]/22)) #사용수명기간
            ULC_list.append(round((U_list[i] * df5['elc'][i]) / 100)) #사용수명사이클
            RL_list.append(round((D_list[i]/100 * EL_list[i])/df5['meancycleM'][i]/22)) #잔여사용기간
            RLC_list.append(round((D_list[i] * df5['elc'][i]) / 100)) #잔여사용사이클

            if (df5['BL'][i] * 0.8) <= RLC_list[i]:
                BS_list.append('GOOD')
            elif (df5['BL'][i] * 0.5) <= RLC_list[i] and (df5['BL'][i] * 0.8) > RLC_list[i]:
                BS_list.append('NORMAL')
            elif (df5['BL'][i] * 0.5) > RLC_list[i]:
                BS_list.append('BAD')
            elif (df5['BL'][i] * 0.2) >= RLC_list[i]:
                df5['batteryCheck'][i] = 'Change'
            else:
                pass

        df5['ul'] = UL_list
        df5['ulc'] = ULC_list
        df5['rl'] = RL_list
        df5['rlc'] = RLC_list #데이터프레임 추가
        df5['batteryState'] = BS_list

        del df5['BL']  #필요없는 파라미터 삭제

        del_list = []

        for i in range(len(df5)-1):
            if (df5['date1'][i] == df5['date1'][i+1]) and i != 0:
                df5['soc'][i+1] = str(df5['soc'][i+1]) +'/'+ str(df5['soc'][i])
                df5['bsoc'][i+1] = str(df5['bsoc'][i+1]) +'/'+ str(df5['bsoc'][i])
                
                df5['voltaicH'][i+1] = str(df5['voltaicH'][i+1]) +'/'+ str(df5['voltaicH'][i])
                df5['voltaicD'][i+1] = str(df5['voltaicD'][i+1]) +'/'+ str(df5['voltaicD'][i])
                df5['voltageH'][i+1] = str(df5['voltageH'][i+1]) +'/'+ str(df5['voltageH'][i])
                df5['voltageD'][i+1] = str(df5['voltageD'][i+1]) +'/'+ str(df5['voltageD'][i])
                df5['temperatureH'][i+1] = str(df5['temperatureH'][i+1]) +'/'+ str(df5['temperatureH'][i])
                df5['temperatureD'][i+1] = str(df5['temperatureD'][i+1]) +'/'+ str(df5['temperatureD'][i])
                df5['voltaicM'][i+1] = str(df5['voltaicM'][i+1]) +'/'+ str(df5['voltaicM'][i])
                df5['voltaicY'][i+1] = str(df5['voltaicY'][i+1]) +'/'+ str(df5['voltaicY'][i])
                df5['voltageM'][i+1] = str(df5['voltageM'][i+1]) +'/'+ str(df5['voltageM'][i])
                df5['voltageY'][i+1] = str(df5['voltageY'][i+1]) +'/'+ str(df5['voltageY'][i])
                df5['temperatureM'][i+1] = str(df5['temperatureM'][i+1]) +'/'+ str(df5['temperatureM'][i])
                df5['temperatureY'][i+1] = str(df5['temperatureY'][i+1]) +'/'+ str(df5['temperatureY'][i]) #충방전값 하나로 합치기

                del_list.append(i)
            elif (df5['date1'][i] == df5['date2'][i]) and i == 0:

                del_list.append(i) #첫행이 중복일 경우 제거 
            else:
                pass          
        
        df6 = df5.drop(del_list) #중복된 부분 제거
        
        df7 = df6.reset_index() #인덱스 재정렬
        
        ID_list = [] #전류(일)
        VD_list = [] #전압(일)
        TD_list = [] #온도(일)
        IM_list = [] #전류(월)
        VM_list = [] #방전전압(월)
        TM_list = [] #방전온도(월)
        IY_list = [] #전류(년)
        VY_list = [] #전압(년)
        TY_list = [] #온도(년)
        
        for i in range(len(df7)):
            if len(str(df7['soc'][i])) > 2:
                ID_list.append(df7['voltaicD'][i]) 
                VD_list.append(df7['voltageD'][i])
                TD_list.append(df7['temperatureD'][i]) 
                IM_list.append(df7['voltaicM'][i])
                VM_list.append(df7['voltageM'][i])
                TM_list.append(df7['temperatureM'][i])
                IY_list.append(df7['voltaicY'][i])
                VY_list.append(df7['voltageY'][i])
                TY_list.append(df7['temperatureY'][i])
            else:
                pass
        if len(ID_list) > 0 :
            for i in range(len(df7)-1):
                if df7['soc'][i] == 1 or df7['soc'][i] == 2 :
                    df7['voltaicD'][i] = ID_list[0]
                    df7['voltageD'][i] = VD_list[0]
                    df7['temperatureD'][i] = TD_list[0]
                    df7['voltaicM'][i] = IM_list[0]
                    df7['voltageM'][i] = VM_list[0]
                    df7['temperatureM'][i] = TM_list[0]
                    df7['voltaicY'][i] = IY_list[0]
                    df7['voltageY'][i] = VY_list[0]
                    df7['temperatureY'][i] = TY_list[0]
                
        df7.to_csv(path_list[j] + 'result.csv',index = False)  
    
    if os.path.exists(path_list[j] +'result.csv'):
        df7.to_csv(path_list[j] +'tempresult.csv',index=False, mode = 'w') #csv로 저장
        #         (중복제거 코드 삽입)
    else: 
        df7.to_csv(path_list[j] +'tempresult.csv',index=False, mode = 'a')
        pass
    
    Ddf = pd.read_csv(path_list[0] + 'sum.csv',encoding = 'cp949')

    D_del_list = []
    M_del_list = []  #날짜 중복 제거용 리스트

    voltaicCharged = [] # 충전시 전류
    voltaicDischarged = [] # 방전시 전류
    voltageCharged = [] # 충전시 전압
    voltageDischarged = [] # 방전시 전압
    temperatureCharged = [] # 충전시 온도
    temperatureDischarged = [] # 방전시 온도
    battery_soc = [] #soc

    for i in range(len(Ddf)-1):
        if(Ddf['date1'].str.slice(0,10,1)[i] == Ddf['date1'].str.slice(0,10,1)[i+1]):
            D_del_list.append(i)

    Ddf1 = Ddf.drop(D_del_list) # 날짜 중복 제거

    Ddf2 = Ddf1.reset_index()

    for i in range(len(Ddf2)):
        if(Ddf2['soc'][i] == 1): # SOC 1이면 충전
            voltaicCharged.append(Ddf2['voltaicD'][i])
            voltageCharged.append(Ddf2['voltageD'][i])
            temperatureCharged.append(Ddf2['temperatureD'][i])
            
            voltaicDischarged.append(0)
            voltageDischarged.append(0)
            temperatureDischarged.append(0)

            battery_soc.append(round(Ddf2['bsoc'][i],0))
        
        elif(Ddf2['soc'][i] == 2): # SOC 2이면 방전
            voltaicDischarged.append(Ddf2['voltaicD'][i])
            voltageDischarged.append(Ddf2['voltageD'][i])
            temperatureDischarged.append(Ddf2['temperatureD'][i])
            
            voltaicCharged.append(0)
            voltageCharged.append(0)
            temperatureCharged.append(0)

            battery_soc.append(round(Ddf2['bsoc'][i],0))

    voltaicCharged_list = []
    voltaicDischarged_list = []
    voltageCharged_list = []
    voltageDischarged_list = []
    temperatureCharged_list = []
    temperatureDischarged_list = [] #중복제거용 리스트

    for i in voltaicCharged:
        if i not in voltaicCharged_list:
            voltaicCharged_list.append(i) # 중복 제거

    for i in voltaicDischarged:
        if i not in voltaicDischarged_list:
            voltaicDischarged_list.append(i) # 중복 제거

    for i in voltageCharged:
        if i not in voltageCharged_list:
            voltageCharged_list.append(i) # 중복 제거

    for i in voltageDischarged:
        if i not in voltageDischarged_list:
            voltageDischarged_list.append(i) # 중복 제거

    for i in temperatureCharged:
        if i not in temperatureCharged_list:
            temperatureCharged_list.append(i) # 중복 제거

    for i in temperatureDischarged:
        if i not in temperatureDischarged_list:
            temperatureDischarged_list.append(i) # 중복 제거

    Mdf = pd.read_csv(path_list[0] + 'tempresult.csv',encoding = 'cp949')

    for i in range(len(Mdf)-1):
        if(Mdf['date1'].str.slice(0,10,1)[i] == Mdf['date1'].str.slice(0,10,1)[i+1]):
            M_del_list.append(i)

    Mdf1 = Mdf.drop(M_del_list)  # 날짜 중복 제거

    del Mdf1['date1']
    del Mdf1['voltaicH']
    del Mdf1['voltaicD']
    del Mdf1['voltageH']
    del Mdf1['voltageD']
    del Mdf1['temperatureH']
    del Mdf1['temperatureD']
    del Mdf1['faceValueH']
    del Mdf1['voltaicM']
    del Mdf1['voltaicY']
    del Mdf1['voltageM']
    del Mdf1['voltageY']
    del Mdf1['temperatureM']
    del Mdf1['temperatureY']
    del Mdf1['faceValueM']
    del Mdf1['faceValueY']
    del Mdf1['dodM']
    del Mdf1['dodY']
    del Mdf1['meancycleM']
    del Mdf1['meancycleY'] 
    del Mdf1['bsoc']

    # 일 평균 구할시 불필요한 값 삭제

    Mdf1['voltaicCharged'] = CIMC
    Mdf1['voltaicDischarged'] = DIMC
    Mdf1['voltageCharged'] = CVMC
    Mdf1['voltageDischarged'] = DVMC
    Mdf1['temperatureCharged'] = CTMC
    Mdf1['temperatureDischarged'] = DTMC # 칼럼 값 변경
    Mdf1['bsoc'] = battery_soc

    Mdf2 = Mdf1.reset_index() # 인덱스 정렬

    chargedCount_list = []

    for i in range(len(Mdf2)):
        chargedCount_list.append(math.floor(Mdf2['chargedCount'][i])) # 충방전 횟수 정수형태 변경

    Mdf2['chargedCount'] = chargedCount_list

    Mdf2.rename(columns = {"locX":"longitude","locY":"latitude","date2":"date","chargeduseTime":"chargedTimeTotal","dischargeduseTime":"dischargedTimeTotal",
                           "faceValueD":"faceValue","dodD":"dod","meancycleD":"meancycle"},inplace = True) #칼럼 이름 수정

#     locator = Nominatim(user_agent="myGeocoder")
    geolocator = Nominatim(user_agent="myGeolocator")

    coordinates_longitude = []
    coordinates_latitude = [] #위도 경도 저장 리스트

    for i in range(len(Mdf2)):
        coordinates_longitude.append(Mdf2['longitude'][i])
        coordinates_latitude.append(Mdf2['latitude'][i])

    address_list = [] #위도 경도 변환 값 저장 리스트

    for i in range(len(Mdf2)):
        coordinates = coordinates_latitude[i],coordinates_longitude[i] # 위도 경도 값으로 주소 반환
#         result = locator.reverse(coordinates) 
        result = geolocator.reverse(coordinates)
        address_list.append(result.address)

    overChargeTimeTotal = [] #과충전 시간(시)
    overdisChargeTimeTotal = [] #과방전 시간(시)
    abnormalTemperatureTimeTotal = [] #온도이상 시간(시)
    lackFaceValueTimeTotal = [] #액면부족 시간(시)

    for i in range(len(Mdf2)):
        overChargeTimeTotal.append(round(sum(Mdf2['overChargeTime'])/60,5))
        overdisChargeTimeTotal.append(round(sum(Mdf2['overdisChargeTime'])/60,5))
        abnormalTemperatureTimeTotal.append(round(sum(Mdf2['abnormalTemperatureTime'])/60,5))
        lackFaceValueTimeTotal.append(round(sum(Mdf2['lackFaceValueTime'])/60,5)) #60분 시간단위로 변환

    Mdf2['address'] = address_list # 주소 값 추가

    Mdf2['overChargeTimeTotal'] = overChargeTimeTotal
    Mdf2['overdisChargeTimeTotal'] = overdisChargeTimeTotal
    Mdf2['abnormalTemperatureTimeTotal'] = abnormalTemperatureTimeTotal
    Mdf2['lackFaceValueTimeTotal'] = lackFaceValueTimeTotal

    Dod = sum(Mdf2['dod'])/len(Mdf2['dod']) #dod 계수
    
    if Dod < 40 :
        Dod_c = 1.5
    elif Dod >= 40 and Dod < 50:
        Dod_c = 1.42
    elif Dod >= 50 and Dod < 60:
        Dod_c = 1.25
    elif Dod >= 60 and Dod < 70:
        Dod_c = 1.08
    elif Dod >= 70 and Dod < 80:
        Dod_c = 0.92
    elif Dod >= 80 and Dod < 90:
        Dod_c = 0.75
    elif Dod >= 90 : # and DD <= 100 <-DD값이 낮아지면 다시 삽입
        Dod_c = 0.58
    else:
        pass
    
    temperature_discharged = [] #방전 온도
    for i in range(len(Mdf2)):
        if Mdf2['temperatureDischarged'][i] != 0:
            temperature_discharged.append(Mdf2['temperatureDischarged'][i])
        else:
            pass

    worktemperature = sum(temperature_discharged)/len(temperature_discharged) #평균 방전온도

    if worktemperature < 30 :
        worktemperature = 30
    else :
        pass
    
    voltaic_discharged = [] #방전 전류
    for i in range(len(Mdf2)):
        if Mdf2['voltaicDischarged'][i] != 0:
            voltaic_discharged.append(Mdf2['voltaicDischarged'][i])
        else:
            pass

    voltaic_discharged_ln = ln(round(sum(voltaic_discharged)/len(voltaic_discharged),1)/batteryCapacity*5)

    if pd.isna(voltaic_discharged_ln) == True : # 결측값 감지
        voltaic_discharged_ln = 0
    else :
        pass
        
    dischargeCycle = round((sum(Mdf2['meancycle'])/len(Mdf2['meancycle'])),2) # 일 평균 방전 사이클
    
    elc = round(((1200*Dod_c)*30/(worktemperature*(1+(7/30)*voltaic_discharged_ln)))*1*0.85*1,1) #기대수명 사이클
    el = round(((1200*Dod_c)*30*12/(worktemperature*(1+(7/30)*voltaic_discharged_ln)*dischargeCycle*220))*1*0.85*1,1) #기대수명 시용기간

    ulc = round((sum(Mdf2['cycle'])/len(Mdf2['cycle'])),1) #사용 수명 사이클
    ul = round((sum(Mdf2['cycle'])/len(Mdf2['cycle']))/(220/12),1)
    
#     ul = [] #사용수명 기간
#     for i in range(len(Mdf2)):
#         ul.append(round((sum(Mdf2['meancycle'])/Mdf2['meancycle'][i])/(220/12),1))
        
#     rlc = round(elc - ulc,1) #잔여수명 사이클
#     rl = [] #잔여수명 기간
#     for i in range(len(Mdf2)):
#         rl.append(round(el-ul[i],1))
        
    remove_set = {0.0}

    voltaicCharged_remove = [i for i  in list(Mdf2['voltaicCharged']) if i not in remove_set]
    voltaicDischarged_remove = [i for i  in list(Mdf2['voltaicDischarged']) if i not in remove_set]
    voltageCharged_remove = [i for i  in list(Mdf2['voltageCharged']) if i not in remove_set]
    voltageDischarged_remove = [i for i  in list(Mdf2['voltageDischarged']) if i not in remove_set]
    temperatureCharged_remove = [i for i  in list(Mdf2['temperatureCharged']) if i not in remove_set]
    temperatureDischarged_remove = [i for i  in list(Mdf2['temperatureDischarged']) if i not in remove_set] #리스트의 0인 값 제거

    try :
        voltaicChargedTotal = round(sum(voltaicCharged_remove)/len(voltaicCharged_remove),1)
    except :
        voltaicChargedTotal = round(sum(voltaicCharged_remove)/1,1)

    try :
        voltaicDischargedTotal = round(sum(voltaicDischarged_remove)/len(voltaicDischarged_remove),1) # 평균 전류(충전/방전)
    except :
        voltaicDischargedTotal = round(sum(voltaicDischarged_remove)/1,1) # 평균 전류(충전/방전)

    try :
        voltageChargedTotal = round(sum(voltageCharged_remove)/len(voltageCharged_remove),1)

    except :
        voltageChargedTotal = round(sum(voltageCharged_remove)/1,1)

    try :
        voltageDischargedTotal = round(sum(voltageDischarged_remove)/len(voltageDischarged_remove),1) # 평균 전압(충전/방전)

    except :
        voltageDischargedTotal = round(sum(voltageDischarged_remove)/1,1) # 평균 전압(충전/방전)

    try :
        temperatureChargedTotal = round(sum(temperatureCharged_remove)/len(temperatureCharged_remove),1)

    except :
        temperatureChargedTotal = round(sum(temperatureCharged_remove)/1,1)

    try :
        temperatureDischargedTotal = round(sum(temperatureDischarged_remove)/len(temperatureDischarged_remove),1) #평균 온도(충전/방전)

    except :
        temperatureDischargedTotal = round(sum(temperatureDischarged_remove)/1,1) #평균 온도(충전/방전)


#     voltaicChargedTotal = round(sum(voltaicCharged_remove)/len(voltaicCharged_remove),1)
#     voltaicDischargedTotal = round(sum(voltaicDischarged_remove)/len(voltaicDischarged_remove),1) # 평균 전류(충전/방전)
#     voltageChargedTotal = round(sum(voltageCharged_remove)/len(voltageCharged_remove),1)
#     voltageDischargedTotal = round(sum(voltageDischarged_remove)/len(voltageDischarged_remove),1) # 평균 전압(충전/방전)
#     temperatureChargedTotal = round(sum(temperatureCharged_remove)/len(temperatureCharged_remove),1)
#     temperatureDischargedTotal = round(sum(temperatureDischarged_remove)/len(temperatureDischarged_remove),1) #평균 온도(충전/방전)

    Mdf2['voltaicChargedTotal'] = voltaicChargedTotal
    Mdf2['voltaicDischargedTotal'] = voltaicDischargedTotal
    Mdf2['voltageChargedTotal'] = voltageChargedTotal
    Mdf2['voltageDischargedTotal'] = voltageDischargedTotal
    Mdf2['temperatureChargedTotal'] = temperatureChargedTotal
    Mdf2['temperatureDischargedTotal'] = temperatureDischargedTotal

    try:
        insufficientCharge = round(sum(Mdf2['insufficientChargeCount']) / (sum(Mdf2['cycle'])/len(Mdf2['cycle'])),1)
    except :
        insufficientCharge = round(sum(Mdf2['insufficientChargeCount']) /1,1)

#   insufficientCharge = round(sum(Mdf2['insufficientChargeCount']) / (sum(Mdf2['cycle'])/len(Mdf2['cycle'])) ,1) # 부족 충전 %
    
    expectedLifePercent = round((elc*100)/Mdf2['Max_cycle'][len(Mdf2)-1],1)
    usedLifePercent = round((ulc*100)/Mdf2['Max_cycle'][len(Mdf2)-1],1)

    if expectedLifePercent > 100:
        expectedLifePercent = 100
    else :
        pass

    faceValue = []
    for i in range(len(Mdf2)):
        if Mdf2['faceValue'][i] > 0.5 :
            faceValue.append(1)
        else :
            faceValue.append(0)
            
    if Mdf2['cycle'][0] <= 200 :
        BS_list = 'GOOD'
    else :
        BS_list = 'NORMAL' #임시 데이터 자세한 값이 오면 normal과 bad 기준 강화 

    del Mdf2['totalCycle']    
    del Mdf2['el']
    del Mdf2['elc']
    del Mdf2['ul']
    del Mdf2['ulc']
#     del Mdf2['rl']
#     del Mdf2['rlc']
    del Mdf2['suspendTime']
    del Mdf2['insufficientCharge']
    del Mdf2['Max']
    del Mdf2['Max_cycle']
    del Mdf2['faceValue']
    del Mdf2['batteryState']
    
    Mdf2['batteryState'] = BS_list
    Mdf2['faceValue'] = faceValue
    Mdf2['insufficientCharge'] = insufficientCharge

#     plusMonth = 14
#     plusPercent = 35.8
#     plusCycle = 430

    Mdf2['expectedLifeCycle'] = elc
    Mdf2['expectedLifeMonth'] = el
    Mdf2['expectedLifePercent'] = expectedLifePercent
    
    Mdf2['usedLifeCycle'] = ulc 
    Mdf2['usedLifeMonth'] = ul
    Mdf2['usedLifePercent'] = usedLifePercent
#     Mdf2['rlc'] = rlc
#     Mdf2['rl'] = rl #서버계산
    
    Mdf2.to_csv(path_list[0] + 'Dayresult.csv',index=False, encoding="utf-8-sig",mode = 'w') # 일 평균 데이터 저장
    
# # -------------------------  
# # 아래에서 만든 일단위 분리 데이터 작성 방법 삽입
# # read_csv,to_csv 에 path_list 추가하여서 파일 찾고 저장할 수 있도록 하기
# # 변환한 json 파일 서버로 보낼때 day와 month 파일 두개다 보내도록 하기
# # ------------------------

    return  

el1('sample3.txt')

# def CtoJ(input_file_name):
     
#     for i in range(len(path_list)):
#         output_file_name = "result.json"
#         with open(path_list[i] + input_file_name, "r", encoding="utf-8", newline="") as input_file, \
#             open(path_list[i] + output_file_name, "w", encoding="utf-8", newline="") as output_file:

#             reader = csv.reader(input_file)
#             # 첫 줄은 col_names 리스트로 읽어 놓고
#             col_names = next(reader)
#             # 그 다음 줄부터 zip으로 묶어서 json으로 dumps
#             output_file.write('[')
#             for cols in reader:
#                 doc = {col_name: col for col_name, col in zip(col_names, cols)}
#                 print(json.dumps(doc, ensure_ascii=True,indent = 4), file=output_file)
#                 output_file.write(',')
#             output_file.write(']') #리스트로 묶기

#         with open(path_list[i] +'result.json', 'r') as f:
#             n_list = (f.readlines())
#             n_list[len(n_list)-1] = ']' #불필요한 마지막값 제거 

#         with open(path_list[i]+'result.json','w') as e:
#             e.writelines(n_list)

#     return

# CtoJ('tempresult.csv')


# headers = {
#     'Content-Type': 'application/x-www-form-urlencoded',
#     'Authorization': 'Basic bXlBcHA6cGFzcw=='
# }
# params = {
#     'username': 'softmile',
#     'password': 'softmile123!',
#     'grant_type': 'password'
# }

# result = requests.post("http://121.137.95.45:8101/oauth/token", headers=headers, params=params) #post 방싱으로 로그인

# r_list=result.text.split('"') 
# ac = r_list[3] #access_token 값 추출

# headers={
#     'Authorization': 'bearer' + ac ,
#     'Content-Type': 'application/json'
# }
# for i in range(len(path_list)):
#     with open(path_list[i] + 'result.json', 'r') as f:

#         json_data = json.load(f) #파일 JSON형식으로 읽기

#     jd = json_data 
#     for i in range(len(jd)):
#     #     requests.post("http://121.137.95.45:8101/inBoundApi/batterySomaSave", headers=headers, data=json.dumps(jd[i],indent = 4)) #data json형식으로 맞춰서 데이터 전송
#         print(requests.post("http://121.137.95.45:8101/inBoundApi/batterySomaSave", headers=headers, data=json.dumps(jd[i],indent = 4))) #data json형식으로 맞춰서 데이터 전송

# headers={
#     'Authorization': '' ,
#     'Content-Type': 'application/json'
# }

# with open(path_list[1] + 'result.json', 'r') as f:

#     json_data = json.load(f) #파일 JSON형식으로 읽기

# jd = json_data
    
# cut_point = 0 
# while True:
#     data=json.dumps(jd[cut_point:cut_point+10],indent = 4)
#     print(requests.post("http://52.78.58.84/api/sensor/analysis", headers=headers, data=data)) #data json형식으로 맞춰서 데이터 전송
#     cut_point += 10
#     if cut_point > len(jd): #10개씩 나눠서 보내기
#         break

C:\Users\sm02\Anaconda3\lib\site-packages\ipykernel_launcher.py:1772: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\sm02\Anaconda3\lib\site-packages\ipykernel_launcher.py:1774: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\sm02\Anaconda3\lib\site-packages\ipykernel_launcher.py:1800: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\sm02\Anaconda3\lib\site-packages\ipykernel_launcher.py:1801: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\sm02\Anaconda3\lib\site-packages\ipykernel_launcher.py:1803: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [3]:
path_list

['C:\\Users\\sm02\\파이썬\\battery/f008d1709732/',
 'C:\\Users\\sm02\\파이썬\\battery/f008d1709732/',
 'C:\\Users\\sm02\\파이썬\\battery/f008d1709732/',
 'C:\\Users\\sm02\\파이썬\\battery/f008d1709732/',
 'C:\\Users\\sm02\\파이썬\\battery/f008d1709732/',
 'C:\\Users\\sm02\\파이썬\\battery/f008d1709732/',
 'C:\\Users\\sm02\\파이썬\\battery/f008d1709732/',
 'C:\\Users\\sm02\\파이썬\\battery/f008d1709732/',
 'C:\\Users\\sm02\\파이썬\\battery/f008d1709732/',
 'C:\\Users\\sm02\\파이썬\\battery/f008d1709732/',
 'C:\\Users\\sm02\\파이썬\\battery/f008d1709732/',
 'C:\\Users\\sm02\\파이썬\\battery/f008d1709732/',
 'C:\\Users\\sm02\\파이썬\\battery/f008d1709732/',
 'C:\\Users\\sm02\\파이썬\\battery/f008d1709732/',
 'C:\\Users\\sm02\\파이썬\\battery/f008d1709732/',
 'C:\\Users\\sm02\\파이썬\\battery/f008d1709732/',
 'C:\\Users\\sm02\\파이썬\\battery/f008d1709732/',
 'C:\\Users\\sm02\\파이썬\\battery/f008d1709732/',
 'C:\\Users\\sm02\\파이썬\\battery/f008d1709732/',
 'C:\\Users\\sm02\\파이썬\\battery/f008d1709732/',
 'C:\\Users\\sm02\\파이썬\\battery/f008d170

In [1]:
import pandas as pd
from openpyxl import load_workbook
import numpy as np
import openpyxl
import csv
import math
from datetime import datetime
from dateutil import rrule
from datetime import date
from datetime import datetime,timedelta
import json
from collections import OrderedDict
from PIL import Image
import io
import requests
import os
import sys
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

월단위 데이터 작업 ▼

In [59]:
Mdf3 = pd.read_csv(path_list[0] + 'Dayresult.csv',encoding = 'utf-8-sig')

In [60]:
Mon_1 = []
Mon_2 = []
Mon_3 = []
Mon_4 = []
Mon_5 = []
Mon_6 = []
Mon_7 = []
Mon_8 = []
Mon_9 = []
Mon_10 = []
Mon_11 = []
Mon_12 = []

In [61]:
for i in range(len(Mdf3)):
    if (Mdf3['date'].str.slice(5,7,1)[i] == '01'):
        Mon_1.append(Mdf3.loc[i])
    elif Mdf3['date'].str.slice(5,7,1)[i] == '02':
        Mon_2.append(Mdf3.loc[i])
    elif Mdf3['date'].str.slice(5,7,1)[i] == '03':
        Mon_3.append(Mdf3.loc[i])
    elif Mdf3['date'].str.slice(5,7,1)[i] == '04':
        Mon_4.append(Mdf3.loc[i])
    elif Mdf3['date'].str.slice(5,7,1)[i] == '05':
        Mon_5.append(Mdf3.loc[i])
    elif Mdf3['date'].str.slice(5,7,1)[i] == '06':
        Mon_6.append(Mdf3.loc[i])
    elif Mdf3['date'].str.slice(5,7,1)[i] == '07':
        Mon_7.append(Mdf3.loc[i])
    elif Mdf3['date'].str.slice(5,7,1)[i] == '08':
        Mon_8.append(Mdf3.loc[i])
    elif Mdf3['date'].str.slice(5,7,1)[i] == '09':
        Mon_9.append(Mdf3.loc[i])
    elif Mdf3['date'].str.slice(5,7,1)[i] == '10':
        Mon_10.append(Mdf3.loc[i])
    elif Mdf3['date'].str.slice(5,7,1)[i] == '11':
        Mon_11.append(Mdf3.loc[i])
    elif Mdf3['date'].str.slice(5,7,1)[i] == '12':
        Mon_12.append(Mdf3.loc[i])
    else :
        pass

In [62]:
Mon_1_chargedTime_list = []
Mon_1_dischargedTime_list = []
Mon_1_elcAverage_list = []
Mon_1_ulcAverage_list = []
Mon_1_batteryState_list = []
Mon_1_overChargeCount_list = []
Mon_1_overdisChargeCount_list = []
Mon_1_abnormalTemperatureCount_list = []
Mon_1_lackFaceValueCount_list = []
Mon_1_insufficientChargeCount_list = []
Mon_1_voltaicCharged_list = []
Mon_1_voltaicDischarged_list = []
Mon_1_voltageCharged_list = []
Mon_1_voltageDischarged_list = []
Mon_1_temperatureCharged_list = []
Mon_1_temperatureDischarged_list = []
Mon_1_faceValue_list = []
Mon_1_dod_list = [] #계산식 저장 리스트
Mon_1_chargedTimeTotal_list = []
Mon_1_dischargedTimeTotal_list = []
Mon_1_overChargeTimeTotal_list = []
Mon_1_overdisChargeTimeTotal_list =[]
Mon_1_abnormalTemperatureTimeTotal_list = []
Mon_1_insufficientChargeTimeTotal_list = []
Mon_1_lackFaceValueTimeTotal_list = []
Mon_1_insufficientCharge_list = []
Mon_1_meancycle_list = []
Mon_1_usedLifeMonth_list = []

for i in range(len(Mon_1)): # 월 데이터 모으기
    Mon_1_chargedTime_list.append(Mon_1[i]['chargedTime'])
    Mon_1_dischargedTime_list.append(Mon_1[i]['dischargedTime'])
    Mon_1_elcAverage_list.append(Mon_1[i]['expectedLifeCycle'])
    Mon_1_ulcAverage_list.append(Mon_1[i]['usedLifeCycle'])
    Mon_1_batteryState_list.append(Mon_1[i]['batteryState'])
    Mon_1_overChargeCount_list.append(Mon_1[i]['overChargeCount'])
    Mon_1_overdisChargeCount_list.append(Mon_1[i]['overdisChargeCount'])
    Mon_1_abnormalTemperatureCount_list.append(Mon_1[i]['abnormalTemperatureCount'])
    Mon_1_lackFaceValueCount_list.append(Mon_1[i]['lackFaceValueCount'])
    Mon_1_insufficientChargeCount_list.append(Mon_1[i]['insufficientChargeCount'])
    Mon_1_voltaicCharged_list.append(Mon_1[i]['voltaicCharged'])
    Mon_1_voltaicDischarged_list.append(Mon_1[i]['voltaicDischarged'])
    Mon_1_voltageCharged_list.append(Mon_1[i]['voltageCharged'])
    Mon_1_voltageDischarged_list.append(Mon_1[i]['voltageDischarged'])
    Mon_1_temperatureCharged_list.append(Mon_1[i]['temperatureCharged'])
    Mon_1_temperatureDischarged_list.append(Mon_1[i]['temperatureDischarged'])
    Mon_1_faceValue_list.append(Mon_1[i]['faceValue'])
    Mon_1_dod_list.append(Mon_1[i]['dod']) 
    Mon_1_chargedTimeTotal_list.append(Mon_1[i]['chargedTime'])
    Mon_1_dischargedTimeTotal_list.append(Mon_1[i]['dischargedTime'])
    Mon_1_overChargeTimeTotal_list.append(Mon_1[i]['overChargeTime'])
    Mon_1_overdisChargeTimeTotal_list.append(Mon_1[i]['overdisChargeTime'])
    Mon_1_abnormalTemperatureTimeTotal_list.append(Mon_1[i]['abnormalTemperatureTime'])
    Mon_1_lackFaceValueTimeTotal_list.append(Mon_1[i]['lackFaceValueTime'])
    Mon_1_insufficientChargeTimeTotal_list.append(Mon_1[i]['insufficientChargeTime'])
    Mon_1_insufficientCharge_list.append(Mon_1[i]['insufficientCharge'])
    Mon_1_meancycle_list.append(Mon_1[i]['meancycle'])
    Mon_1_usedLifeMonth_list.append(Mon_1[i]['usedLifeMonth'])
    
Mon_1_chargedTime = []
Mon_1_dischargedTime = []
Mon_1_elcAverage = []
Mon_1_ulcAverage = []
Mon_1_batteryState = []
Mon_1_overChargeCount = []
Mon_1_overdisChargeCount = []
Mon_1_abnormalTemperatureCount = []
Mon_1_lackFaceValueCount = []
Mon_1_insufficientChargeCount = []
Mon_1_voltaicCharged = []
Mon_1_voltaicDischarged = []
Mon_1_voltageCharged = []
Mon_1_voltageDischarged = []
Mon_1_temperatureCharged = []
Mon_1_temperatureDischarged = []
Mon_1_faceValue = []
Mon_1_dod = [] # 월 데이터 리스트
Mon_1_year = []
Mon_1_month = []
Mon_1_sensorID = []
Mon_1_chargedTimeTotal = []
Mon_1_dischargedTimeTotal = []
Mon_1_overChargeTimeTotal = []
Mon_1_overdisChargeTimeTotal =[]
Mon_1_abnormalTemperatureTimeTotal = []
Mon_1_lackFaceValueTimeTotal = []
Mon_1_insufficientChargeTimeTotal = []
Mon_1_insufficientCharge = []
Mon_1_meancycle = []
Mon_1_expectedLifeMonth = []
Mon_1_expectedLifeCycle = []
Mon_1_usedLifeMonth = []
Mon_1_expectedLifePercent = []
Mon_1_usedLifePercent = []

try :
    Mon_1_chargedTime.append(sum(Mon_1_chargedTime_list)/len(Mon_1_chargedTime_list))
    Mon_1_dischargedTime.append(sum(Mon_1_dischargedTime_list)/len(Mon_1_dischargedTime_list))
    Mon_1_elcAverage.append(sum(Mon_1_elcAverage_list)/len(Mon_1_elcAverage_list))
    Mon_1_ulcAverage.append(sum(Mon_1_ulcAverage_list)/len(Mon_1_ulcAverage_list))
    Mon_1_batteryState.append(Mon_1[-1]['batteryState'])
    Mon_1_overChargeCount.append(sum(Mon_1_overChargeCount_list)/len(Mon_1_overChargeCount_list))
    Mon_1_overdisChargeCount.append(sum(Mon_1_overdisChargeCount_list)/len(Mon_1_overdisChargeCount_list))
    Mon_1_abnormalTemperatureCount.append(sum(Mon_1_abnormalTemperatureCount_list)/len(Mon_1_abnormalTemperatureCount_list))
    Mon_1_lackFaceValueCount.append(sum(Mon_1_lackFaceValueCount_list)/len(Mon_1_lackFaceValueCount_list))
    Mon_1_insufficientChargeCount.append(sum(Mon_1_insufficientChargeCount_list)/len(Mon_1_insufficientChargeCount_list))
    Mon_1_voltaicCharged.append(sum(Mon_1_voltaicCharged_list)/len(Mon_1_voltaicCharged_list))
    Mon_1_voltaicDischarged.append(sum(Mon_1_voltaicDischarged_list)/len(Mon_1_voltaicDischarged_list))
    Mon_1_voltageCharged.append(sum(Mon_1_voltageCharged_list)/len(Mon_1_voltageCharged_list))
    Mon_1_voltageDischarged.append(sum(Mon_1_voltageDischarged_list)/len(Mon_1_voltageDischarged_list))
    Mon_1_temperatureCharged.append(sum(Mon_1_temperatureCharged_list)/len(Mon_1_temperatureCharged_list))
    Mon_1_temperatureDischarged.append(sum(Mon_1_temperatureDischarged_list)/len(Mon_1_temperatureDischarged_list))
    Mon_1_faceValue.append(sum(Mon_1_faceValue_list)/len(Mon_1_faceValue_list))
    Mon_1_dod.append(sum(Mon_1_dod_list)/len(Mon_1_dod_list)) 
    Mon_1_year.append(Mon_1[0]['date'][0:4])
    Mon_1_month.append(Mon_1[0]['date'][5:7])# 월 평균 데이터 계산
    Mon_1_sensorID.append(Mon_1[0]['sensorID'])
    Mon_1_chargedTimeTotal.append(sum(Mon_1_chargedTimeTotal_list))
    Mon_1_dischargedTimeTotal.append(sum(Mon_1_dischargedTimeTotal_list))
    Mon_1_overChargeTimeTotal.append(sum(Mon_1_overChargeTimeTotal_list)/60)
    Mon_1_overdisChargeTimeTotal.append(sum(Mon_1_overdisChargeTimeTotal_list)/60)
    Mon_1_abnormalTemperatureTimeTotal.append(sum(Mon_1_abnormalTemperatureTimeTotal_list)/60) 
    Mon_1_lackFaceValueTimeTotal.append(sum(Mon_1_lackFaceValueTimeTotal_list)/60)
    Mon_1_insufficientChargeTimeTotal.append(sum(Mon_1_insufficientChargeTimeTotal_list)/60)   
    Mon_1_insufficientCharge.append(sum(Mon_1_insufficientCharge_list)/len(Mon_1_insufficientCharge_list))
    Mon_1_meancycle.append(math.floor(sum(Mon_1_meancycle_list)/len(Mon_1_meancycle_list)))
    Mon_1_usedLifeMonth.append(round(sum(Mon_1_usedLifeMonth_list)/len(Mon_1_usedLifeMonth_list),1))
    
except ZeroDivisionError:
    pass

if len(Mon_1) > 0:
    if Mon_1_dod[0] < 40 :
        Dod_c = 1.5
    elif Mon_1_dod[0] >= 40 and Mon_1_dod[0] < 50:
        Dod_c = 1.42
    elif Mon_1_dod[0] >= 50 and Mon_1_dod[0] < 60:
        Dod_c = 1.25
    elif Mon_1_dod[0] >= 60 and Mon_1_dod[0] < 70:
        Dod_c = 1.08
    elif Mon_1_dod[0] >= 70 and Mon_1_dod[0] < 80:
        Dod_c = 0.92
    elif Mon_1_dod[0] >= 80 and Mon_1_dod[0] < 90:
        Dod_c = 0.75
    elif Mon_1_dod[0] >= 90 : # and DD <= 100 <-DD값이 낮아지면 다시 삽입
        Dod_c = 0.58
    else:
        pass

    if Mon_1_temperatureDischarged[0] < 30 :
        Mon_1_average_temperatureDischarged = 30
    else :
        Mon_1_average_temperatureDischarged = Mon_1_temperatureDischarged[0]

    Mon_1_voltaicDischarged_ln = round(ln((Mon_1_voltaicDischarged[0]/800)*5),1)

    if pd.isna(Mon_1_voltaicDischarged_ln) == True :
        Mon_1_voltaicDischarged_ln = 0
    else :
        pass

    Mon_1_expectedLifeMonth.append(round(((Dod_c*1200)*30*12/(Mon_1_average_temperatureDischarged*(1+(7/30)*Mon_1_voltaicDischarged_ln)*Mon_1_meancycle[0]*220))*1*0.85*1,1))
    Mon_1_expectedLifeCycle.append(round(((Dod_c*1200)*30/(Mon_1_average_temperatureDischarged*(1+(7/30)*Mon_1_voltaicDischarged_ln)))*1*0.85*1,1))

    if round((Mon_1_expectedLifeCycle[0] * 100) / 1200,1) > 100 :
        Mon_1_expectedLifePercent.append(100)
    else :
        Mon_1_expectedLifePercent.append(round((Mon_1_expectedLifeCycle[0] * 100) / 1200,1))

    Mon_1_usedLifePercent.append(round((Mon_1_ulcAverage[0] * 100) / 1200,1))


In [63]:
Mon_2_chargedTime_list = []
Mon_2_dischargedTime_list = []
Mon_2_elcAverage_list = []
Mon_2_ulcAverage_list = []
Mon_2_batteryState_list = []
Mon_2_overChargeCount_list = []
Mon_2_overdisChargeCount_list = []
Mon_2_abnormalTemperatureCount_list = []
Mon_2_lackFaceValueCount_list = []
Mon_2_insufficientChargeCount_list = []
Mon_2_voltaicCharged_list = []
Mon_2_voltaicDischarged_list = []
Mon_2_voltageCharged_list = []
Mon_2_voltageDischarged_list = []
Mon_2_temperatureCharged_list = []
Mon_2_temperatureDischarged_list = []
Mon_2_faceValue_list = []
Mon_2_dod_list = [] #계산식 저장 리스트
Mon_2_chargedTimeTotal_list = []
Mon_2_dischargedTimeTotal_list = []
Mon_2_overChargeTimeTotal_list = []
Mon_2_overdisChargeTimeTotal_list =[]
Mon_2_abnormalTemperatureTimeTotal_list = []
Mon_2_insufficientChargeTimeTotal_list = []
Mon_2_lackFaceValueTimeTotal_list = []
Mon_2_insufficientCharge_list = []
Mon_2_meancycle_list = []
Mon_2_usedLifeMonth_list = []

for i in range(len(Mon_2)): # 월 데이터 모으기
    Mon_2_chargedTime_list.append(Mon_2[i]['chargedTime'])
    Mon_2_dischargedTime_list.append(Mon_2[i]['dischargedTime'])
    Mon_2_elcAverage_list.append(Mon_2[i]['expectedLifeCycle'])
    Mon_2_ulcAverage_list.append(Mon_2[i]['usedLifeCycle'])
    Mon_2_batteryState_list.append(Mon_2[i]['batteryState'])
    Mon_2_overChargeCount_list.append(Mon_2[i]['overChargeCount'])
    Mon_2_overdisChargeCount_list.append(Mon_2[i]['overdisChargeCount'])
    Mon_2_abnormalTemperatureCount_list.append(Mon_2[i]['abnormalTemperatureCount'])
    Mon_2_lackFaceValueCount_list.append(Mon_2[i]['lackFaceValueCount'])
    Mon_2_insufficientChargeCount_list.append(Mon_2[i]['insufficientChargeCount'])
    Mon_2_voltaicCharged_list.append(Mon_2[i]['voltaicCharged'])
    Mon_2_voltaicDischarged_list.append(Mon_2[i]['voltaicDischarged'])
    Mon_2_voltageCharged_list.append(Mon_2[i]['voltageCharged'])
    Mon_2_voltageDischarged_list.append(Mon_2[i]['voltageDischarged'])
    Mon_2_temperatureCharged_list.append(Mon_2[i]['temperatureCharged'])
    Mon_2_temperatureDischarged_list.append(Mon_2[i]['temperatureDischarged'])
    Mon_2_faceValue_list.append(Mon_2[i]['faceValue'])
    Mon_2_dod_list.append(Mon_2[i]['dod']) 
    Mon_2_chargedTimeTotal_list.append(Mon_2[i]['chargedTime'])
    Mon_2_dischargedTimeTotal_list.append(Mon_2[i]['dischargedTime'])
    Mon_2_overChargeTimeTotal_list.append(Mon_2[i]['overChargeTime'])
    Mon_2_overdisChargeTimeTotal_list.append(Mon_2[i]['overdisChargeTime'])
    Mon_2_abnormalTemperatureTimeTotal_list.append(Mon_2[i]['abnormalTemperatureTime'])
    Mon_2_lackFaceValueTimeTotal_list.append(Mon_2[i]['lackFaceValueTime'])
    Mon_2_insufficientChargeTimeTotal_list.append(Mon_2[i]['insufficientChargeTime'])
    Mon_2_insufficientCharge_list.append(Mon_2[i]['insufficientCharge'])
    Mon_2_meancycle_list.append(Mon_2[i]['meancycle'])
    Mon_2_usedLifeMonth_list.append(Mon_2[i]['usedLifeMonth'])
    
Mon_2_chargedTime = []
Mon_2_dischargedTime = []
Mon_2_elcAverage = []
Mon_2_ulcAverage = []
Mon_2_batteryState = []
Mon_2_overChargeCount = []
Mon_2_overdisChargeCount = []
Mon_2_abnormalTemperatureCount = []
Mon_2_lackFaceValueCount = []
Mon_2_insufficientChargeCount = []
Mon_2_voltaicCharged = []
Mon_2_voltaicDischarged = []
Mon_2_voltageCharged = []
Mon_2_voltageDischarged = []
Mon_2_temperatureCharged = []
Mon_2_temperatureDischarged = []
Mon_2_faceValue = []
Mon_2_dod = [] # 월 데이터 리스트
Mon_2_year = []
Mon_2_month = []
Mon_2_sensorID = []
Mon_2_chargedTimeTotal = []
Mon_2_dischargedTimeTotal = []
Mon_2_overChargeTimeTotal = []
Mon_2_overdisChargeTimeTotal =[]
Mon_2_abnormalTemperatureTimeTotal = []
Mon_2_lackFaceValueTimeTotal = []
Mon_2_insufficientChargeTimeTotal = []
Mon_2_insufficientCharge = []
Mon_2_meancycle = []
Mon_2_expectedLifeMonth = []
Mon_2_expectedLifeCycle = []
Mon_2_usedLifeMonth = []
Mon_2_expectedLifePercent = []
Mon_2_usedLifePercent = []

try :
    Mon_2_chargedTime.append(sum(Mon_2_chargedTime_list)/len(Mon_2_chargedTime_list))
    Mon_2_dischargedTime.append(sum(Mon_2_dischargedTime_list)/len(Mon_2_dischargedTime_list))
    Mon_2_elcAverage.append(sum(Mon_2_elcAverage_list)/len(Mon_2_elcAverage_list))
    Mon_2_ulcAverage.append(sum(Mon_2_ulcAverage_list)/len(Mon_2_ulcAverage_list))
    Mon_2_batteryState.append(Mon_2[-1]['batteryState'])
    Mon_2_overChargeCount.append(sum(Mon_2_overChargeCount_list)/len(Mon_2_overChargeCount_list))
    Mon_2_overdisChargeCount.append(sum(Mon_2_overdisChargeCount_list)/len(Mon_2_overdisChargeCount_list))
    Mon_2_abnormalTemperatureCount.append(sum(Mon_2_abnormalTemperatureCount_list)/len(Mon_2_abnormalTemperatureCount_list))
    Mon_2_lackFaceValueCount.append(sum(Mon_2_lackFaceValueCount_list)/len(Mon_2_lackFaceValueCount_list))
    Mon_2_insufficientChargeCount.append(sum(Mon_2_insufficientChargeCount_list)/len(Mon_2_insufficientChargeCount_list))
    Mon_2_voltaicCharged.append(sum(Mon_2_voltaicCharged_list)/len(Mon_2_voltaicCharged_list))
    Mon_2_voltaicDischarged.append(sum(Mon_2_voltaicDischarged_list)/len(Mon_2_voltaicDischarged_list))
    Mon_2_voltageCharged.append(sum(Mon_2_voltageCharged_list)/len(Mon_2_voltageCharged_list))
    Mon_2_voltageDischarged.append(sum(Mon_2_voltageDischarged_list)/len(Mon_2_voltageDischarged_list))
    Mon_2_temperatureCharged.append(sum(Mon_2_temperatureCharged_list)/len(Mon_2_temperatureCharged_list))
    Mon_2_temperatureDischarged.append(sum(Mon_2_temperatureDischarged_list)/len(Mon_2_temperatureDischarged_list))
    Mon_2_faceValue.append(sum(Mon_2_faceValue_list)/len(Mon_2_faceValue_list))
    Mon_2_dod.append(sum(Mon_2_dod_list)/len(Mon_2_dod_list)) 
    Mon_2_year.append(Mon_2[0]['date'][0:4])
    Mon_2_month.append(Mon_2[0]['date'][5:7])# 월 평균 데이터 계산
    Mon_2_sensorID.append(Mon_2[0]['sensorID'])
    Mon_2_chargedTimeTotal.append(sum(Mon_2_chargedTimeTotal_list))
    Mon_2_dischargedTimeTotal.append(sum(Mon_2_dischargedTimeTotal_list))
    Mon_2_overChargeTimeTotal.append(sum(Mon_2_overChargeTimeTotal_list)/60)
    Mon_2_overdisChargeTimeTotal.append(sum(Mon_2_overdisChargeTimeTotal_list)/60)
    Mon_2_abnormalTemperatureTimeTotal.append(sum(Mon_2_abnormalTemperatureTimeTotal_list)/60) 
    Mon_2_lackFaceValueTimeTotal.append(sum(Mon_2_lackFaceValueTimeTotal_list)/60)
    Mon_2_insufficientChargeTimeTotal.append(sum(Mon_2_insufficientChargeTimeTotal_list)/60)   
    Mon_2_insufficientCharge.append(sum(Mon_2_insufficientCharge_list)/len(Mon_2_insufficientCharge_list))
    Mon_2_meancycle.append(math.floor(sum(Mon_2_meancycle_list)/len(Mon_2_meancycle_list)))
    Mon_2_usedLifeMonth.append(round(sum(Mon_2_usedLifeMonth_list)/len(Mon_2_usedLifeMonth_list),1))
    
except ZeroDivisionError:
    pass

if len(Mon_2) > 0:
    if Mon_2_dod[0] < 40 :
        Dod_c = 1.5
    elif Mon_2_dod[0] >= 40 and Mon_2_dod[0] < 50:
        Dod_c = 1.42
    elif Mon_2_dod[0] >= 50 and Mon_2_dod[0] < 60:
        Dod_c = 1.25
    elif Mon_2_dod[0] >= 60 and Mon_2_dod[0] < 70:
        Dod_c = 1.08
    elif Mon_2_dod[0] >= 70 and Mon_2_dod[0] < 80:
        Dod_c = 0.92
    elif Mon_2_dod[0] >= 80 and Mon_2_dod[0] < 90:
        Dod_c = 0.75
    elif Mon_2_dod[0] >= 90 : # and DD <= 100 <-DD값이 낮아지면 다시 삽입
        Dod_c = 0.58
    else:
        pass

    if Mon_2_temperatureDischarged[0] < 30 :
        Mon_2_average_temperatureDischarged = 30
    else :
        Mon_2_average_temperatureDischarged = Mon_2_temperatureDischarged[0]

    Mon_2_voltaicDischarged_ln = round(ln((Mon_2_voltaicDischarged[0]/800)*5),1)

    if pd.isna(Mon_2_voltaicDischarged_ln) == True :
        Mon_2_voltaicDischarged_ln = 0
    else :
        pass

    Mon_2_expectedLifeMonth.append(round(((Dod_c*1200)*30*12/(Mon_2_average_temperatureDischarged*(1+(7/30)*Mon_2_voltaicDischarged_ln)*Mon_2_meancycle[0]*220))*1*0.85*1,1))
    Mon_2_expectedLifeCycle.append(round(((Dod_c*1200)*30/(Mon_2_average_temperatureDischarged*(1+(7/30)*Mon_2_voltaicDischarged_ln)))*1*0.85*1,1))

    if round((Mon_2_expectedLifeCycle[0] * 100) / 1200,1) > 100 :
        Mon_2_expectedLifePercent.append(100)
    else :
        Mon_2_expectedLifePercent.append(round((Mon_2_expectedLifeCycle[0] * 100) / 1200,1))

    Mon_2_usedLifePercent.append(round((Mon_2_ulcAverage[0] * 100) / 1200,1))


In [64]:
Mon_3_chargedTime_list = []
Mon_3_dischargedTime_list = []
Mon_3_elcAverage_list = []
Mon_3_ulcAverage_list = []
Mon_3_batteryState_list = []
Mon_3_overChargeCount_list = []
Mon_3_overdisChargeCount_list = []
Mon_3_abnormalTemperatureCount_list = []
Mon_3_lackFaceValueCount_list = []
Mon_3_insufficientChargeCount_list = []
Mon_3_voltaicCharged_list = []
Mon_3_voltaicDischarged_list = []
Mon_3_voltageCharged_list = []
Mon_3_voltageDischarged_list = []
Mon_3_temperatureCharged_list = []
Mon_3_temperatureDischarged_list = []
Mon_3_faceValue_list = []
Mon_3_dod_list = [] #계산식 저장 리스트
Mon_3_chargedTimeTotal_list = []
Mon_3_dischargedTimeTotal_list = []
Mon_3_overChargeTimeTotal_list = []
Mon_3_overdisChargeTimeTotal_list =[]
Mon_3_abnormalTemperatureTimeTotal_list = []
Mon_3_insufficientChargeTimeTotal_list = []
Mon_3_lackFaceValueTimeTotal_list = []
Mon_3_insufficientCharge_list = []
Mon_3_meancycle_list = []
Mon_3_usedLifeMonth_list = []

for i in range(len(Mon_3)): # 월 데이터 모으기
    Mon_3_chargedTime_list.append(Mon_3[i]['chargedTime'])
    Mon_3_dischargedTime_list.append(Mon_3[i]['dischargedTime'])
    Mon_3_elcAverage_list.append(Mon_3[i]['expectedLifeCycle'])
    Mon_3_ulcAverage_list.append(Mon_3[i]['usedLifeCycle'])
    Mon_3_batteryState_list.append(Mon_3[i]['batteryState'])
    Mon_3_overChargeCount_list.append(Mon_3[i]['overChargeCount'])
    Mon_3_overdisChargeCount_list.append(Mon_3[i]['overdisChargeCount'])
    Mon_3_abnormalTemperatureCount_list.append(Mon_3[i]['abnormalTemperatureCount'])
    Mon_3_lackFaceValueCount_list.append(Mon_3[i]['lackFaceValueCount'])
    Mon_3_insufficientChargeCount_list.append(Mon_3[i]['insufficientChargeCount'])
    Mon_3_voltaicCharged_list.append(Mon_3[i]['voltaicCharged'])
    Mon_3_voltaicDischarged_list.append(Mon_3[i]['voltaicDischarged'])
    Mon_3_voltageCharged_list.append(Mon_3[i]['voltageCharged'])
    Mon_3_voltageDischarged_list.append(Mon_3[i]['voltageDischarged'])
    Mon_3_temperatureCharged_list.append(Mon_3[i]['temperatureCharged'])
    Mon_3_temperatureDischarged_list.append(Mon_3[i]['temperatureDischarged'])
    Mon_3_faceValue_list.append(Mon_3[i]['faceValue'])
    Mon_3_dod_list.append(Mon_3[i]['dod']) 
    Mon_3_chargedTimeTotal_list.append(Mon_3[i]['chargedTime'])
    Mon_3_dischargedTimeTotal_list.append(Mon_3[i]['dischargedTime'])
    Mon_3_overChargeTimeTotal_list.append(Mon_3[i]['overChargeTime'])
    Mon_3_overdisChargeTimeTotal_list.append(Mon_3[i]['overdisChargeTime'])
    Mon_3_abnormalTemperatureTimeTotal_list.append(Mon_3[i]['abnormalTemperatureTime'])
    Mon_3_lackFaceValueTimeTotal_list.append(Mon_3[i]['lackFaceValueTime'])
    Mon_3_insufficientChargeTimeTotal_list.append(Mon_3[i]['insufficientChargeTime'])
    Mon_3_insufficientCharge_list.append(Mon_3[i]['insufficientCharge'])
    Mon_3_meancycle_list.append(Mon_3[i]['meancycle'])
    Mon_3_usedLifeMonth_list.append(Mon_3[i]['usedLifeMonth'])
    
Mon_3_chargedTime = []
Mon_3_dischargedTime = []
Mon_3_elcAverage = []
Mon_3_ulcAverage = []
Mon_3_batteryState = []
Mon_3_overChargeCount = []
Mon_3_overdisChargeCount = []
Mon_3_abnormalTemperatureCount = []
Mon_3_lackFaceValueCount = []
Mon_3_insufficientChargeCount = []
Mon_3_voltaicCharged = []
Mon_3_voltaicDischarged = []
Mon_3_voltageCharged = []
Mon_3_voltageDischarged = []
Mon_3_temperatureCharged = []
Mon_3_temperatureDischarged = []
Mon_3_faceValue = []
Mon_3_dod = [] # 월 데이터 리스트
Mon_3_year = []
Mon_3_month = []
Mon_3_sensorID = []
Mon_3_chargedTimeTotal = []
Mon_3_dischargedTimeTotal = []
Mon_3_overChargeTimeTotal = []
Mon_3_overdisChargeTimeTotal =[]
Mon_3_abnormalTemperatureTimeTotal = []
Mon_3_lackFaceValueTimeTotal = []
Mon_3_insufficientChargeTimeTotal = []
Mon_3_insufficientCharge = []
Mon_3_meancycle = []
Mon_3_expectedLifeMonth = []
Mon_3_expectedLifeCycle = []
Mon_3_usedLifeMonth = []
Mon_3_expectedLifePercent = []
Mon_3_usedLifePercent = []

try :
    Mon_3_chargedTime.append(sum(Mon_3_chargedTime_list)/len(Mon_3_chargedTime_list))
    Mon_3_dischargedTime.append(sum(Mon_3_dischargedTime_list)/len(Mon_3_dischargedTime_list))
    Mon_3_elcAverage.append(sum(Mon_3_elcAverage_list)/len(Mon_3_elcAverage_list))
    Mon_3_ulcAverage.append(sum(Mon_3_ulcAverage_list)/len(Mon_3_ulcAverage_list))
    Mon_3_batteryState.append(Mon_3[-1]['batteryState'])
    Mon_3_overChargeCount.append(sum(Mon_3_overChargeCount_list)/len(Mon_3_overChargeCount_list))
    Mon_3_overdisChargeCount.append(sum(Mon_3_overdisChargeCount_list)/len(Mon_3_overdisChargeCount_list))
    Mon_3_abnormalTemperatureCount.append(sum(Mon_3_abnormalTemperatureCount_list)/len(Mon_3_abnormalTemperatureCount_list))
    Mon_3_lackFaceValueCount.append(sum(Mon_3_lackFaceValueCount_list)/len(Mon_3_lackFaceValueCount_list))
    Mon_3_insufficientChargeCount.append(sum(Mon_3_insufficientChargeCount_list)/len(Mon_3_insufficientChargeCount_list))
    Mon_3_voltaicCharged.append(sum(Mon_3_voltaicCharged_list)/len(Mon_3_voltaicCharged_list))
    Mon_3_voltaicDischarged.append(sum(Mon_3_voltaicDischarged_list)/len(Mon_3_voltaicDischarged_list))
    Mon_3_voltageCharged.append(sum(Mon_3_voltageCharged_list)/len(Mon_3_voltageCharged_list))
    Mon_3_voltageDischarged.append(sum(Mon_3_voltageDischarged_list)/len(Mon_3_voltageDischarged_list))
    Mon_3_temperatureCharged.append(sum(Mon_3_temperatureCharged_list)/len(Mon_3_temperatureCharged_list))
    Mon_3_temperatureDischarged.append(sum(Mon_3_temperatureDischarged_list)/len(Mon_3_temperatureDischarged_list))
    Mon_3_faceValue.append(sum(Mon_3_faceValue_list)/len(Mon_3_faceValue_list))
    Mon_3_dod.append(sum(Mon_3_dod_list)/len(Mon_3_dod_list)) 
    Mon_3_year.append(Mon_3[0]['date'][0:4])
    Mon_3_month.append(Mon_3[0]['date'][5:7])# 월 평균 데이터 계산
    Mon_3_sensorID.append(Mon_3[0]['sensorID'])
    Mon_3_chargedTimeTotal.append(sum(Mon_3_chargedTimeTotal_list))
    Mon_3_dischargedTimeTotal.append(sum(Mon_3_dischargedTimeTotal_list))
    Mon_3_overChargeTimeTotal.append(sum(Mon_3_overChargeTimeTotal_list)/60)
    Mon_3_overdisChargeTimeTotal.append(sum(Mon_3_overdisChargeTimeTotal_list)/60)
    Mon_3_abnormalTemperatureTimeTotal.append(sum(Mon_3_abnormalTemperatureTimeTotal_list)/60) 
    Mon_3_lackFaceValueTimeTotal.append(sum(Mon_3_lackFaceValueTimeTotal_list)/60)
    Mon_3_insufficientChargeTimeTotal.append(sum(Mon_3_insufficientChargeTimeTotal_list)/60)   
    Mon_3_insufficientCharge.append(sum(Mon_3_insufficientCharge_list)/len(Mon_3_insufficientCharge_list))
    Mon_3_meancycle.append(math.floor(sum(Mon_3_meancycle_list)/len(Mon_3_meancycle_list)))
    Mon_3_usedLifeMonth.append(round(sum(Mon_3_usedLifeMonth_list)/len(Mon_3_usedLifeMonth_list),1))
    
except ZeroDivisionError:
    pass

if len(Mon_3) > 0:
    if Mon_3_dod[0] < 40 :
        Dod_c = 1.5
    elif Mon_3_dod[0] >= 40 and Mon_3_dod[0] < 50:
        Dod_c = 1.42
    elif Mon_3_dod[0] >= 50 and Mon_3_dod[0] < 60:
        Dod_c = 1.25
    elif Mon_3_dod[0] >= 60 and Mon_3_dod[0] < 70:
        Dod_c = 1.08
    elif Mon_3_dod[0] >= 70 and Mon_3_dod[0] < 80:
        Dod_c = 0.92
    elif Mon_3_dod[0] >= 80 and Mon_3_dod[0] < 90:
        Dod_c = 0.75
    elif Mon_3_dod[0] >= 90 : # and DD <= 100 <-DD값이 낮아지면 다시 삽입
        Dod_c = 0.58
    else:
        pass

    if Mon_3_temperatureDischarged[0] < 30 :
        Mon_3_average_temperatureDischarged = 30
    else :
        Mon_3_average_temperatureDischarged = Mon_3_temperatureDischarged[0]

    Mon_3_voltaicDischarged_ln = round(ln((Mon_3_voltaicDischarged[0]/800)*5),1)

    if pd.isna(Mon_3_voltaicDischarged_ln) == True :
        Mon_3_voltaicDischarged_ln = 0
    else :
        pass

    Mon_3_expectedLifeMonth.append(round(((Dod_c*1200)*30*12/(Mon_3_average_temperatureDischarged*(1+(7/30)*Mon_3_voltaicDischarged_ln)*Mon_3_meancycle[0]*220))*1*0.85*1,1))
    Mon_3_expectedLifeCycle.append(round(((Dod_c*1200)*30/(Mon_3_average_temperatureDischarged*(1+(7/30)*Mon_3_voltaicDischarged_ln)))*1*0.85*1,1))

    if round((Mon_3_expectedLifeCycle[0] * 100) / 1200,1) > 100 :
        Mon_3_expectedLifePercent.append(100)
    else :
        Mon_3_expectedLifePercent.append(round((Mon_3_expectedLifeCycle[0] * 100) / 1200,1))

    Mon_3_usedLifePercent.append(round((Mon_3_ulcAverage[0] * 100) / 1200,1))


In [65]:
Mon_4_chargedTime_list = []
Mon_4_dischargedTime_list = []
Mon_4_elcAverage_list = []
Mon_4_ulcAverage_list = []
Mon_4_batteryState_list = []
Mon_4_overChargeCount_list = []
Mon_4_overdisChargeCount_list = []
Mon_4_abnormalTemperatureCount_list = []
Mon_4_lackFaceValueCount_list = []
Mon_4_insufficientChargeCount_list = []
Mon_4_voltaicCharged_list = []
Mon_4_voltaicDischarged_list = []
Mon_4_voltageCharged_list = []
Mon_4_voltageDischarged_list = []
Mon_4_temperatureCharged_list = []
Mon_4_temperatureDischarged_list = []
Mon_4_faceValue_list = []
Mon_4_dod_list = [] #계산식 저장 리스트
Mon_4_chargedTimeTotal_list = []
Mon_4_dischargedTimeTotal_list = []
Mon_4_overChargeTimeTotal_list = []
Mon_4_overdisChargeTimeTotal_list =[]
Mon_4_abnormalTemperatureTimeTotal_list = []
Mon_4_insufficientChargeTimeTotal_list = []
Mon_4_lackFaceValueTimeTotal_list = []
Mon_4_insufficientCharge_list = []
Mon_4_meancycle_list = []
Mon_4_usedLifeMonth_list = []

for i in range(len(Mon_4)): # 월 데이터 모으기
    Mon_4_chargedTime_list.append(Mon_4[i]['chargedTime'])
    Mon_4_dischargedTime_list.append(Mon_4[i]['dischargedTime'])
    Mon_4_elcAverage_list.append(Mon_4[i]['expectedLifeCycle'])
    Mon_4_ulcAverage_list.append(Mon_4[i]['usedLifeCycle'])
    Mon_4_batteryState_list.append(Mon_4[i]['batteryState'])
    Mon_4_overChargeCount_list.append(Mon_4[i]['overChargeCount'])
    Mon_4_overdisChargeCount_list.append(Mon_4[i]['overdisChargeCount'])
    Mon_4_abnormalTemperatureCount_list.append(Mon_4[i]['abnormalTemperatureCount'])
    Mon_4_lackFaceValueCount_list.append(Mon_4[i]['lackFaceValueCount'])
    Mon_4_insufficientChargeCount_list.append(Mon_4[i]['insufficientChargeCount'])
    Mon_4_voltaicCharged_list.append(Mon_4[i]['voltaicCharged'])
    Mon_4_voltaicDischarged_list.append(Mon_4[i]['voltaicDischarged'])
    Mon_4_voltageCharged_list.append(Mon_4[i]['voltageCharged'])
    Mon_4_voltageDischarged_list.append(Mon_4[i]['voltageDischarged'])
    Mon_4_temperatureCharged_list.append(Mon_4[i]['temperatureCharged'])
    Mon_4_temperatureDischarged_list.append(Mon_4[i]['temperatureDischarged'])
    Mon_4_faceValue_list.append(Mon_4[i]['faceValue'])
    Mon_4_dod_list.append(Mon_4[i]['dod']) 
    Mon_4_chargedTimeTotal_list.append(Mon_4[i]['chargedTime'])
    Mon_4_dischargedTimeTotal_list.append(Mon_4[i]['dischargedTime'])
    Mon_4_overChargeTimeTotal_list.append(Mon_4[i]['overChargeTime'])
    Mon_4_overdisChargeTimeTotal_list.append(Mon_4[i]['overdisChargeTime'])
    Mon_4_abnormalTemperatureTimeTotal_list.append(Mon_4[i]['abnormalTemperatureTime'])
    Mon_4_lackFaceValueTimeTotal_list.append(Mon_4[i]['lackFaceValueTime'])
    Mon_4_insufficientChargeTimeTotal_list.append(Mon_4[i]['insufficientChargeTime'])
    Mon_4_insufficientCharge_list.append(Mon_4[i]['insufficientCharge'])
    Mon_4_meancycle_list.append(Mon_4[i]['meancycle'])
    Mon_4_usedLifeMonth_list.append(Mon_4[i]['usedLifeMonth'])
    
Mon_4_chargedTime = []
Mon_4_dischargedTime = []
Mon_4_elcAverage = []
Mon_4_ulcAverage = []
Mon_4_batteryState = []
Mon_4_overChargeCount = []
Mon_4_overdisChargeCount = []
Mon_4_abnormalTemperatureCount = []
Mon_4_lackFaceValueCount = []
Mon_4_insufficientChargeCount = []
Mon_4_voltaicCharged = []
Mon_4_voltaicDischarged = []
Mon_4_voltageCharged = []
Mon_4_voltageDischarged = []
Mon_4_temperatureCharged = []
Mon_4_temperatureDischarged = []
Mon_4_faceValue = []
Mon_4_dod = [] # 월 데이터 리스트
Mon_4_year = []
Mon_4_month = []
Mon_4_sensorID = []
Mon_4_chargedTimeTotal = []
Mon_4_dischargedTimeTotal = []
Mon_4_overChargeTimeTotal = []
Mon_4_overdisChargeTimeTotal =[]
Mon_4_abnormalTemperatureTimeTotal = []
Mon_4_lackFaceValueTimeTotal = []
Mon_4_insufficientChargeTimeTotal = []
Mon_4_insufficientCharge = []
Mon_4_meancycle = []
Mon_4_expectedLifeMonth = []
Mon_4_expectedLifeCycle = []
Mon_4_usedLifeMonth = []
Mon_4_expectedLifePercent = []
Mon_4_usedLifePercent = []

try :
    Mon_4_chargedTime.append(sum(Mon_4_chargedTime_list)/len(Mon_4_chargedTime_list))
    Mon_4_dischargedTime.append(sum(Mon_4_dischargedTime_list)/len(Mon_4_dischargedTime_list))
    Mon_4_elcAverage.append(sum(Mon_4_elcAverage_list)/len(Mon_4_elcAverage_list))
    Mon_4_ulcAverage.append(sum(Mon_4_ulcAverage_list)/len(Mon_4_ulcAverage_list))
    Mon_4_batteryState.append(Mon_4[-1]['batteryState'])
    Mon_4_overChargeCount.append(sum(Mon_4_overChargeCount_list)/len(Mon_4_overChargeCount_list))
    Mon_4_overdisChargeCount.append(sum(Mon_4_overdisChargeCount_list)/len(Mon_4_overdisChargeCount_list))
    Mon_4_abnormalTemperatureCount.append(sum(Mon_4_abnormalTemperatureCount_list)/len(Mon_4_abnormalTemperatureCount_list))
    Mon_4_lackFaceValueCount.append(sum(Mon_4_lackFaceValueCount_list)/len(Mon_4_lackFaceValueCount_list))
    Mon_4_insufficientChargeCount.append(sum(Mon_4_insufficientChargeCount_list)/len(Mon_4_insufficientChargeCount_list))
    Mon_4_voltaicCharged.append(sum(Mon_4_voltaicCharged_list)/len(Mon_4_voltaicCharged_list))
    Mon_4_voltaicDischarged.append(sum(Mon_4_voltaicDischarged_list)/len(Mon_4_voltaicDischarged_list))
    Mon_4_voltageCharged.append(sum(Mon_4_voltageCharged_list)/len(Mon_4_voltageCharged_list))
    Mon_4_voltageDischarged.append(sum(Mon_4_voltageDischarged_list)/len(Mon_4_voltageDischarged_list))
    Mon_4_temperatureCharged.append(sum(Mon_4_temperatureCharged_list)/len(Mon_4_temperatureCharged_list))
    Mon_4_temperatureDischarged.append(sum(Mon_4_temperatureDischarged_list)/len(Mon_4_temperatureDischarged_list))
    Mon_4_faceValue.append(sum(Mon_4_faceValue_list)/len(Mon_4_faceValue_list))
    Mon_4_dod.append(sum(Mon_4_dod_list)/len(Mon_4_dod_list)) 
    Mon_4_year.append(Mon_4[0]['date'][0:4])
    Mon_4_month.append(Mon_4[0]['date'][5:7])# 월 평균 데이터 계산
    Mon_4_sensorID.append(Mon_4[0]['sensorID'])
    Mon_4_chargedTimeTotal.append(sum(Mon_4_chargedTimeTotal_list))
    Mon_4_dischargedTimeTotal.append(sum(Mon_4_dischargedTimeTotal_list))
    Mon_4_overChargeTimeTotal.append(sum(Mon_4_overChargeTimeTotal_list)/60)
    Mon_4_overdisChargeTimeTotal.append(sum(Mon_4_overdisChargeTimeTotal_list)/60)
    Mon_4_abnormalTemperatureTimeTotal.append(sum(Mon_4_abnormalTemperatureTimeTotal_list)/60) 
    Mon_4_lackFaceValueTimeTotal.append(sum(Mon_4_lackFaceValueTimeTotal_list)/60)
    Mon_4_insufficientChargeTimeTotal.append(sum(Mon_4_insufficientChargeTimeTotal_list)/60)   
    Mon_4_insufficientCharge.append(sum(Mon_4_insufficientCharge_list)/len(Mon_4_insufficientCharge_list))
    Mon_4_meancycle.append(math.floor(sum(Mon_4_meancycle_list)/len(Mon_4_meancycle_list)))
    Mon_4_usedLifeMonth.append(round(sum(Mon_4_usedLifeMonth_list)/len(Mon_4_usedLifeMonth_list),1))
    
except ZeroDivisionError:
    pass

if len(Mon_4) > 0:
    if Mon_4_dod[0] < 40 :
        Dod_c = 1.5
    elif Mon_4_dod[0] >= 40 and Mon_4_dod[0] < 50:
        Dod_c = 1.42
    elif Mon_4_dod[0] >= 50 and Mon_4_dod[0] < 60:
        Dod_c = 1.25
    elif Mon_4_dod[0] >= 60 and Mon_4_dod[0] < 70:
        Dod_c = 1.08
    elif Mon_4_dod[0] >= 70 and Mon_4_dod[0] < 80:
        Dod_c = 0.92
    elif Mon_4_dod[0] >= 80 and Mon_4_dod[0] < 90:
        Dod_c = 0.75
    elif Mon_4_dod[0] >= 90 : # and DD <= 100 <-DD값이 낮아지면 다시 삽입
        Dod_c = 0.58
    else:
        pass

    if Mon_4_temperatureDischarged[0] < 30 :
        Mon_4_average_temperatureDischarged = 30
    else :
        Mon_4_average_temperatureDischarged = Mon_4_temperatureDischarged[0]

    Mon_4_voltaicDischarged_ln = round(ln((Mon_4_voltaicDischarged[0]/800)*5),1)

    if pd.isna(Mon_4_voltaicDischarged_ln) == True :
        Mon_4_voltaicDischarged_ln = 0
    else :
        pass

    Mon_4_expectedLifeMonth.append(round(((Dod_c*1200)*30*12/(Mon_4_average_temperatureDischarged*(1+(7/30)*Mon_4_voltaicDischarged_ln)*Mon_4_meancycle[0]*220))*1*0.85*1,1))
    Mon_4_expectedLifeCycle.append(round(((Dod_c*1200)*30/(Mon_4_average_temperatureDischarged*(1+(7/30)*Mon_4_voltaicDischarged_ln)))*1*0.85*1,1))

    if round((Mon_4_expectedLifeCycle[0] * 100) / 1200,1) > 100 :
        Mon_4_expectedLifePercent.append(100)
    else :
        Mon_4_expectedLifePercent.append(round((Mon_4_expectedLifeCycle[0] * 100) / 1200,1))

    Mon_4_usedLifePercent.append(round((Mon_4_ulcAverage[0] * 100) / 1200,1))


In [66]:
Mon_5_chargedTime_list = []
Mon_5_dischargedTime_list = []
Mon_5_elcAverage_list = []
Mon_5_ulcAverage_list = []
Mon_5_batteryState_list = []
Mon_5_overChargeCount_list = []
Mon_5_overdisChargeCount_list = []
Mon_5_abnormalTemperatureCount_list = []
Mon_5_lackFaceValueCount_list = []
Mon_5_insufficientChargeCount_list = []
Mon_5_voltaicCharged_list = []
Mon_5_voltaicDischarged_list = []
Mon_5_voltageCharged_list = []
Mon_5_voltageDischarged_list = []
Mon_5_temperatureCharged_list = []
Mon_5_temperatureDischarged_list = []
Mon_5_faceValue_list = []
Mon_5_dod_list = [] #계산식 저장 리스트
Mon_5_chargedTimeTotal_list = []
Mon_5_dischargedTimeTotal_list = []
Mon_5_overChargeTimeTotal_list = []
Mon_5_overdisChargeTimeTotal_list =[]
Mon_5_abnormalTemperatureTimeTotal_list = []
Mon_5_insufficientChargeTimeTotal_list = []
Mon_5_lackFaceValueTimeTotal_list = []
Mon_5_insufficientCharge_list = []
Mon_5_meancycle_list = []
Mon_5_usedLifeMonth_list = []

for i in range(len(Mon_5)): # 월 데이터 모으기
    Mon_5_chargedTime_list.append(Mon_5[i]['chargedTime'])
    Mon_5_dischargedTime_list.append(Mon_5[i]['dischargedTime'])
    Mon_5_elcAverage_list.append(Mon_5[i]['expectedLifeCycle'])
    Mon_5_ulcAverage_list.append(Mon_5[i]['usedLifeCycle'])
    Mon_5_batteryState_list.append(Mon_5[i]['batteryState'])
    Mon_5_overChargeCount_list.append(Mon_5[i]['overChargeCount'])
    Mon_5_overdisChargeCount_list.append(Mon_5[i]['overdisChargeCount'])
    Mon_5_abnormalTemperatureCount_list.append(Mon_5[i]['abnormalTemperatureCount'])
    Mon_5_lackFaceValueCount_list.append(Mon_5[i]['lackFaceValueCount'])
    Mon_5_insufficientChargeCount_list.append(Mon_5[i]['insufficientChargeCount'])
    Mon_5_voltaicCharged_list.append(Mon_5[i]['voltaicCharged'])
    Mon_5_voltaicDischarged_list.append(Mon_5[i]['voltaicDischarged'])
    Mon_5_voltageCharged_list.append(Mon_5[i]['voltageCharged'])
    Mon_5_voltageDischarged_list.append(Mon_5[i]['voltageDischarged'])
    Mon_5_temperatureCharged_list.append(Mon_5[i]['temperatureCharged'])
    Mon_5_temperatureDischarged_list.append(Mon_5[i]['temperatureDischarged'])
    Mon_5_faceValue_list.append(Mon_5[i]['faceValue'])
    Mon_5_dod_list.append(Mon_5[i]['dod']) 
    Mon_5_chargedTimeTotal_list.append(Mon_5[i]['chargedTime'])
    Mon_5_dischargedTimeTotal_list.append(Mon_5[i]['dischargedTime'])
    Mon_5_overChargeTimeTotal_list.append(Mon_5[i]['overChargeTime'])
    Mon_5_overdisChargeTimeTotal_list.append(Mon_5[i]['overdisChargeTime'])
    Mon_5_abnormalTemperatureTimeTotal_list.append(Mon_5[i]['abnormalTemperatureTime'])
    Mon_5_lackFaceValueTimeTotal_list.append(Mon_5[i]['lackFaceValueTime'])
    Mon_5_insufficientChargeTimeTotal_list.append(Mon_5[i]['insufficientChargeTime'])
    Mon_5_insufficientCharge_list.append(Mon_5[i]['insufficientCharge'])
    Mon_5_meancycle_list.append(Mon_5[i]['meancycle'])
    Mon_5_usedLifeMonth_list.append(Mon_5[i]['usedLifeMonth'])
    
Mon_5_chargedTime = []
Mon_5_dischargedTime = []
Mon_5_elcAverage = []
Mon_5_ulcAverage = []
Mon_5_batteryState = []
Mon_5_overChargeCount = []
Mon_5_overdisChargeCount = []
Mon_5_abnormalTemperatureCount = []
Mon_5_lackFaceValueCount = []
Mon_5_insufficientChargeCount = []
Mon_5_voltaicCharged = []
Mon_5_voltaicDischarged = []
Mon_5_voltageCharged = []
Mon_5_voltageDischarged = []
Mon_5_temperatureCharged = []
Mon_5_temperatureDischarged = []
Mon_5_faceValue = []
Mon_5_dod = [] # 월 데이터 리스트
Mon_5_year = []
Mon_5_month = []
Mon_5_sensorID = []
Mon_5_chargedTimeTotal = []
Mon_5_dischargedTimeTotal = []
Mon_5_overChargeTimeTotal = []
Mon_5_overdisChargeTimeTotal =[]
Mon_5_abnormalTemperatureTimeTotal = []
Mon_5_lackFaceValueTimeTotal = []
Mon_5_insufficientChargeTimeTotal = []
Mon_5_insufficientCharge = []
Mon_5_meancycle = []
Mon_5_expectedLifeMonth = []
Mon_5_expectedLifeCycle = []
Mon_5_usedLifeMonth = []
Mon_5_expectedLifePercent = []
Mon_5_usedLifePercent = []

try :
    Mon_5_chargedTime.append(sum(Mon_5_chargedTime_list)/len(Mon_5_chargedTime_list))
    Mon_5_dischargedTime.append(sum(Mon_5_dischargedTime_list)/len(Mon_5_dischargedTime_list))
    Mon_5_elcAverage.append(sum(Mon_5_elcAverage_list)/len(Mon_5_elcAverage_list))
    Mon_5_ulcAverage.append(sum(Mon_5_ulcAverage_list)/len(Mon_5_ulcAverage_list))
    Mon_5_batteryState.append(Mon_5[-1]['batteryState'])
    Mon_5_overChargeCount.append(sum(Mon_5_overChargeCount_list)/len(Mon_5_overChargeCount_list))
    Mon_5_overdisChargeCount.append(sum(Mon_5_overdisChargeCount_list)/len(Mon_5_overdisChargeCount_list))
    Mon_5_abnormalTemperatureCount.append(sum(Mon_5_abnormalTemperatureCount_list)/len(Mon_5_abnormalTemperatureCount_list))
    Mon_5_lackFaceValueCount.append(sum(Mon_5_lackFaceValueCount_list)/len(Mon_5_lackFaceValueCount_list))
    Mon_5_insufficientChargeCount.append(sum(Mon_5_insufficientChargeCount_list)/len(Mon_5_insufficientChargeCount_list))
    Mon_5_voltaicCharged.append(sum(Mon_5_voltaicCharged_list)/len(Mon_5_voltaicCharged_list))
    Mon_5_voltaicDischarged.append(sum(Mon_5_voltaicDischarged_list)/len(Mon_5_voltaicDischarged_list))
    Mon_5_voltageCharged.append(sum(Mon_5_voltageCharged_list)/len(Mon_5_voltageCharged_list))
    Mon_5_voltageDischarged.append(sum(Mon_5_voltageDischarged_list)/len(Mon_5_voltageDischarged_list))
    Mon_5_temperatureCharged.append(sum(Mon_5_temperatureCharged_list)/len(Mon_5_temperatureCharged_list))
    Mon_5_temperatureDischarged.append(sum(Mon_5_temperatureDischarged_list)/len(Mon_5_temperatureDischarged_list))
    Mon_5_faceValue.append(sum(Mon_5_faceValue_list)/len(Mon_5_faceValue_list))
    Mon_5_dod.append(sum(Mon_5_dod_list)/len(Mon_5_dod_list)) 
    Mon_5_year.append(Mon_5[0]['date'][0:4])
    Mon_5_month.append(Mon_5[0]['date'][5:7])# 월 평균 데이터 계산
    Mon_5_sensorID.append(Mon_5[0]['sensorID'])
    Mon_5_chargedTimeTotal.append(sum(Mon_5_chargedTimeTotal_list))
    Mon_5_dischargedTimeTotal.append(sum(Mon_5_dischargedTimeTotal_list))
    Mon_5_overChargeTimeTotal.append(sum(Mon_5_overChargeTimeTotal_list)/60)
    Mon_5_overdisChargeTimeTotal.append(sum(Mon_5_overdisChargeTimeTotal_list)/60)
    Mon_5_abnormalTemperatureTimeTotal.append(sum(Mon_5_abnormalTemperatureTimeTotal_list)/60) 
    Mon_5_lackFaceValueTimeTotal.append(sum(Mon_5_lackFaceValueTimeTotal_list)/60)
    Mon_5_insufficientChargeTimeTotal.append(sum(Mon_5_insufficientChargeTimeTotal_list)/60)   
    Mon_5_insufficientCharge.append(sum(Mon_5_insufficientCharge_list)/len(Mon_5_insufficientCharge_list))
    Mon_5_meancycle.append(math.floor(sum(Mon_5_meancycle_list)/len(Mon_5_meancycle_list)))
    Mon_5_usedLifeMonth.append(round(sum(Mon_5_usedLifeMonth_list)/len(Mon_5_usedLifeMonth_list),1))
    
except ZeroDivisionError:
    pass

if len(Mon_5) > 0:
    if Mon_5_dod[0] < 40 :
        Dod_c = 1.5
    elif Mon_5_dod[0] >= 40 and Mon_5_dod[0] < 50:
        Dod_c = 1.42
    elif Mon_5_dod[0] >= 50 and Mon_5_dod[0] < 60:
        Dod_c = 1.25
    elif Mon_5_dod[0] >= 60 and Mon_5_dod[0] < 70:
        Dod_c = 1.08
    elif Mon_5_dod[0] >= 70 and Mon_5_dod[0] < 80:
        Dod_c = 0.92
    elif Mon_5_dod[0] >= 80 and Mon_5_dod[0] < 90:
        Dod_c = 0.75
    elif Mon_5_dod[0] >= 90 : # and DD <= 100 <-DD값이 낮아지면 다시 삽입
        Dod_c = 0.58
    else:
        pass

    if Mon_5_temperatureDischarged[0] < 30 :
        Mon_5_average_temperatureDischarged = 30
    else :
        Mon_5_average_temperatureDischarged = Mon_5_temperatureDischarged[0]

    Mon_5_voltaicDischarged_ln = round(ln((Mon_5_voltaicDischarged[0]/800)*5),1)

    if pd.isna(Mon_5_voltaicDischarged_ln) == True :
        Mon_5_voltaicDischarged_ln = 0
    else :
        pass

    Mon_5_expectedLifeMonth.append(round(((Dod_c*1200)*30*12/(Mon_5_average_temperatureDischarged*(1+(7/30)*Mon_5_voltaicDischarged_ln)*Mon_5_meancycle[0]*220))*1*0.85*1,1))
    Mon_5_expectedLifeCycle.append(round(((Dod_c*1200)*30/(Mon_5_average_temperatureDischarged*(1+(7/30)*Mon_5_voltaicDischarged_ln)))*1*0.85*1,1))

    if round((Mon_5_expectedLifeCycle[0] * 100) / 1200,1) > 100 :
        Mon_5_expectedLifePercent.append(100)
    else :
        Mon_5_expectedLifePercent.append(round((Mon_5_expectedLifeCycle[0] * 100) / 1200,1))

    Mon_5_usedLifePercent.append(round((Mon_5_ulcAverage[0] * 100) / 1200,1))


In [67]:
Mon_6_chargedTime_list = []
Mon_6_dischargedTime_list = []
Mon_6_elcAverage_list = []
Mon_6_ulcAverage_list = []
Mon_6_batteryState_list = []
Mon_6_overChargeCount_list = []
Mon_6_overdisChargeCount_list = []
Mon_6_abnormalTemperatureCount_list = []
Mon_6_lackFaceValueCount_list = []
Mon_6_insufficientChargeCount_list = []
Mon_6_voltaicCharged_list = []
Mon_6_voltaicDischarged_list = []
Mon_6_voltageCharged_list = []
Mon_6_voltageDischarged_list = []
Mon_6_temperatureCharged_list = []
Mon_6_temperatureDischarged_list = []
Mon_6_faceValue_list = []
Mon_6_dod_list = [] #계산식 저장 리스트
Mon_6_chargedTimeTotal_list = []
Mon_6_dischargedTimeTotal_list = []
Mon_6_overChargeTimeTotal_list = []
Mon_6_overdisChargeTimeTotal_list =[]
Mon_6_abnormalTemperatureTimeTotal_list = []
Mon_6_insufficientChargeTimeTotal_list = []
Mon_6_lackFaceValueTimeTotal_list = []
Mon_6_insufficientCharge_list = []
Mon_6_meancycle_list = []
Mon_6_usedLifeMonth_list = []

for i in range(len(Mon_6)): # 월 데이터 모으기
    Mon_6_chargedTime_list.append(Mon_6[i]['chargedTime'])
    Mon_6_dischargedTime_list.append(Mon_6[i]['dischargedTime'])
    Mon_6_elcAverage_list.append(Mon_6[i]['expectedLifeCycle'])
    Mon_6_ulcAverage_list.append(Mon_6[i]['usedLifeCycle'])
    Mon_6_batteryState_list.append(Mon_6[i]['batteryState'])
    Mon_6_overChargeCount_list.append(Mon_6[i]['overChargeCount'])
    Mon_6_overdisChargeCount_list.append(Mon_6[i]['overdisChargeCount'])
    Mon_6_abnormalTemperatureCount_list.append(Mon_6[i]['abnormalTemperatureCount'])
    Mon_6_lackFaceValueCount_list.append(Mon_6[i]['lackFaceValueCount'])
    Mon_6_insufficientChargeCount_list.append(Mon_6[i]['insufficientChargeCount'])
    Mon_6_voltaicCharged_list.append(Mon_6[i]['voltaicCharged'])
    Mon_6_voltaicDischarged_list.append(Mon_6[i]['voltaicDischarged'])
    Mon_6_voltageCharged_list.append(Mon_6[i]['voltageCharged'])
    Mon_6_voltageDischarged_list.append(Mon_6[i]['voltageDischarged'])
    Mon_6_temperatureCharged_list.append(Mon_6[i]['temperatureCharged'])
    Mon_6_temperatureDischarged_list.append(Mon_6[i]['temperatureDischarged'])
    Mon_6_faceValue_list.append(Mon_6[i]['faceValue'])
    Mon_6_dod_list.append(Mon_6[i]['dod']) 
    Mon_6_chargedTimeTotal_list.append(Mon_6[i]['chargedTime'])
    Mon_6_dischargedTimeTotal_list.append(Mon_6[i]['dischargedTime'])
    Mon_6_overChargeTimeTotal_list.append(Mon_6[i]['overChargeTime'])
    Mon_6_overdisChargeTimeTotal_list.append(Mon_6[i]['overdisChargeTime'])
    Mon_6_abnormalTemperatureTimeTotal_list.append(Mon_6[i]['abnormalTemperatureTime'])
    Mon_6_lackFaceValueTimeTotal_list.append(Mon_6[i]['lackFaceValueTime'])
    Mon_6_insufficientChargeTimeTotal_list.append(Mon_6[i]['insufficientChargeTime'])
    Mon_6_insufficientCharge_list.append(Mon_6[i]['insufficientCharge'])
    Mon_6_meancycle_list.append(Mon_6[i]['meancycle'])
    Mon_6_usedLifeMonth_list.append(Mon_6[i]['usedLifeMonth'])
    
Mon_6_chargedTime = []
Mon_6_dischargedTime = []
Mon_6_elcAverage = []
Mon_6_ulcAverage = []
Mon_6_batteryState = []
Mon_6_overChargeCount = []
Mon_6_overdisChargeCount = []
Mon_6_abnormalTemperatureCount = []
Mon_6_lackFaceValueCount = []
Mon_6_insufficientChargeCount = []
Mon_6_voltaicCharged = []
Mon_6_voltaicDischarged = []
Mon_6_voltageCharged = []
Mon_6_voltageDischarged = []
Mon_6_temperatureCharged = []
Mon_6_temperatureDischarged = []
Mon_6_faceValue = []
Mon_6_dod = [] # 월 데이터 리스트
Mon_6_year = []
Mon_6_month = []
Mon_6_sensorID = []
Mon_6_chargedTimeTotal = []
Mon_6_dischargedTimeTotal = []
Mon_6_overChargeTimeTotal = []
Mon_6_overdisChargeTimeTotal =[]
Mon_6_abnormalTemperatureTimeTotal = []
Mon_6_lackFaceValueTimeTotal = []
Mon_6_insufficientChargeTimeTotal = []
Mon_6_insufficientCharge = []
Mon_6_meancycle = []
Mon_6_expectedLifeMonth = []
Mon_6_expectedLifeCycle = []
Mon_6_usedLifeMonth = []
Mon_6_expectedLifePercent = []
Mon_6_usedLifePercent = []

try :
    Mon_6_chargedTime.append(sum(Mon_6_chargedTime_list)/len(Mon_6_chargedTime_list))
    Mon_6_dischargedTime.append(sum(Mon_6_dischargedTime_list)/len(Mon_6_dischargedTime_list))
    Mon_6_elcAverage.append(sum(Mon_6_elcAverage_list)/len(Mon_6_elcAverage_list))
    Mon_6_ulcAverage.append(sum(Mon_6_ulcAverage_list)/len(Mon_6_ulcAverage_list))
    Mon_6_batteryState.append(Mon_6[-1]['batteryState'])
    Mon_6_overChargeCount.append(sum(Mon_6_overChargeCount_list)/len(Mon_6_overChargeCount_list))
    Mon_6_overdisChargeCount.append(sum(Mon_6_overdisChargeCount_list)/len(Mon_6_overdisChargeCount_list))
    Mon_6_abnormalTemperatureCount.append(sum(Mon_6_abnormalTemperatureCount_list)/len(Mon_6_abnormalTemperatureCount_list))
    Mon_6_lackFaceValueCount.append(sum(Mon_6_lackFaceValueCount_list)/len(Mon_6_lackFaceValueCount_list))
    Mon_6_insufficientChargeCount.append(sum(Mon_6_insufficientChargeCount_list)/len(Mon_6_insufficientChargeCount_list))
    Mon_6_voltaicCharged.append(sum(Mon_6_voltaicCharged_list)/len(Mon_6_voltaicCharged_list))
    Mon_6_voltaicDischarged.append(sum(Mon_6_voltaicDischarged_list)/len(Mon_6_voltaicDischarged_list))
    Mon_6_voltageCharged.append(sum(Mon_6_voltageCharged_list)/len(Mon_6_voltageCharged_list))
    Mon_6_voltageDischarged.append(sum(Mon_6_voltageDischarged_list)/len(Mon_6_voltageDischarged_list))
    Mon_6_temperatureCharged.append(sum(Mon_6_temperatureCharged_list)/len(Mon_6_temperatureCharged_list))
    Mon_6_temperatureDischarged.append(sum(Mon_6_temperatureDischarged_list)/len(Mon_6_temperatureDischarged_list))
    Mon_6_faceValue.append(sum(Mon_6_faceValue_list)/len(Mon_6_faceValue_list))
    Mon_6_dod.append(sum(Mon_6_dod_list)/len(Mon_6_dod_list)) 
    Mon_6_year.append(Mon_6[0]['date'][0:4])
    Mon_6_month.append(Mon_6[0]['date'][5:7])# 월 평균 데이터 계산
    Mon_6_sensorID.append(Mon_6[0]['sensorID'])
    Mon_6_chargedTimeTotal.append(sum(Mon_6_chargedTimeTotal_list))
    Mon_6_dischargedTimeTotal.append(sum(Mon_6_dischargedTimeTotal_list))
    Mon_6_overChargeTimeTotal.append(sum(Mon_6_overChargeTimeTotal_list)/60)
    Mon_6_overdisChargeTimeTotal.append(sum(Mon_6_overdisChargeTimeTotal_list)/60)
    Mon_6_abnormalTemperatureTimeTotal.append(sum(Mon_6_abnormalTemperatureTimeTotal_list)/60) 
    Mon_6_lackFaceValueTimeTotal.append(sum(Mon_6_lackFaceValueTimeTotal_list)/60)
    Mon_6_insufficientChargeTimeTotal.append(sum(Mon_6_insufficientChargeTimeTotal_list)/60)   
    Mon_6_insufficientCharge.append(sum(Mon_6_insufficientCharge_list)/len(Mon_6_insufficientCharge_list))
    Mon_6_meancycle.append(math.floor(sum(Mon_6_meancycle_list)/len(Mon_6_meancycle_list)))
    Mon_6_usedLifeMonth.append(round(sum(Mon_6_usedLifeMonth_list)/len(Mon_6_usedLifeMonth_list),1))
    
except ZeroDivisionError:
    pass

if len(Mon_6) > 0:
    if Mon_6_dod[0] < 40 :
        Dod_c = 1.5
    elif Mon_6_dod[0] >= 40 and Mon_6_dod[0] < 50:
        Dod_c = 1.42
    elif Mon_6_dod[0] >= 50 and Mon_6_dod[0] < 60:
        Dod_c = 1.25
    elif Mon_6_dod[0] >= 60 and Mon_6_dod[0] < 70:
        Dod_c = 1.08
    elif Mon_6_dod[0] >= 70 and Mon_6_dod[0] < 80:
        Dod_c = 0.92
    elif Mon_6_dod[0] >= 80 and Mon_6_dod[0] < 90:
        Dod_c = 0.75
    elif Mon_6_dod[0] >= 90 : # and DD <= 100 <-DD값이 낮아지면 다시 삽입
        Dod_c = 0.58
    else:
        pass

    if Mon_6_temperatureDischarged[0] < 30 :
        Mon_6_average_temperatureDischarged = 30
    else :
        Mon_6_average_temperatureDischarged = Mon_6_temperatureDischarged[0]

    Mon_6_voltaicDischarged_ln = round(ln((Mon_6_voltaicDischarged[0]/800)*5),1)

    if pd.isna(Mon_6_voltaicDischarged_ln) == True :
        Mon_6_voltaicDischarged_ln = 0
    else :
        pass

    Mon_6_expectedLifeMonth.append(round(((Dod_c*1200)*30*12/(Mon_6_average_temperatureDischarged*(1+(7/30)*Mon_6_voltaicDischarged_ln)*Mon_6_meancycle[0]*220))*1*0.85*1,1))
    Mon_6_expectedLifeCycle.append(round(((Dod_c*1200)*30/(Mon_6_average_temperatureDischarged*(1+(7/30)*Mon_6_voltaicDischarged_ln)))*1*0.85*1,1))

    if round((Mon_6_expectedLifeCycle[0] * 100) / 1200,1) > 100 :
        Mon_6_expectedLifePercent.append(100)
    else :
        Mon_6_expectedLifePercent.append(round((Mon_6_expectedLifeCycle[0] * 100) / 1200,1))

    Mon_6_usedLifePercent.append(round((Mon_6_ulcAverage[0] * 100) / 1200,1))


In [68]:
Mon_7_chargedTime_list = []
Mon_7_dischargedTime_list = []
Mon_7_elcAverage_list = []
Mon_7_ulcAverage_list = []
Mon_7_batteryState_list = []
Mon_7_overChargeCount_list = []
Mon_7_overdisChargeCount_list = []
Mon_7_abnormalTemperatureCount_list = []
Mon_7_lackFaceValueCount_list = []
Mon_7_insufficientChargeCount_list = []
Mon_7_voltaicCharged_list = []
Mon_7_voltaicDischarged_list = []
Mon_7_voltageCharged_list = []
Mon_7_voltageDischarged_list = []
Mon_7_temperatureCharged_list = []
Mon_7_temperatureDischarged_list = []
Mon_7_faceValue_list = []
Mon_7_dod_list = [] #계산식 저장 리스트
Mon_7_chargedTimeTotal_list = []
Mon_7_dischargedTimeTotal_list = []
Mon_7_overChargeTimeTotal_list = []
Mon_7_overdisChargeTimeTotal_list =[]
Mon_7_abnormalTemperatureTimeTotal_list = []
Mon_7_insufficientChargeTimeTotal_list = []
Mon_7_lackFaceValueTimeTotal_list = []
Mon_7_insufficientCharge_list = []
Mon_7_meancycle_list = []
Mon_7_usedLifeMonth_list = []

for i in range(len(Mon_7)): # 월 데이터 모으기
    Mon_7_chargedTime_list.append(Mon_7[i]['chargedTime'])
    Mon_7_dischargedTime_list.append(Mon_7[i]['dischargedTime'])
    Mon_7_elcAverage_list.append(Mon_7[i]['expectedLifeCycle'])
    Mon_7_ulcAverage_list.append(Mon_7[i]['usedLifeCycle'])
    Mon_7_batteryState_list.append(Mon_7[i]['batteryState'])
    Mon_7_overChargeCount_list.append(Mon_7[i]['overChargeCount'])
    Mon_7_overdisChargeCount_list.append(Mon_7[i]['overdisChargeCount'])
    Mon_7_abnormalTemperatureCount_list.append(Mon_7[i]['abnormalTemperatureCount'])
    Mon_7_lackFaceValueCount_list.append(Mon_7[i]['lackFaceValueCount'])
    Mon_7_insufficientChargeCount_list.append(Mon_7[i]['insufficientChargeCount'])
    Mon_7_voltaicCharged_list.append(Mon_7[i]['voltaicCharged'])
    Mon_7_voltaicDischarged_list.append(Mon_7[i]['voltaicDischarged'])
    Mon_7_voltageCharged_list.append(Mon_7[i]['voltageCharged'])
    Mon_7_voltageDischarged_list.append(Mon_7[i]['voltageDischarged'])
    Mon_7_temperatureCharged_list.append(Mon_7[i]['temperatureCharged'])
    Mon_7_temperatureDischarged_list.append(Mon_7[i]['temperatureDischarged'])
    Mon_7_faceValue_list.append(Mon_7[i]['faceValue'])
    Mon_7_dod_list.append(Mon_7[i]['dod']) 
    Mon_7_chargedTimeTotal_list.append(Mon_7[i]['chargedTime'])
    Mon_7_dischargedTimeTotal_list.append(Mon_7[i]['dischargedTime'])
    Mon_7_overChargeTimeTotal_list.append(Mon_7[i]['overChargeTime'])
    Mon_7_overdisChargeTimeTotal_list.append(Mon_7[i]['overdisChargeTime'])
    Mon_7_abnormalTemperatureTimeTotal_list.append(Mon_7[i]['abnormalTemperatureTime'])
    Mon_7_lackFaceValueTimeTotal_list.append(Mon_7[i]['lackFaceValueTime'])
    Mon_7_insufficientChargeTimeTotal_list.append(Mon_7[i]['insufficientChargeTime'])
    Mon_7_insufficientCharge_list.append(Mon_7[i]['insufficientCharge'])
    Mon_7_meancycle_list.append(Mon_7[i]['meancycle'])
    Mon_7_usedLifeMonth_list.append(Mon_7[i]['usedLifeMonth'])
    
Mon_7_chargedTime = []
Mon_7_dischargedTime = []
Mon_7_elcAverage = []
Mon_7_ulcAverage = []
Mon_7_batteryState = []
Mon_7_overChargeCount = []
Mon_7_overdisChargeCount = []
Mon_7_abnormalTemperatureCount = []
Mon_7_lackFaceValueCount = []
Mon_7_insufficientChargeCount = []
Mon_7_voltaicCharged = []
Mon_7_voltaicDischarged = []
Mon_7_voltageCharged = []
Mon_7_voltageDischarged = []
Mon_7_temperatureCharged = []
Mon_7_temperatureDischarged = []
Mon_7_faceValue = []
Mon_7_dod = [] # 월 데이터 리스트
Mon_7_year = []
Mon_7_month = []
Mon_7_sensorID = []
Mon_7_chargedTimeTotal = []
Mon_7_dischargedTimeTotal = []
Mon_7_overChargeTimeTotal = []
Mon_7_overdisChargeTimeTotal =[]
Mon_7_abnormalTemperatureTimeTotal = []
Mon_7_lackFaceValueTimeTotal = []
Mon_7_insufficientChargeTimeTotal = []
Mon_7_insufficientCharge = []
Mon_7_meancycle = []
Mon_7_expectedLifeMonth = []
Mon_7_expectedLifeCycle = []
Mon_7_usedLifeMonth = []
Mon_7_expectedLifePercent = []
Mon_7_usedLifePercent = []

try :
    Mon_7_chargedTime.append(sum(Mon_7_chargedTime_list)/len(Mon_7_chargedTime_list))
    Mon_7_dischargedTime.append(sum(Mon_7_dischargedTime_list)/len(Mon_7_dischargedTime_list))
    Mon_7_elcAverage.append(sum(Mon_7_elcAverage_list)/len(Mon_7_elcAverage_list))
    Mon_7_ulcAverage.append(sum(Mon_7_ulcAverage_list)/len(Mon_7_ulcAverage_list))
    Mon_7_batteryState.append(Mon_7[-1]['batteryState'])
    Mon_7_overChargeCount.append(sum(Mon_7_overChargeCount_list)/len(Mon_7_overChargeCount_list))
    Mon_7_overdisChargeCount.append(sum(Mon_7_overdisChargeCount_list)/len(Mon_7_overdisChargeCount_list))
    Mon_7_abnormalTemperatureCount.append(sum(Mon_7_abnormalTemperatureCount_list)/len(Mon_7_abnormalTemperatureCount_list))
    Mon_7_lackFaceValueCount.append(sum(Mon_7_lackFaceValueCount_list)/len(Mon_7_lackFaceValueCount_list))
    Mon_7_insufficientChargeCount.append(sum(Mon_7_insufficientChargeCount_list)/len(Mon_7_insufficientChargeCount_list))
    Mon_7_voltaicCharged.append(sum(Mon_7_voltaicCharged_list)/len(Mon_7_voltaicCharged_list))
    Mon_7_voltaicDischarged.append(sum(Mon_7_voltaicDischarged_list)/len(Mon_7_voltaicDischarged_list))
    Mon_7_voltageCharged.append(sum(Mon_7_voltageCharged_list)/len(Mon_7_voltageCharged_list))
    Mon_7_voltageDischarged.append(sum(Mon_7_voltageDischarged_list)/len(Mon_7_voltageDischarged_list))
    Mon_7_temperatureCharged.append(sum(Mon_7_temperatureCharged_list)/len(Mon_7_temperatureCharged_list))
    Mon_7_temperatureDischarged.append(sum(Mon_7_temperatureDischarged_list)/len(Mon_7_temperatureDischarged_list))
    Mon_7_faceValue.append(sum(Mon_7_faceValue_list)/len(Mon_7_faceValue_list))
    Mon_7_dod.append(sum(Mon_7_dod_list)/len(Mon_7_dod_list)) 
    Mon_7_year.append(Mon_7[0]['date'][0:4])
    Mon_7_month.append(Mon_7[0]['date'][5:7])# 월 평균 데이터 계산
    Mon_7_sensorID.append(Mon_7[0]['sensorID'])
    Mon_7_chargedTimeTotal.append(sum(Mon_7_chargedTimeTotal_list))
    Mon_7_dischargedTimeTotal.append(sum(Mon_7_dischargedTimeTotal_list))
    Mon_7_overChargeTimeTotal.append(sum(Mon_7_overChargeTimeTotal_list)/60)
    Mon_7_overdisChargeTimeTotal.append(sum(Mon_7_overdisChargeTimeTotal_list)/60)
    Mon_7_abnormalTemperatureTimeTotal.append(sum(Mon_7_abnormalTemperatureTimeTotal_list)/60) 
    Mon_7_lackFaceValueTimeTotal.append(sum(Mon_7_lackFaceValueTimeTotal_list)/60)
    Mon_7_insufficientChargeTimeTotal.append(sum(Mon_7_insufficientChargeTimeTotal_list)/60)   
    Mon_7_insufficientCharge.append(sum(Mon_7_insufficientCharge_list)/len(Mon_7_insufficientCharge_list))
    Mon_7_meancycle.append(math.floor(sum(Mon_7_meancycle_list)/len(Mon_7_meancycle_list)))
    Mon_7_usedLifeMonth.append(round(sum(Mon_7_usedLifeMonth_list)/len(Mon_7_usedLifeMonth_list),1))
    
except ZeroDivisionError:
    pass

if len(Mon_7) > 0:
    if Mon_7_dod[0] < 40 :
        Dod_c = 1.5
    elif Mon_7_dod[0] >= 40 and Mon_7_dod[0] < 50:
        Dod_c = 1.42
    elif Mon_7_dod[0] >= 50 and Mon_7_dod[0] < 60:
        Dod_c = 1.25
    elif Mon_7_dod[0] >= 60 and Mon_7_dod[0] < 70:
        Dod_c = 1.08
    elif Mon_7_dod[0] >= 70 and Mon_7_dod[0] < 80:
        Dod_c = 0.92
    elif Mon_7_dod[0] >= 80 and Mon_7_dod[0] < 90:
        Dod_c = 0.75
    elif Mon_7_dod[0] >= 90 : # and DD <= 100 <-DD값이 낮아지면 다시 삽입
        Dod_c = 0.58
    else:
        pass

    if Mon_7_temperatureDischarged[0] < 30 :
        Mon_7_average_temperatureDischarged = 30
    else :
        Mon_7_average_temperatureDischarged = Mon_7_temperatureDischarged[0]

    Mon_7_voltaicDischarged_ln = round(ln((Mon_7_voltaicDischarged[0]/800)*5),1)

    if pd.isna(Mon_7_voltaicDischarged_ln) == True :
        Mon_7_voltaicDischarged_ln = 0
    else :
        pass

    Mon_7_expectedLifeMonth.append(round(((Dod_c*1200)*30*12/(Mon_7_average_temperatureDischarged*(1+(7/30)*Mon_7_voltaicDischarged_ln)*Mon_7_meancycle[0]*220))*1*0.85*1,1))
    Mon_7_expectedLifeCycle.append(round(((Dod_c*1200)*30/(Mon_7_average_temperatureDischarged*(1+(7/30)*Mon_7_voltaicDischarged_ln)))*1*0.85*1,1))

    if round((Mon_7_expectedLifeCycle[0] * 100) / 1200,1) > 100 :
        Mon_7_expectedLifePercent.append(100)
    else :
        Mon_7_expectedLifePercent.append(round((Mon_7_expectedLifeCycle[0] * 100) / 1200,1))

    Mon_7_usedLifePercent.append(round((Mon_7_ulcAverage[0] * 100) / 1200,1))


In [69]:
Mon_8_chargedTime_list = []
Mon_8_dischargedTime_list = []
Mon_8_elcAverage_list = []
Mon_8_ulcAverage_list = []
Mon_8_batteryState_list = []
Mon_8_overChargeCount_list = []
Mon_8_overdisChargeCount_list = []
Mon_8_abnormalTemperatureCount_list = []
Mon_8_lackFaceValueCount_list = []
Mon_8_insufficientChargeCount_list = []
Mon_8_voltaicCharged_list = []
Mon_8_voltaicDischarged_list = []
Mon_8_voltageCharged_list = []
Mon_8_voltageDischarged_list = []
Mon_8_temperatureCharged_list = []
Mon_8_temperatureDischarged_list = []
Mon_8_faceValue_list = []
Mon_8_dod_list = [] #계산식 저장 리스트
Mon_8_chargedTimeTotal_list = []
Mon_8_dischargedTimeTotal_list = []
Mon_8_overChargeTimeTotal_list = []
Mon_8_overdisChargeTimeTotal_list =[]
Mon_8_abnormalTemperatureTimeTotal_list = []
Mon_8_insufficientChargeTimeTotal_list = []
Mon_8_lackFaceValueTimeTotal_list = []
Mon_8_insufficientCharge_list = []
Mon_8_meancycle_list = []
Mon_8_usedLifeMonth_list = []

for i in range(len(Mon_8)): # 월 데이터 모으기
    Mon_8_chargedTime_list.append(Mon_8[i]['chargedTime'])
    Mon_8_dischargedTime_list.append(Mon_8[i]['dischargedTime'])
    Mon_8_elcAverage_list.append(Mon_8[i]['expectedLifeCycle'])
    Mon_8_ulcAverage_list.append(Mon_8[i]['usedLifeCycle'])
    Mon_8_batteryState_list.append(Mon_8[i]['batteryState'])
    Mon_8_overChargeCount_list.append(Mon_8[i]['overChargeCount'])
    Mon_8_overdisChargeCount_list.append(Mon_8[i]['overdisChargeCount'])
    Mon_8_abnormalTemperatureCount_list.append(Mon_8[i]['abnormalTemperatureCount'])
    Mon_8_lackFaceValueCount_list.append(Mon_8[i]['lackFaceValueCount'])
    Mon_8_insufficientChargeCount_list.append(Mon_8[i]['insufficientChargeCount'])
    Mon_8_voltaicCharged_list.append(Mon_8[i]['voltaicCharged'])
    Mon_8_voltaicDischarged_list.append(Mon_8[i]['voltaicDischarged'])
    Mon_8_voltageCharged_list.append(Mon_8[i]['voltageCharged'])
    Mon_8_voltageDischarged_list.append(Mon_8[i]['voltageDischarged'])
    Mon_8_temperatureCharged_list.append(Mon_8[i]['temperatureCharged'])
    Mon_8_temperatureDischarged_list.append(Mon_8[i]['temperatureDischarged'])
    Mon_8_faceValue_list.append(Mon_8[i]['faceValue'])
    Mon_8_dod_list.append(Mon_8[i]['dod']) 
    Mon_8_chargedTimeTotal_list.append(Mon_8[i]['chargedTime'])
    Mon_8_dischargedTimeTotal_list.append(Mon_8[i]['dischargedTime'])
    Mon_8_overChargeTimeTotal_list.append(Mon_8[i]['overChargeTime'])
    Mon_8_overdisChargeTimeTotal_list.append(Mon_8[i]['overdisChargeTime'])
    Mon_8_abnormalTemperatureTimeTotal_list.append(Mon_8[i]['abnormalTemperatureTime'])
    Mon_8_lackFaceValueTimeTotal_list.append(Mon_8[i]['lackFaceValueTime'])
    Mon_8_insufficientChargeTimeTotal_list.append(Mon_8[i]['insufficientChargeTime'])
    Mon_8_insufficientCharge_list.append(Mon_8[i]['insufficientCharge'])
    Mon_8_meancycle_list.append(Mon_8[i]['meancycle'])
    Mon_8_usedLifeMonth_list.append(Mon_8[i]['usedLifeMonth'])
    
Mon_8_chargedTime = []
Mon_8_dischargedTime = []
Mon_8_elcAverage = []
Mon_8_ulcAverage = []
Mon_8_batteryState = []
Mon_8_overChargeCount = []
Mon_8_overdisChargeCount = []
Mon_8_abnormalTemperatureCount = []
Mon_8_lackFaceValueCount = []
Mon_8_insufficientChargeCount = []
Mon_8_voltaicCharged = []
Mon_8_voltaicDischarged = []
Mon_8_voltageCharged = []
Mon_8_voltageDischarged = []
Mon_8_temperatureCharged = []
Mon_8_temperatureDischarged = []
Mon_8_faceValue = []
Mon_8_dod = [] # 월 데이터 리스트
Mon_8_year = []
Mon_8_month = []
Mon_8_sensorID = []
Mon_8_chargedTimeTotal = []
Mon_8_dischargedTimeTotal = []
Mon_8_overChargeTimeTotal = []
Mon_8_overdisChargeTimeTotal =[]
Mon_8_abnormalTemperatureTimeTotal = []
Mon_8_lackFaceValueTimeTotal = []
Mon_8_insufficientChargeTimeTotal = []
Mon_8_insufficientCharge = []
Mon_8_meancycle = []
Mon_8_expectedLifeMonth = []
Mon_8_expectedLifeCycle = []
Mon_8_usedLifeMonth = []
Mon_8_expectedLifePercent = []
Mon_8_usedLifePercent = []

try :
    Mon_8_chargedTime.append(sum(Mon_8_chargedTime_list)/len(Mon_8_chargedTime_list))
    Mon_8_dischargedTime.append(sum(Mon_8_dischargedTime_list)/len(Mon_8_dischargedTime_list))
    Mon_8_elcAverage.append(sum(Mon_8_elcAverage_list)/len(Mon_8_elcAverage_list))
    Mon_8_ulcAverage.append(sum(Mon_8_ulcAverage_list)/len(Mon_8_ulcAverage_list))
    Mon_8_batteryState.append(Mon_8[-1]['batteryState'])
    Mon_8_overChargeCount.append(sum(Mon_8_overChargeCount_list)/len(Mon_8_overChargeCount_list))
    Mon_8_overdisChargeCount.append(sum(Mon_8_overdisChargeCount_list)/len(Mon_8_overdisChargeCount_list))
    Mon_8_abnormalTemperatureCount.append(sum(Mon_8_abnormalTemperatureCount_list)/len(Mon_8_abnormalTemperatureCount_list))
    Mon_8_lackFaceValueCount.append(sum(Mon_8_lackFaceValueCount_list)/len(Mon_8_lackFaceValueCount_list))
    Mon_8_insufficientChargeCount.append(sum(Mon_8_insufficientChargeCount_list)/len(Mon_8_insufficientChargeCount_list))
    Mon_8_voltaicCharged.append(sum(Mon_8_voltaicCharged_list)/len(Mon_8_voltaicCharged_list))
    Mon_8_voltaicDischarged.append(sum(Mon_8_voltaicDischarged_list)/len(Mon_8_voltaicDischarged_list))
    Mon_8_voltageCharged.append(sum(Mon_8_voltageCharged_list)/len(Mon_8_voltageCharged_list))
    Mon_8_voltageDischarged.append(sum(Mon_8_voltageDischarged_list)/len(Mon_8_voltageDischarged_list))
    Mon_8_temperatureCharged.append(sum(Mon_8_temperatureCharged_list)/len(Mon_8_temperatureCharged_list))
    Mon_8_temperatureDischarged.append(sum(Mon_8_temperatureDischarged_list)/len(Mon_8_temperatureDischarged_list))
    Mon_8_faceValue.append(sum(Mon_8_faceValue_list)/len(Mon_8_faceValue_list))
    Mon_8_dod.append(sum(Mon_8_dod_list)/len(Mon_8_dod_list)) 
    Mon_8_year.append(Mon_8[0]['date'][0:4])
    Mon_8_month.append(Mon_8[0]['date'][5:7])# 월 평균 데이터 계산
    Mon_8_sensorID.append(Mon_8[0]['sensorID'])
    Mon_8_chargedTimeTotal.append(sum(Mon_8_chargedTimeTotal_list))
    Mon_8_dischargedTimeTotal.append(sum(Mon_8_dischargedTimeTotal_list))
    Mon_8_overChargeTimeTotal.append(sum(Mon_8_overChargeTimeTotal_list)/60)
    Mon_8_overdisChargeTimeTotal.append(sum(Mon_8_overdisChargeTimeTotal_list)/60)
    Mon_8_abnormalTemperatureTimeTotal.append(sum(Mon_8_abnormalTemperatureTimeTotal_list)/60) 
    Mon_8_lackFaceValueTimeTotal.append(sum(Mon_8_lackFaceValueTimeTotal_list)/60)
    Mon_8_insufficientChargeTimeTotal.append(sum(Mon_8_insufficientChargeTimeTotal_list)/60)   
    Mon_8_insufficientCharge.append(sum(Mon_8_insufficientCharge_list)/len(Mon_8_insufficientCharge_list))
    Mon_8_meancycle.append(math.floor(sum(Mon_8_meancycle_list)/len(Mon_8_meancycle_list)))
    Mon_8_usedLifeMonth.append(round(sum(Mon_8_usedLifeMonth_list)/len(Mon_8_usedLifeMonth_list),1))
    
except ZeroDivisionError:
    pass

if len(Mon_8) > 0:
    if Mon_8_dod[0] < 40 :
        Dod_c = 1.5
    elif Mon_8_dod[0] >= 40 and Mon_8_dod[0] < 50:
        Dod_c = 1.42
    elif Mon_8_dod[0] >= 50 and Mon_8_dod[0] < 60:
        Dod_c = 1.25
    elif Mon_8_dod[0] >= 60 and Mon_8_dod[0] < 70:
        Dod_c = 1.08
    elif Mon_8_dod[0] >= 70 and Mon_8_dod[0] < 80:
        Dod_c = 0.92
    elif Mon_8_dod[0] >= 80 and Mon_8_dod[0] < 90:
        Dod_c = 0.75
    elif Mon_8_dod[0] >= 90 : # and DD <= 100 <-DD값이 낮아지면 다시 삽입
        Dod_c = 0.58
    else:
        pass

    if Mon_8_temperatureDischarged[0] < 30 :
        Mon_8_average_temperatureDischarged = 30
    else :
        Mon_8_average_temperatureDischarged = Mon_8_temperatureDischarged[0]

    Mon_8_voltaicDischarged_ln = round(ln((Mon_8_voltaicDischarged[0]/800)*5),1)

    if pd.isna(Mon_8_voltaicDischarged_ln) == True :
        Mon_8_voltaicDischarged_ln = 0
    else :
        pass

    Mon_8_expectedLifeMonth.append(round(((Dod_c*1200)*30*12/(Mon_8_average_temperatureDischarged*(1+(7/30)*Mon_8_voltaicDischarged_ln)*Mon_8_meancycle[0]*220))*1*0.85*1,1))
    Mon_8_expectedLifeCycle.append(round(((Dod_c*1200)*30/(Mon_8_average_temperatureDischarged*(1+(7/30)*Mon_8_voltaicDischarged_ln)))*1*0.85*1,1))

    if round((Mon_8_expectedLifeCycle[0] * 100) / 1200,1) > 100 :
        Mon_8_expectedLifePercent.append(100)
    else :
        Mon_8_expectedLifePercent.append(round((Mon_8_expectedLifeCycle[0] * 100) / 1200,1))

    Mon_8_usedLifePercent.append(round((Mon_8_ulcAverage[0] * 100) / 1200,1))


In [70]:
Mon_9_chargedTime_list = []
Mon_9_dischargedTime_list = []
Mon_9_elcAverage_list = []
Mon_9_ulcAverage_list = []
Mon_9_batteryState_list = []
Mon_9_overChargeCount_list = []
Mon_9_overdisChargeCount_list = []
Mon_9_abnormalTemperatureCount_list = []
Mon_9_lackFaceValueCount_list = []
Mon_9_insufficientChargeCount_list = []
Mon_9_voltaicCharged_list = []
Mon_9_voltaicDischarged_list = []
Mon_9_voltageCharged_list = []
Mon_9_voltageDischarged_list = []
Mon_9_temperatureCharged_list = []
Mon_9_temperatureDischarged_list = []
Mon_9_faceValue_list = []
Mon_9_dod_list = [] #계산식 저장 리스트
Mon_9_chargedTimeTotal_list = []
Mon_9_dischargedTimeTotal_list = []
Mon_9_overChargeTimeTotal_list = []
Mon_9_overdisChargeTimeTotal_list =[]
Mon_9_abnormalTemperatureTimeTotal_list = []
Mon_9_insufficientChargeTimeTotal_list = []
Mon_9_lackFaceValueTimeTotal_list = []
Mon_9_insufficientCharge_list = []
Mon_9_meancycle_list = []
Mon_9_usedLifeMonth_list = []

for i in range(len(Mon_9)): # 월 데이터 모으기
    Mon_9_chargedTime_list.append(Mon_9[i]['chargedTime'])
    Mon_9_dischargedTime_list.append(Mon_9[i]['dischargedTime'])
    Mon_9_elcAverage_list.append(Mon_9[i]['expectedLifeCycle'])
    Mon_9_ulcAverage_list.append(Mon_9[i]['usedLifeCycle'])
    Mon_9_batteryState_list.append(Mon_9[i]['batteryState'])
    Mon_9_overChargeCount_list.append(Mon_9[i]['overChargeCount'])
    Mon_9_overdisChargeCount_list.append(Mon_9[i]['overdisChargeCount'])
    Mon_9_abnormalTemperatureCount_list.append(Mon_9[i]['abnormalTemperatureCount'])
    Mon_9_lackFaceValueCount_list.append(Mon_9[i]['lackFaceValueCount'])
    Mon_9_insufficientChargeCount_list.append(Mon_9[i]['insufficientChargeCount'])
    Mon_9_voltaicCharged_list.append(Mon_9[i]['voltaicCharged'])
    Mon_9_voltaicDischarged_list.append(Mon_9[i]['voltaicDischarged'])
    Mon_9_voltageCharged_list.append(Mon_9[i]['voltageCharged'])
    Mon_9_voltageDischarged_list.append(Mon_9[i]['voltageDischarged'])
    Mon_9_temperatureCharged_list.append(Mon_9[i]['temperatureCharged'])
    Mon_9_temperatureDischarged_list.append(Mon_9[i]['temperatureDischarged'])
    Mon_9_faceValue_list.append(Mon_9[i]['faceValue'])
    Mon_9_dod_list.append(Mon_9[i]['dod']) 
    Mon_9_chargedTimeTotal_list.append(Mon_9[i]['chargedTime'])
    Mon_9_dischargedTimeTotal_list.append(Mon_9[i]['dischargedTime'])
    Mon_9_overChargeTimeTotal_list.append(Mon_9[i]['overChargeTime'])
    Mon_9_overdisChargeTimeTotal_list.append(Mon_9[i]['overdisChargeTime'])
    Mon_9_abnormalTemperatureTimeTotal_list.append(Mon_9[i]['abnormalTemperatureTime'])
    Mon_9_lackFaceValueTimeTotal_list.append(Mon_9[i]['lackFaceValueTime'])
    Mon_9_insufficientChargeTimeTotal_list.append(Mon_9[i]['insufficientChargeTime'])
    Mon_9_insufficientCharge_list.append(Mon_9[i]['insufficientCharge'])
    Mon_9_meancycle_list.append(Mon_9[i]['meancycle'])
    Mon_9_usedLifeMonth_list.append(Mon_9[i]['usedLifeMonth'])
    
Mon_9_chargedTime = []
Mon_9_dischargedTime = []
Mon_9_elcAverage = []
Mon_9_ulcAverage = []
Mon_9_batteryState = []
Mon_9_overChargeCount = []
Mon_9_overdisChargeCount = []
Mon_9_abnormalTemperatureCount = []
Mon_9_lackFaceValueCount = []
Mon_9_insufficientChargeCount = []
Mon_9_voltaicCharged = []
Mon_9_voltaicDischarged = []
Mon_9_voltageCharged = []
Mon_9_voltageDischarged = []
Mon_9_temperatureCharged = []
Mon_9_temperatureDischarged = []
Mon_9_faceValue = []
Mon_9_dod = [] # 월 데이터 리스트
Mon_9_year = []
Mon_9_month = []
Mon_9_sensorID = []
Mon_9_chargedTimeTotal = []
Mon_9_dischargedTimeTotal = []
Mon_9_overChargeTimeTotal = []
Mon_9_overdisChargeTimeTotal =[]
Mon_9_abnormalTemperatureTimeTotal = []
Mon_9_lackFaceValueTimeTotal = []
Mon_9_insufficientChargeTimeTotal = []
Mon_9_insufficientCharge = []
Mon_9_meancycle = []
Mon_9_expectedLifeMonth = []
Mon_9_expectedLifeCycle = []
Mon_9_usedLifeMonth = []
Mon_9_expectedLifePercent = []
Mon_9_usedLifePercent = []

try :
    Mon_9_chargedTime.append(sum(Mon_9_chargedTime_list)/len(Mon_9_chargedTime_list))
    Mon_9_dischargedTime.append(sum(Mon_9_dischargedTime_list)/len(Mon_9_dischargedTime_list))
    Mon_9_elcAverage.append(sum(Mon_9_elcAverage_list)/len(Mon_9_elcAverage_list))
    Mon_9_ulcAverage.append(sum(Mon_9_ulcAverage_list)/len(Mon_9_ulcAverage_list))
    Mon_9_batteryState.append(Mon_9[-1]['batteryState'])
    Mon_9_overChargeCount.append(sum(Mon_9_overChargeCount_list)/len(Mon_9_overChargeCount_list))
    Mon_9_overdisChargeCount.append(sum(Mon_9_overdisChargeCount_list)/len(Mon_9_overdisChargeCount_list))
    Mon_9_abnormalTemperatureCount.append(sum(Mon_9_abnormalTemperatureCount_list)/len(Mon_9_abnormalTemperatureCount_list))
    Mon_9_lackFaceValueCount.append(sum(Mon_9_lackFaceValueCount_list)/len(Mon_9_lackFaceValueCount_list))
    Mon_9_insufficientChargeCount.append(sum(Mon_9_insufficientChargeCount_list)/len(Mon_9_insufficientChargeCount_list))
    Mon_9_voltaicCharged.append(sum(Mon_9_voltaicCharged_list)/len(Mon_9_voltaicCharged_list))
    Mon_9_voltaicDischarged.append(sum(Mon_9_voltaicDischarged_list)/len(Mon_9_voltaicDischarged_list))
    Mon_9_voltageCharged.append(sum(Mon_9_voltageCharged_list)/len(Mon_9_voltageCharged_list))
    Mon_9_voltageDischarged.append(sum(Mon_9_voltageDischarged_list)/len(Mon_9_voltageDischarged_list))
    Mon_9_temperatureCharged.append(sum(Mon_9_temperatureCharged_list)/len(Mon_9_temperatureCharged_list))
    Mon_9_temperatureDischarged.append(sum(Mon_9_temperatureDischarged_list)/len(Mon_9_temperatureDischarged_list))
    Mon_9_faceValue.append(sum(Mon_9_faceValue_list)/len(Mon_9_faceValue_list))
    Mon_9_dod.append(sum(Mon_9_dod_list)/len(Mon_9_dod_list)) 
    Mon_9_year.append(Mon_9[0]['date'][0:4])
    Mon_9_month.append(Mon_9[0]['date'][5:7])# 월 평균 데이터 계산
    Mon_9_sensorID.append(Mon_9[0]['sensorID'])
    Mon_9_chargedTimeTotal.append(sum(Mon_9_chargedTimeTotal_list))
    Mon_9_dischargedTimeTotal.append(sum(Mon_9_dischargedTimeTotal_list))
    Mon_9_overChargeTimeTotal.append(sum(Mon_9_overChargeTimeTotal_list)/60)
    Mon_9_overdisChargeTimeTotal.append(sum(Mon_9_overdisChargeTimeTotal_list)/60)
    Mon_9_abnormalTemperatureTimeTotal.append(sum(Mon_9_abnormalTemperatureTimeTotal_list)/60) 
    Mon_9_lackFaceValueTimeTotal.append(sum(Mon_9_lackFaceValueTimeTotal_list)/60)
    Mon_9_insufficientChargeTimeTotal.append(sum(Mon_9_insufficientChargeTimeTotal_list)/60)   
    Mon_9_insufficientCharge.append(sum(Mon_9_insufficientCharge_list)/len(Mon_9_insufficientCharge_list))
    Mon_9_meancycle.append(math.floor(sum(Mon_9_meancycle_list)/len(Mon_9_meancycle_list)))
    Mon_9_usedLifeMonth.append(round(sum(Mon_9_usedLifeMonth_list)/len(Mon_9_usedLifeMonth_list),1))
    
except ZeroDivisionError:
    pass

if len(Mon_9) > 0:
    if Mon_9_dod[0] < 40 :
        Dod_c = 1.5
    elif Mon_9_dod[0] >= 40 and Mon_9_dod[0] < 50:
        Dod_c = 1.42
    elif Mon_9_dod[0] >= 50 and Mon_9_dod[0] < 60:
        Dod_c = 1.25
    elif Mon_9_dod[0] >= 60 and Mon_9_dod[0] < 70:
        Dod_c = 1.08
    elif Mon_9_dod[0] >= 70 and Mon_9_dod[0] < 80:
        Dod_c = 0.92
    elif Mon_9_dod[0] >= 80 and Mon_9_dod[0] < 90:
        Dod_c = 0.75
    elif Mon_9_dod[0] >= 90 : # and DD <= 100 <-DD값이 낮아지면 다시 삽입
        Dod_c = 0.58
    else:
        pass

    if Mon_9_temperatureDischarged[0] < 30 :
        Mon_9_average_temperatureDischarged = 30
    else :
        Mon_9_average_temperatureDischarged = Mon_9_temperatureDischarged[0]

    Mon_9_voltaicDischarged_ln = round(ln((Mon_9_voltaicDischarged[0]/800)*5),1)

    if pd.isna(Mon_9_voltaicDischarged_ln) == True :
        Mon_9_voltaicDischarged_ln = 0
    else :
        pass

    Mon_9_expectedLifeMonth.append(round(((Dod_c*1200)*30*12/(Mon_9_average_temperatureDischarged*(1+(7/30)*Mon_9_voltaicDischarged_ln)*Mon_9_meancycle[0]*220))*1*0.85*1,1))
    Mon_9_expectedLifeCycle.append(round(((Dod_c*1200)*30/(Mon_9_average_temperatureDischarged*(1+(7/30)*Mon_9_voltaicDischarged_ln)))*1*0.85*1,1))

    if round((Mon_9_expectedLifeCycle[0] * 100) / 1200,1) > 100 :
        Mon_9_expectedLifePercent.append(100)
    else :
        Mon_9_expectedLifePercent.append(round((Mon_9_expectedLifeCycle[0] * 100) / 1200,1))

    Mon_9_usedLifePercent.append(round((Mon_9_ulcAverage[0] * 100) / 1200,1))


In [71]:
Mon_10_chargedTime_list = []
Mon_10_dischargedTime_list = []
Mon_10_elcAverage_list = []
Mon_10_ulcAverage_list = []
Mon_10_batteryState_list = []
Mon_10_overChargeCount_list = []
Mon_10_overdisChargeCount_list = []
Mon_10_abnormalTemperatureCount_list = []
Mon_10_lackFaceValueCount_list = []
Mon_10_insufficientChargeCount_list = []
Mon_10_voltaicCharged_list = []
Mon_10_voltaicDischarged_list = []
Mon_10_voltageCharged_list = []
Mon_10_voltageDischarged_list = []
Mon_10_temperatureCharged_list = []
Mon_10_temperatureDischarged_list = []
Mon_10_faceValue_list = []
Mon_10_dod_list = [] #계산식 저장 리스트
Mon_10_chargedTimeTotal_list = []
Mon_10_dischargedTimeTotal_list = []
Mon_10_overChargeTimeTotal_list = []
Mon_10_overdisChargeTimeTotal_list =[]
Mon_10_abnormalTemperatureTimeTotal_list = []
Mon_10_insufficientChargeTimeTotal_list = []
Mon_10_lackFaceValueTimeTotal_list = []
Mon_10_insufficientCharge_list = []
Mon_10_meancycle_list = []
Mon_10_usedLifeMonth_list = []

for i in range(len(Mon_10)): # 월 데이터 모으기
    Mon_10_chargedTime_list.append(Mon_10[i]['chargedTime'])
    Mon_10_dischargedTime_list.append(Mon_10[i]['dischargedTime'])
    Mon_10_elcAverage_list.append(Mon_10[i]['expectedLifeCycle'])
    Mon_10_ulcAverage_list.append(Mon_10[i]['usedLifeCycle'])
    Mon_10_batteryState_list.append(Mon_10[i]['batteryState'])
    Mon_10_overChargeCount_list.append(Mon_10[i]['overChargeCount'])
    Mon_10_overdisChargeCount_list.append(Mon_10[i]['overdisChargeCount'])
    Mon_10_abnormalTemperatureCount_list.append(Mon_10[i]['abnormalTemperatureCount'])
    Mon_10_lackFaceValueCount_list.append(Mon_10[i]['lackFaceValueCount'])
    Mon_10_insufficientChargeCount_list.append(Mon_10[i]['insufficientChargeCount'])
    Mon_10_voltaicCharged_list.append(Mon_10[i]['voltaicCharged'])
    Mon_10_voltaicDischarged_list.append(Mon_10[i]['voltaicDischarged'])
    Mon_10_voltageCharged_list.append(Mon_10[i]['voltageCharged'])
    Mon_10_voltageDischarged_list.append(Mon_10[i]['voltageDischarged'])
    Mon_10_temperatureCharged_list.append(Mon_10[i]['temperatureCharged'])
    Mon_10_temperatureDischarged_list.append(Mon_10[i]['temperatureDischarged'])
    Mon_10_faceValue_list.append(Mon_10[i]['faceValue'])
    Mon_10_dod_list.append(Mon_10[i]['dod']) 
    Mon_10_chargedTimeTotal_list.append(Mon_10[i]['chargedTime'])
    Mon_10_dischargedTimeTotal_list.append(Mon_10[i]['dischargedTime'])
    Mon_10_overChargeTimeTotal_list.append(Mon_10[i]['overChargeTime'])
    Mon_10_overdisChargeTimeTotal_list.append(Mon_10[i]['overdisChargeTime'])
    Mon_10_abnormalTemperatureTimeTotal_list.append(Mon_10[i]['abnormalTemperatureTime'])
    Mon_10_lackFaceValueTimeTotal_list.append(Mon_10[i]['lackFaceValueTime'])
    Mon_10_insufficientChargeTimeTotal_list.append(Mon_10[i]['insufficientChargeTime'])
    Mon_10_insufficientCharge_list.append(Mon_10[i]['insufficientCharge'])
    Mon_10_meancycle_list.append(Mon_10[i]['meancycle'])
    Mon_10_usedLifeMonth_list.append(Mon_10[i]['usedLifeMonth'])
    
Mon_10_chargedTime = []
Mon_10_dischargedTime = []
Mon_10_elcAverage = []
Mon_10_ulcAverage = []
Mon_10_batteryState = []
Mon_10_overChargeCount = []
Mon_10_overdisChargeCount = []
Mon_10_abnormalTemperatureCount = []
Mon_10_lackFaceValueCount = []
Mon_10_insufficientChargeCount = []
Mon_10_voltaicCharged = []
Mon_10_voltaicDischarged = []
Mon_10_voltageCharged = []
Mon_10_voltageDischarged = []
Mon_10_temperatureCharged = []
Mon_10_temperatureDischarged = []
Mon_10_faceValue = []
Mon_10_dod = [] # 월 데이터 리스트
Mon_10_year = []
Mon_10_month = []
Mon_10_sensorID = []
Mon_10_chargedTimeTotal = []
Mon_10_dischargedTimeTotal = []
Mon_10_overChargeTimeTotal = []
Mon_10_overdisChargeTimeTotal =[]
Mon_10_abnormalTemperatureTimeTotal = []
Mon_10_lackFaceValueTimeTotal = []
Mon_10_insufficientChargeTimeTotal = []
Mon_10_insufficientCharge = []
Mon_10_meancycle = []
Mon_10_expectedLifeMonth = []
Mon_10_expectedLifeCycle = []
Mon_10_usedLifeMonth = []
Mon_10_expectedLifePercent = []
Mon_10_usedLifePercent = []

try :
    Mon_10_chargedTime.append(sum(Mon_10_chargedTime_list)/len(Mon_10_chargedTime_list))
    Mon_10_dischargedTime.append(sum(Mon_10_dischargedTime_list)/len(Mon_10_dischargedTime_list))
    Mon_10_elcAverage.append(sum(Mon_10_elcAverage_list)/len(Mon_10_elcAverage_list))
    Mon_10_ulcAverage.append(sum(Mon_10_ulcAverage_list)/len(Mon_10_ulcAverage_list))
    Mon_10_batteryState.append(Mon_10[-1]['batteryState'])
    Mon_10_overChargeCount.append(sum(Mon_10_overChargeCount_list)/len(Mon_10_overChargeCount_list))
    Mon_10_overdisChargeCount.append(sum(Mon_10_overdisChargeCount_list)/len(Mon_10_overdisChargeCount_list))
    Mon_10_abnormalTemperatureCount.append(sum(Mon_10_abnormalTemperatureCount_list)/len(Mon_10_abnormalTemperatureCount_list))
    Mon_10_lackFaceValueCount.append(sum(Mon_10_lackFaceValueCount_list)/len(Mon_10_lackFaceValueCount_list))
    Mon_10_insufficientChargeCount.append(sum(Mon_10_insufficientChargeCount_list)/len(Mon_10_insufficientChargeCount_list))
    Mon_10_voltaicCharged.append(sum(Mon_10_voltaicCharged_list)/len(Mon_10_voltaicCharged_list))
    Mon_10_voltaicDischarged.append(sum(Mon_10_voltaicDischarged_list)/len(Mon_10_voltaicDischarged_list))
    Mon_10_voltageCharged.append(sum(Mon_10_voltageCharged_list)/len(Mon_10_voltageCharged_list))
    Mon_10_voltageDischarged.append(sum(Mon_10_voltageDischarged_list)/len(Mon_10_voltageDischarged_list))
    Mon_10_temperatureCharged.append(sum(Mon_10_temperatureCharged_list)/len(Mon_10_temperatureCharged_list))
    Mon_10_temperatureDischarged.append(sum(Mon_10_temperatureDischarged_list)/len(Mon_10_temperatureDischarged_list))
    Mon_10_faceValue.append(sum(Mon_10_faceValue_list)/len(Mon_10_faceValue_list))
    Mon_10_dod.append(sum(Mon_10_dod_list)/len(Mon_10_dod_list)) 
    Mon_10_year.append(Mon_10[0]['date'][0:4])
    Mon_10_month.append(Mon_10[0]['date'][5:7])# 월 평균 데이터 계산
    Mon_10_sensorID.append(Mon_10[0]['sensorID'])
    Mon_10_chargedTimeTotal.append(sum(Mon_10_chargedTimeTotal_list))
    Mon_10_dischargedTimeTotal.append(sum(Mon_10_dischargedTimeTotal_list))
    Mon_10_overChargeTimeTotal.append(sum(Mon_10_overChargeTimeTotal_list)/60)
    Mon_10_overdisChargeTimeTotal.append(sum(Mon_10_overdisChargeTimeTotal_list)/60)
    Mon_10_abnormalTemperatureTimeTotal.append(sum(Mon_10_abnormalTemperatureTimeTotal_list)/60) 
    Mon_10_lackFaceValueTimeTotal.append(sum(Mon_10_lackFaceValueTimeTotal_list)/60)
    Mon_10_insufficientChargeTimeTotal.append(sum(Mon_10_insufficientChargeTimeTotal_list)/60)   
    Mon_10_insufficientCharge.append(sum(Mon_10_insufficientCharge_list)/len(Mon_10_insufficientCharge_list))
    Mon_10_meancycle.append(math.floor(sum(Mon_10_meancycle_list)/len(Mon_10_meancycle_list)))
    Mon_10_usedLifeMonth.append(round(sum(Mon_10_usedLifeMonth_list)/len(Mon_10_usedLifeMonth_list),1))
    
except ZeroDivisionError:
    pass

if len(Mon_10) > 0:
    if Mon_10_dod[0] < 40 :
        Dod_c = 1.5
    elif Mon_10_dod[0] >= 40 and Mon_10_dod[0] < 50:
        Dod_c = 1.42
    elif Mon_10_dod[0] >= 50 and Mon_10_dod[0] < 60:
        Dod_c = 1.25
    elif Mon_10_dod[0] >= 60 and Mon_10_dod[0] < 70:
        Dod_c = 1.08
    elif Mon_10_dod[0] >= 70 and Mon_10_dod[0] < 80:
        Dod_c = 0.92
    elif Mon_10_dod[0] >= 80 and Mon_10_dod[0] < 90:
        Dod_c = 0.75
    elif Mon_10_dod[0] >= 90 : # and DD <= 100 <-DD값이 낮아지면 다시 삽입
        Dod_c = 0.58
    else:
        pass

    if Mon_10_temperatureDischarged[0] < 30 :
        Mon_10_average_temperatureDischarged = 30
    else :
        Mon_10_average_temperatureDischarged = Mon_10_temperatureDischarged[0]

    Mon_10_voltaicDischarged_ln = round(ln((Mon_10_voltaicDischarged[0]/800)*5),1)

    if pd.isna(Mon_10_voltaicDischarged_ln) == True :
        Mon_10_voltaicDischarged_ln = 0
    else :
        pass

    Mon_10_expectedLifeMonth.append(round(((Dod_c*1200)*30*12/(Mon_10_average_temperatureDischarged*(1+(7/30)*Mon_10_voltaicDischarged_ln)*Mon_10_meancycle[0]*220))*1*0.85*1,1))
    Mon_10_expectedLifeCycle.append(round(((Dod_c*1200)*30/(Mon_10_average_temperatureDischarged*(1+(7/30)*Mon_10_voltaicDischarged_ln)))*1*0.85*1,1))

    if round((Mon_10_expectedLifeCycle[0] * 100) / 1200,1) > 100 :
        Mon_10_expectedLifePercent.append(100)
    else :
        Mon_10_expectedLifePercent.append(round((Mon_10_expectedLifeCycle[0] * 100) / 1200,1))

    Mon_10_usedLifePercent.append(round((Mon_10_ulcAverage[0] * 100) / 1200,1))


In [72]:
Mon_11_chargedTime_list = []
Mon_11_dischargedTime_list = []
Mon_11_elcAverage_list = []
Mon_11_ulcAverage_list = []
Mon_11_batteryState_list = []
Mon_11_overChargeCount_list = []
Mon_11_overdisChargeCount_list = []
Mon_11_abnormalTemperatureCount_list = []
Mon_11_lackFaceValueCount_list = []
Mon_11_insufficientChargeCount_list = []
Mon_11_voltaicCharged_list = []
Mon_11_voltaicDischarged_list = []
Mon_11_voltageCharged_list = []
Mon_11_voltageDischarged_list = []
Mon_11_temperatureCharged_list = []
Mon_11_temperatureDischarged_list = []
Mon_11_faceValue_list = []
Mon_11_dod_list = [] #계산식 저장 리스트
Mon_11_chargedTimeTotal_list = []
Mon_11_dischargedTimeTotal_list = []
Mon_11_overChargeTimeTotal_list = []
Mon_11_overdisChargeTimeTotal_list =[]
Mon_11_abnormalTemperatureTimeTotal_list = []
Mon_11_insufficientChargeTimeTotal_list = []
Mon_11_lackFaceValueTimeTotal_list = []
Mon_11_insufficientCharge_list = []
Mon_11_meancycle_list = []
Mon_11_usedLifeMonth_list = []

for i in range(len(Mon_11)): # 월 데이터 모으기
    Mon_11_chargedTime_list.append(Mon_11[i]['chargedTime'])
    Mon_11_dischargedTime_list.append(Mon_11[i]['dischargedTime'])
    Mon_11_elcAverage_list.append(Mon_11[i]['expectedLifeCycle'])
    Mon_11_ulcAverage_list.append(Mon_11[i]['usedLifeCycle'])
    Mon_11_batteryState_list.append(Mon_11[i]['batteryState'])
    Mon_11_overChargeCount_list.append(Mon_11[i]['overChargeCount'])
    Mon_11_overdisChargeCount_list.append(Mon_11[i]['overdisChargeCount'])
    Mon_11_abnormalTemperatureCount_list.append(Mon_11[i]['abnormalTemperatureCount'])
    Mon_11_lackFaceValueCount_list.append(Mon_11[i]['lackFaceValueCount'])
    Mon_11_insufficientChargeCount_list.append(Mon_11[i]['insufficientChargeCount'])
    Mon_11_voltaicCharged_list.append(Mon_11[i]['voltaicCharged'])
    Mon_11_voltaicDischarged_list.append(Mon_11[i]['voltaicDischarged'])
    Mon_11_voltageCharged_list.append(Mon_11[i]['voltageCharged'])
    Mon_11_voltageDischarged_list.append(Mon_11[i]['voltageDischarged'])
    Mon_11_temperatureCharged_list.append(Mon_11[i]['temperatureCharged'])
    Mon_11_temperatureDischarged_list.append(Mon_11[i]['temperatureDischarged'])
    Mon_11_faceValue_list.append(Mon_11[i]['faceValue'])
    Mon_11_dod_list.append(Mon_11[i]['dod']) 
    Mon_11_chargedTimeTotal_list.append(Mon_11[i]['chargedTime'])
    Mon_11_dischargedTimeTotal_list.append(Mon_11[i]['dischargedTime'])
    Mon_11_overChargeTimeTotal_list.append(Mon_11[i]['overChargeTime'])
    Mon_11_overdisChargeTimeTotal_list.append(Mon_11[i]['overdisChargeTime'])
    Mon_11_abnormalTemperatureTimeTotal_list.append(Mon_11[i]['abnormalTemperatureTime'])
    Mon_11_lackFaceValueTimeTotal_list.append(Mon_11[i]['lackFaceValueTime'])
    Mon_11_insufficientChargeTimeTotal_list.append(Mon_11[i]['insufficientChargeTime'])
    Mon_11_insufficientCharge_list.append(Mon_11[i]['insufficientCharge'])
    Mon_11_meancycle_list.append(Mon_11[i]['meancycle'])
    Mon_11_usedLifeMonth_list.append(Mon_11[i]['usedLifeMonth'])
    
Mon_11_chargedTime = []
Mon_11_dischargedTime = []
Mon_11_elcAverage = []
Mon_11_ulcAverage = []
Mon_11_batteryState = []
Mon_11_overChargeCount = []
Mon_11_overdisChargeCount = []
Mon_11_abnormalTemperatureCount = []
Mon_11_lackFaceValueCount = []
Mon_11_insufficientChargeCount = []
Mon_11_voltaicCharged = []
Mon_11_voltaicDischarged = []
Mon_11_voltageCharged = []
Mon_11_voltageDischarged = []
Mon_11_temperatureCharged = []
Mon_11_temperatureDischarged = []
Mon_11_faceValue = []
Mon_11_dod = [] # 월 데이터 리스트
Mon_11_year = []
Mon_11_month = []
Mon_11_sensorID = []
Mon_11_chargedTimeTotal = []
Mon_11_dischargedTimeTotal = []
Mon_11_overChargeTimeTotal = []
Mon_11_overdisChargeTimeTotal =[]
Mon_11_abnormalTemperatureTimeTotal = []
Mon_11_lackFaceValueTimeTotal = []
Mon_11_insufficientChargeTimeTotal = []
Mon_11_insufficientCharge = []
Mon_11_meancycle = []
Mon_11_expectedLifeMonth = []
Mon_11_expectedLifeCycle = []
Mon_11_usedLifeMonth = []
Mon_11_expectedLifePercent = []
Mon_11_usedLifePercent = []

try :
    Mon_11_chargedTime.append(sum(Mon_11_chargedTime_list)/len(Mon_11_chargedTime_list))
    Mon_11_dischargedTime.append(sum(Mon_11_dischargedTime_list)/len(Mon_11_dischargedTime_list))
    Mon_11_elcAverage.append(sum(Mon_11_elcAverage_list)/len(Mon_11_elcAverage_list))
    Mon_11_ulcAverage.append(sum(Mon_11_ulcAverage_list)/len(Mon_11_ulcAverage_list))
    Mon_11_batteryState.append(Mon_11[-1]['batteryState'])
    Mon_11_overChargeCount.append(sum(Mon_11_overChargeCount_list)/len(Mon_11_overChargeCount_list))
    Mon_11_overdisChargeCount.append(sum(Mon_11_overdisChargeCount_list)/len(Mon_11_overdisChargeCount_list))
    Mon_11_abnormalTemperatureCount.append(sum(Mon_11_abnormalTemperatureCount_list)/len(Mon_11_abnormalTemperatureCount_list))
    Mon_11_lackFaceValueCount.append(sum(Mon_11_lackFaceValueCount_list)/len(Mon_11_lackFaceValueCount_list))
    Mon_11_insufficientChargeCount.append(sum(Mon_11_insufficientChargeCount_list)/len(Mon_11_insufficientChargeCount_list))
    Mon_11_voltaicCharged.append(sum(Mon_11_voltaicCharged_list)/len(Mon_11_voltaicCharged_list))
    Mon_11_voltaicDischarged.append(sum(Mon_11_voltaicDischarged_list)/len(Mon_11_voltaicDischarged_list))
    Mon_11_voltageCharged.append(sum(Mon_11_voltageCharged_list)/len(Mon_11_voltageCharged_list))
    Mon_11_voltageDischarged.append(sum(Mon_11_voltageDischarged_list)/len(Mon_11_voltageDischarged_list))
    Mon_11_temperatureCharged.append(sum(Mon_11_temperatureCharged_list)/len(Mon_11_temperatureCharged_list))
    Mon_11_temperatureDischarged.append(sum(Mon_11_temperatureDischarged_list)/len(Mon_11_temperatureDischarged_list))
    Mon_11_faceValue.append(sum(Mon_11_faceValue_list)/len(Mon_11_faceValue_list))
    Mon_11_dod.append(sum(Mon_11_dod_list)/len(Mon_11_dod_list)) 
    Mon_11_year.append(Mon_11[0]['date'][0:4])
    Mon_11_month.append(Mon_11[0]['date'][5:7])# 월 평균 데이터 계산
    Mon_11_sensorID.append(Mon_11[0]['sensorID'])
    Mon_11_chargedTimeTotal.append(sum(Mon_11_chargedTimeTotal_list))
    Mon_11_dischargedTimeTotal.append(sum(Mon_11_dischargedTimeTotal_list))
    Mon_11_overChargeTimeTotal.append(sum(Mon_11_overChargeTimeTotal_list)/60)
    Mon_11_overdisChargeTimeTotal.append(sum(Mon_11_overdisChargeTimeTotal_list)/60)
    Mon_11_abnormalTemperatureTimeTotal.append(sum(Mon_11_abnormalTemperatureTimeTotal_list)/60) 
    Mon_11_lackFaceValueTimeTotal.append(sum(Mon_11_lackFaceValueTimeTotal_list)/60)
    Mon_11_insufficientChargeTimeTotal.append(sum(Mon_11_insufficientChargeTimeTotal_list)/60)   
    Mon_11_insufficientCharge.append(sum(Mon_11_insufficientCharge_list)/len(Mon_11_insufficientCharge_list))
    Mon_11_meancycle.append(math.floor(sum(Mon_11_meancycle_list)/len(Mon_11_meancycle_list)))
    Mon_11_usedLifeMonth.append(round(sum(Mon_11_usedLifeMonth_list)/len(Mon_11_usedLifeMonth_list),1))
    
except ZeroDivisionError:
    pass

if len(Mon_11) > 0:
    if Mon_11_dod[0] < 40 :
        Dod_c = 1.5
    elif Mon_11_dod[0] >= 40 and Mon_11_dod[0] < 50:
        Dod_c = 1.42
    elif Mon_11_dod[0] >= 50 and Mon_11_dod[0] < 60:
        Dod_c = 1.25
    elif Mon_11_dod[0] >= 60 and Mon_11_dod[0] < 70:
        Dod_c = 1.08
    elif Mon_11_dod[0] >= 70 and Mon_11_dod[0] < 80:
        Dod_c = 0.92
    elif Mon_11_dod[0] >= 80 and Mon_11_dod[0] < 90:
        Dod_c = 0.75
    elif Mon_11_dod[0] >= 90 : # and DD <= 100 <-DD값이 낮아지면 다시 삽입
        Dod_c = 0.58
    else:
        pass

    if Mon_11_temperatureDischarged[0] < 30 :
        Mon_11_average_temperatureDischarged = 30
    else :
        Mon_11_average_temperatureDischarged = Mon_11_temperatureDischarged[0]

    Mon_11_voltaicDischarged_ln = round(ln((Mon_11_voltaicDischarged[0]/800)*5),1)

    if pd.isna(Mon_11_voltaicDischarged_ln) == True :
        Mon_11_voltaicDischarged_ln = 0
    else :
        pass

    Mon_11_expectedLifeMonth.append(round(((Dod_c*1200)*30*12/(Mon_11_average_temperatureDischarged*(1+(7/30)*Mon_11_voltaicDischarged_ln)*Mon_11_meancycle[0]*220))*1*0.85*1,1))
    Mon_11_expectedLifeCycle.append(round(((Dod_c*1200)*30/(Mon_11_average_temperatureDischarged*(1+(7/30)*Mon_11_voltaicDischarged_ln)))*1*0.85*1,1))

    if round((Mon_11_expectedLifeCycle[0] * 100) / 1200,1) > 100 :
        Mon_11_expectedLifePercent.append(100)
    else :
        Mon_11_expectedLifePercent.append(round((Mon_11_expectedLifeCycle[0] * 100) / 1200,1))

    Mon_11_usedLifePercent.append(round((Mon_11_ulcAverage[0] * 100) / 1200,1))


C:\Users\sm02\Anaconda3\lib\site-packages\ipykernel_launcher.py:155: RuntimeWarning: invalid value encountered in log


In [73]:
Mon_12_chargedTime_list = []
Mon_12_dischargedTime_list = []
Mon_12_elcAverage_list = []
Mon_12_ulcAverage_list = []
Mon_12_batteryState_list = []
Mon_12_overChargeCount_list = []
Mon_12_overdisChargeCount_list = []
Mon_12_abnormalTemperatureCount_list = []
Mon_12_lackFaceValueCount_list = []
Mon_12_insufficientChargeCount_list = []
Mon_12_voltaicCharged_list = []
Mon_12_voltaicDischarged_list = []
Mon_12_voltageCharged_list = []
Mon_12_voltageDischarged_list = []
Mon_12_temperatureCharged_list = []
Mon_12_temperatureDischarged_list = []
Mon_12_faceValue_list = []
Mon_12_dod_list = [] #계산식 저장 리스트
Mon_12_chargedTimeTotal_list = []
Mon_12_dischargedTimeTotal_list = []
Mon_12_overChargeTimeTotal_list = []
Mon_12_overdisChargeTimeTotal_list =[]
Mon_12_abnormalTemperatureTimeTotal_list = []
Mon_12_insufficientChargeTimeTotal_list = []
Mon_12_lackFaceValueTimeTotal_list = []
Mon_12_insufficientCharge_list = []
Mon_12_meancycle_list = []
Mon_12_usedLifeMonth_list = []

for i in range(len(Mon_12)): # 월 데이터 모으기
    Mon_12_chargedTime_list.append(Mon_12[i]['chargedTime'])
    Mon_12_dischargedTime_list.append(Mon_12[i]['dischargedTime'])
    Mon_12_elcAverage_list.append(Mon_12[i]['expectedLifeCycle'])
    Mon_12_ulcAverage_list.append(Mon_12[i]['usedLifeCycle'])
    Mon_12_batteryState_list.append(Mon_12[i]['batteryState'])
    Mon_12_overChargeCount_list.append(Mon_12[i]['overChargeCount'])
    Mon_12_overdisChargeCount_list.append(Mon_12[i]['overdisChargeCount'])
    Mon_12_abnormalTemperatureCount_list.append(Mon_12[i]['abnormalTemperatureCount'])
    Mon_12_lackFaceValueCount_list.append(Mon_12[i]['lackFaceValueCount'])
    Mon_12_insufficientChargeCount_list.append(Mon_12[i]['insufficientChargeCount'])
    Mon_12_voltaicCharged_list.append(Mon_12[i]['voltaicCharged'])
    Mon_12_voltaicDischarged_list.append(Mon_12[i]['voltaicDischarged'])
    Mon_12_voltageCharged_list.append(Mon_12[i]['voltageCharged'])
    Mon_12_voltageDischarged_list.append(Mon_12[i]['voltageDischarged'])
    Mon_12_temperatureCharged_list.append(Mon_12[i]['temperatureCharged'])
    Mon_12_temperatureDischarged_list.append(Mon_12[i]['temperatureDischarged'])
    Mon_12_faceValue_list.append(Mon_12[i]['faceValue'])
    Mon_12_dod_list.append(Mon_12[i]['dod']) 
    Mon_12_chargedTimeTotal_list.append(Mon_12[i]['chargedTime'])
    Mon_12_dischargedTimeTotal_list.append(Mon_12[i]['dischargedTime'])
    Mon_12_overChargeTimeTotal_list.append(Mon_12[i]['overChargeTime'])
    Mon_12_overdisChargeTimeTotal_list.append(Mon_12[i]['overdisChargeTime'])
    Mon_12_abnormalTemperatureTimeTotal_list.append(Mon_12[i]['abnormalTemperatureTime'])
    Mon_12_lackFaceValueTimeTotal_list.append(Mon_12[i]['lackFaceValueTime'])
    Mon_12_insufficientChargeTimeTotal_list.append(Mon_12[i]['insufficientChargeTime'])
    Mon_12_insufficientCharge_list.append(Mon_12[i]['insufficientCharge'])
    Mon_12_meancycle_list.append(Mon_12[i]['meancycle'])
    Mon_12_usedLifeMonth_list.append(Mon_12[i]['usedLifeMonth'])
    
Mon_12_chargedTime = []
Mon_12_dischargedTime = []
Mon_12_elcAverage = []
Mon_12_ulcAverage = []
Mon_12_batteryState = []
Mon_12_overChargeCount = []
Mon_12_overdisChargeCount = []
Mon_12_abnormalTemperatureCount = []
Mon_12_lackFaceValueCount = []
Mon_12_insufficientChargeCount = []
Mon_12_voltaicCharged = []
Mon_12_voltaicDischarged = []
Mon_12_voltageCharged = []
Mon_12_voltageDischarged = []
Mon_12_temperatureCharged = []
Mon_12_temperatureDischarged = []
Mon_12_faceValue = []
Mon_12_dod = [] # 월 데이터 리스트
Mon_12_year = []
Mon_12_month = []
Mon_12_sensorID = []
Mon_12_chargedTimeTotal = []
Mon_12_dischargedTimeTotal = []
Mon_12_overChargeTimeTotal = []
Mon_12_overdisChargeTimeTotal =[]
Mon_12_abnormalTemperatureTimeTotal = []
Mon_12_lackFaceValueTimeTotal = []
Mon_12_insufficientChargeTimeTotal = []
Mon_12_insufficientCharge = []
Mon_12_meancycle = []
Mon_12_expectedLifeMonth = []
Mon_12_expectedLifeCycle = []
Mon_12_usedLifeMonth = []
Mon_12_expectedLifePercent = []
Mon_12_usedLifePercent = []

try :
    Mon_12_chargedTime.append(sum(Mon_12_chargedTime_list)/len(Mon_12_chargedTime_list))
    Mon_12_dischargedTime.append(sum(Mon_12_dischargedTime_list)/len(Mon_12_dischargedTime_list))
    Mon_12_elcAverage.append(sum(Mon_12_elcAverage_list)/len(Mon_12_elcAverage_list))
    Mon_12_ulcAverage.append(sum(Mon_12_ulcAverage_list)/len(Mon_12_ulcAverage_list))
    Mon_12_batteryState.append(Mon_12[-1]['batteryState'])
    Mon_12_overChargeCount.append(sum(Mon_12_overChargeCount_list)/len(Mon_12_overChargeCount_list))
    Mon_12_overdisChargeCount.append(sum(Mon_12_overdisChargeCount_list)/len(Mon_12_overdisChargeCount_list))
    Mon_12_abnormalTemperatureCount.append(sum(Mon_12_abnormalTemperatureCount_list)/len(Mon_12_abnormalTemperatureCount_list))
    Mon_12_lackFaceValueCount.append(sum(Mon_12_lackFaceValueCount_list)/len(Mon_12_lackFaceValueCount_list))
    Mon_12_insufficientChargeCount.append(sum(Mon_12_insufficientChargeCount_list)/len(Mon_12_insufficientChargeCount_list))
    Mon_12_voltaicCharged.append(sum(Mon_12_voltaicCharged_list)/len(Mon_12_voltaicCharged_list))
    Mon_12_voltaicDischarged.append(sum(Mon_12_voltaicDischarged_list)/len(Mon_12_voltaicDischarged_list))
    Mon_12_voltageCharged.append(sum(Mon_12_voltageCharged_list)/len(Mon_12_voltageCharged_list))
    Mon_12_voltageDischarged.append(sum(Mon_12_voltageDischarged_list)/len(Mon_12_voltageDischarged_list))
    Mon_12_temperatureCharged.append(sum(Mon_12_temperatureCharged_list)/len(Mon_12_temperatureCharged_list))
    Mon_12_temperatureDischarged.append(sum(Mon_12_temperatureDischarged_list)/len(Mon_12_temperatureDischarged_list))
    Mon_12_faceValue.append(sum(Mon_12_faceValue_list)/len(Mon_12_faceValue_list))
    Mon_12_dod.append(sum(Mon_12_dod_list)/len(Mon_12_dod_list)) 
    Mon_12_year.append(Mon_12[0]['date'][0:4])
    Mon_12_month.append(Mon_12[0]['date'][5:7])# 월 평균 데이터 계산
    Mon_12_sensorID.append(Mon_12[0]['sensorID'])
    Mon_12_chargedTimeTotal.append(sum(Mon_12_chargedTimeTotal_list))
    Mon_12_dischargedTimeTotal.append(sum(Mon_12_dischargedTimeTotal_list))
    Mon_12_overChargeTimeTotal.append(sum(Mon_12_overChargeTimeTotal_list)/60)
    Mon_12_overdisChargeTimeTotal.append(sum(Mon_12_overdisChargeTimeTotal_list)/60)
    Mon_12_abnormalTemperatureTimeTotal.append(sum(Mon_12_abnormalTemperatureTimeTotal_list)/60) 
    Mon_12_lackFaceValueTimeTotal.append(sum(Mon_12_lackFaceValueTimeTotal_list)/60)
    Mon_12_insufficientChargeTimeTotal.append(sum(Mon_12_insufficientChargeTimeTotal_list)/60)   
    Mon_12_insufficientCharge.append(sum(Mon_12_insufficientCharge_list)/len(Mon_12_insufficientCharge_list))
    Mon_12_meancycle.append(math.floor(sum(Mon_12_meancycle_list)/len(Mon_12_meancycle_list)))
    Mon_12_usedLifeMonth.append(round(sum(Mon_12_usedLifeMonth_list)/len(Mon_12_usedLifeMonth_list),1))
    
except ZeroDivisionError:
    pass

if len(Mon_12) > 0:
    if Mon_12_dod[0] < 40 :
        Dod_c = 1.5
    elif Mon_12_dod[0] >= 40 and Mon_12_dod[0] < 50:
        Dod_c = 1.42
    elif Mon_12_dod[0] >= 50 and Mon_12_dod[0] < 60:
        Dod_c = 1.25
    elif Mon_12_dod[0] >= 60 and Mon_12_dod[0] < 70:
        Dod_c = 1.08
    elif Mon_12_dod[0] >= 70 and Mon_12_dod[0] < 80:
        Dod_c = 0.92
    elif Mon_12_dod[0] >= 80 and Mon_12_dod[0] < 90:
        Dod_c = 0.75
    elif Mon_12_dod[0] >= 90 : # and DD <= 100 <-DD값이 낮아지면 다시 삽입
        Dod_c = 0.58
    else:
        pass

    if Mon_12_temperatureDischarged[0] < 30 :
        Mon_12_average_temperatureDischarged = 30
    else :
        Mon_12_average_temperatureDischarged = Mon_12_temperatureDischarged[0]

    Mon_12_voltaicDischarged_ln = round(ln((Mon_12_voltaicDischarged[0]/800)*5),1)

    if pd.isna(Mon_12_voltaicDischarged_ln) == True :
        Mon_12_voltaicDischarged_ln = 0
    else :
        pass

    Mon_12_expectedLifeMonth.append(round(((Dod_c*1200)*30*12/(Mon_12_average_temperatureDischarged*(1+(7/30)*Mon_12_voltaicDischarged_ln)*Mon_12_meancycle[0]*220))*1*0.85*1,1))
    Mon_12_expectedLifeCycle.append(round(((Dod_c*1200)*30/(Mon_12_average_temperatureDischarged*(1+(7/30)*Mon_12_voltaicDischarged_ln)))*1*0.85*1,1))

    if round((Mon_12_expectedLifeCycle[0] * 100) / 1200,1) > 100 :
        Mon_12_expectedLifePercent.append(100)
    else :
        Mon_12_expectedLifePercent.append(round((Mon_12_expectedLifeCycle[0] * 100) / 1200,1))

    Mon_12_usedLifePercent.append(round((Mon_12_ulcAverage[0] * 100) / 1200,1))


C:\Users\sm02\Anaconda3\lib\site-packages\ipykernel_launcher.py:155: RuntimeWarning: invalid value encountered in log


In [74]:
col_name = ['sensorID'] # 년 월 배터리 상태 추가

sensorID_list = Mon_1_sensorID+Mon_2_sensorID+Mon_3_sensorID+Mon_4_sensorID+Mon_5_sensorID+Mon_6_sensorID+Mon_7_sensorID+Mon_8_sensorID+Mon_9_sensorID+Mon_10_sensorID+Mon_11_sensorID+Mon_12_sensorID

Mon_df = pd.DataFrame(sensorID_list, columns = col_name)

In [75]:
Mon_df['year'] = Mon_1_year+Mon_2_year+Mon_3_year+Mon_4_year+Mon_5_year+Mon_6_year+Mon_7_year+Mon_8_year+Mon_9_year+Mon_10_year+Mon_11_year+Mon_12_year
Mon_df['month'] = Mon_1_month+Mon_2_month+Mon_3_month+Mon_4_month+Mon_5_month+Mon_6_month+Mon_7_month+Mon_8_month+Mon_9_month+Mon_10_month+Mon_11_month+Mon_12_month
Mon_df['chargedTime'] = np.round(Mon_1_chargedTime+Mon_2_chargedTime+Mon_3_chargedTime+Mon_4_chargedTime+Mon_5_chargedTime+Mon_6_chargedTime+Mon_7_chargedTime+Mon_8_chargedTime+Mon_9_chargedTime+Mon_10_chargedTime+Mon_11_chargedTime+Mon_12_chargedTime,5)
Mon_df['dischargedTime'] = np.round(Mon_1_dischargedTime+Mon_2_dischargedTime+Mon_3_dischargedTime+Mon_4_dischargedTime+Mon_5_dischargedTime+Mon_6_dischargedTime+Mon_7_dischargedTime+Mon_8_dischargedTime+Mon_9_dischargedTime+Mon_10_dischargedTime+Mon_11_dischargedTime+Mon_12_dischargedTime,5)
Mon_df['batteryState'] = Mon_1_batteryState+Mon_2_batteryState+Mon_3_batteryState+Mon_4_batteryState+Mon_5_batteryState+Mon_6_batteryState+Mon_7_batteryState+Mon_8_batteryState+Mon_9_batteryState+Mon_10_batteryState+Mon_11_batteryState+Mon_12_batteryState 
# Mon_df['batteryState'] = "GOOD"
Mon_df['overChargeCount'] = np.round(Mon_1_overChargeCount+Mon_2_overChargeCount+Mon_3_overChargeCount+Mon_4_overChargeCount+Mon_5_overChargeCount+Mon_6_overChargeCount+Mon_7_overChargeCount+Mon_8_overChargeCount+Mon_9_overChargeCount+Mon_10_overChargeCount+Mon_11_overChargeCount+Mon_12_overChargeCount,1)
Mon_df['overdisChargeCount'] = np.round(Mon_1_overdisChargeCount+Mon_2_overdisChargeCount+Mon_3_overdisChargeCount+Mon_4_overdisChargeCount+Mon_5_overdisChargeCount+Mon_6_overdisChargeCount+Mon_7_overdisChargeCount+Mon_8_overdisChargeCount+Mon_9_overdisChargeCount+Mon_10_overdisChargeCount+Mon_11_overdisChargeCount+Mon_12_overdisChargeCount,1)
Mon_df['abnormalTemperatureCount'] = np.round(Mon_1_abnormalTemperatureCount+Mon_2_abnormalTemperatureCount+Mon_3_abnormalTemperatureCount+Mon_4_abnormalTemperatureCount+Mon_5_abnormalTemperatureCount+Mon_6_abnormalTemperatureCount+Mon_7_abnormalTemperatureCount+Mon_8_abnormalTemperatureCount+Mon_9_abnormalTemperatureCount+Mon_10_abnormalTemperatureCount+Mon_11_abnormalTemperatureCount+Mon_12_abnormalTemperatureCount,1)
Mon_df['lackFaceValueCount'] = np.round(Mon_1_lackFaceValueCount+Mon_2_lackFaceValueCount+Mon_3_lackFaceValueCount+Mon_4_lackFaceValueCount+Mon_5_lackFaceValueCount+Mon_6_lackFaceValueCount+Mon_7_lackFaceValueCount+Mon_8_lackFaceValueCount+Mon_9_lackFaceValueCount+Mon_10_lackFaceValueCount+Mon_11_lackFaceValueCount+Mon_12_lackFaceValueCount,1)
Mon_df['insufficientChargeCount'] = np.round(Mon_1_insufficientChargeCount+Mon_2_insufficientChargeCount+Mon_3_insufficientChargeCount+Mon_4_insufficientChargeCount+Mon_5_insufficientChargeCount+Mon_6_insufficientChargeCount+Mon_7_insufficientChargeCount+Mon_8_insufficientChargeCount+Mon_9_insufficientChargeCount+Mon_10_insufficientChargeCount+Mon_11_insufficientChargeCount+Mon_12_insufficientChargeCount,1)
Mon_df['voltaicCharged'] = np.round(Mon_1_voltaicCharged+Mon_2_voltaicCharged+Mon_3_voltaicCharged+Mon_4_voltaicCharged+Mon_5_voltaicCharged+Mon_6_voltaicCharged+Mon_7_voltaicCharged+Mon_8_voltaicCharged+Mon_9_voltaicCharged+Mon_10_voltaicCharged+Mon_11_voltaicCharged+Mon_12_voltaicCharged,1)
Mon_df['voltaicDischarged'] = np.round(Mon_1_voltaicDischarged+Mon_2_voltaicDischarged+Mon_3_voltaicDischarged+Mon_4_voltaicDischarged+Mon_5_voltaicDischarged+Mon_6_voltaicDischarged+Mon_7_voltaicDischarged+Mon_8_voltaicDischarged+Mon_9_voltaicDischarged+Mon_10_voltaicDischarged+Mon_11_voltaicDischarged+Mon_12_voltaicDischarged,1)
Mon_df['voltageCharged'] = np.round(Mon_1_voltageCharged+Mon_2_voltageCharged+Mon_3_voltageCharged+Mon_4_voltageCharged+Mon_5_voltageCharged+Mon_6_voltageCharged+Mon_7_voltageCharged+Mon_8_voltageCharged+Mon_9_voltageCharged+Mon_10_voltageCharged+Mon_11_voltageCharged+Mon_12_voltageCharged,1)
Mon_df['voltageDischarged'] = np.round(Mon_1_voltageDischarged+Mon_2_voltageDischarged+Mon_3_voltageDischarged+Mon_4_voltageDischarged+Mon_5_voltageDischarged+Mon_6_voltageDischarged+Mon_7_voltageDischarged+Mon_8_voltageDischarged+Mon_9_voltageDischarged+Mon_10_voltageDischarged+Mon_11_voltageDischarged+Mon_12_voltageDischarged,1)
Mon_df['temperatureCharged'] = np.round(Mon_1_temperatureCharged+Mon_2_temperatureCharged+Mon_3_temperatureCharged+Mon_4_temperatureCharged+Mon_5_temperatureCharged+Mon_6_temperatureCharged+Mon_7_temperatureCharged+Mon_8_temperatureCharged+Mon_9_temperatureCharged+Mon_10_temperatureCharged+Mon_11_temperatureCharged+Mon_12_temperatureCharged,1)
Mon_df['temperatureDischarged'] = np.round(Mon_1_temperatureDischarged+Mon_2_temperatureDischarged+Mon_3_temperatureDischarged+Mon_4_temperatureDischarged+Mon_5_temperatureDischarged+Mon_6_temperatureDischarged+Mon_7_temperatureDischarged+Mon_8_temperatureDischarged+Mon_9_temperatureDischarged+Mon_10_temperatureDischarged+Mon_11_temperatureDischarged+Mon_12_temperatureDischarged,1)
Mon_df['faceValue'] = np.round(Mon_1_faceValue+Mon_2_faceValue+Mon_3_faceValue+Mon_4_faceValue+Mon_5_faceValue+Mon_6_faceValue+Mon_7_faceValue+Mon_8_faceValue+Mon_9_faceValue+Mon_10_faceValue+Mon_11_faceValue+Mon_12_faceValue,1)
Mon_df['dod'] = np.round(Mon_1_dod+Mon_2_dod+Mon_3_dod+Mon_4_dod+Mon_5_dod+Mon_6_dod+Mon_7_dod+Mon_8_dod+Mon_9_dod+Mon_10_dod+Mon_11_dod+Mon_12_dod,1)
Mon_df['chargedTimeTotal'] = np.round(Mon_1_chargedTimeTotal+Mon_2_chargedTimeTotal+Mon_3_chargedTimeTotal+Mon_4_chargedTimeTotal+Mon_5_chargedTimeTotal+Mon_6_chargedTimeTotal+Mon_7_chargedTimeTotal+Mon_8_chargedTimeTotal+Mon_9_chargedTimeTotal+Mon_10_chargedTimeTotal+Mon_11_chargedTimeTotal+Mon_12_chargedTimeTotal,5)
Mon_df['dischargedTimeTotal'] = np.round(Mon_1_dischargedTimeTotal+Mon_2_dischargedTimeTotal+Mon_3_dischargedTimeTotal+Mon_4_dischargedTimeTotal+Mon_5_dischargedTimeTotal+Mon_6_dischargedTimeTotal+Mon_7_dischargedTimeTotal+Mon_8_dischargedTimeTotal+Mon_9_dischargedTimeTotal+Mon_10_dischargedTimeTotal+Mon_11_dischargedTimeTotal+Mon_12_dischargedTimeTotal,5)
Mon_df['overChargeTimeTotal'] = np.round(Mon_1_overChargeTimeTotal+Mon_2_overChargeTimeTotal+Mon_3_overChargeTimeTotal+Mon_4_overChargeTimeTotal+Mon_5_overChargeTimeTotal+Mon_6_overChargeTimeTotal+Mon_7_overChargeTimeTotal+Mon_8_overChargeTimeTotal+Mon_9_overChargeTimeTotal+Mon_10_overChargeTimeTotal+Mon_11_overChargeTimeTotal+Mon_12_overChargeTimeTotal,5)
Mon_df['overdisChargeTimeTotal'] = np.round(Mon_1_overdisChargeTimeTotal+Mon_2_overdisChargeTimeTotal+Mon_3_overdisChargeTimeTotal+Mon_4_overdisChargeTimeTotal+Mon_5_overdisChargeTimeTotal+Mon_6_overdisChargeTimeTotal+Mon_7_overdisChargeTimeTotal+Mon_8_overdisChargeTimeTotal+Mon_9_overdisChargeTimeTotal+Mon_10_overdisChargeTimeTotal+Mon_11_overdisChargeTimeTotal+Mon_12_overdisChargeTimeTotal,5)
Mon_df['abnormalTemperatureTimeTotal'] = np.round(Mon_1_abnormalTemperatureTimeTotal+Mon_2_abnormalTemperatureTimeTotal+Mon_3_abnormalTemperatureTimeTotal+Mon_4_abnormalTemperatureTimeTotal+Mon_5_abnormalTemperatureTimeTotal+Mon_6_abnormalTemperatureTimeTotal+Mon_7_abnormalTemperatureTimeTotal+Mon_8_abnormalTemperatureTimeTotal+Mon_9_abnormalTemperatureTimeTotal+Mon_10_abnormalTemperatureTimeTotal+Mon_11_abnormalTemperatureTimeTotal+Mon_12_abnormalTemperatureTimeTotal,5)
Mon_df['lackFaceValueTimeTotal'] = np.round(Mon_1_lackFaceValueTimeTotal+Mon_2_lackFaceValueTimeTotal+Mon_3_lackFaceValueTimeTotal+Mon_4_lackFaceValueTimeTotal+Mon_5_lackFaceValueTimeTotal+Mon_6_lackFaceValueTimeTotal+Mon_7_lackFaceValueTimeTotal+Mon_8_lackFaceValueTimeTotal+Mon_9_lackFaceValueTimeTotal+Mon_10_lackFaceValueTimeTotal+Mon_11_lackFaceValueTimeTotal+Mon_12_lackFaceValueTimeTotal,5)
Mon_df['insufficientCharge'] = np.round(Mon_1_insufficientCharge+Mon_2_insufficientCharge+Mon_3_insufficientCharge+Mon_4_insufficientCharge+Mon_5_insufficientCharge+Mon_6_insufficientCharge+Mon_7_insufficientCharge+Mon_8_insufficientCharge+Mon_9_insufficientCharge+Mon_10_insufficientCharge+Mon_11_insufficientCharge+Mon_12_insufficientCharge,1)
Mon_df['insufficientChargeTimeTotal'] = np.round(Mon_1_insufficientChargeTimeTotal+Mon_2_insufficientChargeTimeTotal+Mon_3_insufficientChargeTimeTotal+Mon_4_insufficientChargeTimeTotal+Mon_5_insufficientChargeTimeTotal+Mon_6_insufficientChargeTimeTotal+Mon_7_insufficientChargeTimeTotal+Mon_8_insufficientChargeTimeTotal+Mon_9_insufficientChargeTimeTotal+Mon_10_insufficientChargeTimeTotal+Mon_11_insufficientChargeTimeTotal+Mon_12_insufficientChargeTimeTotal,5)

Mon_df['expectedLifeCycle'] = np.round(Mon_1_expectedLifeCycle+Mon_2_expectedLifeCycle+Mon_3_expectedLifeCycle+Mon_4_expectedLifeCycle+Mon_5_expectedLifeCycle+Mon_6_expectedLifeCycle+Mon_7_expectedLifeCycle+Mon_8_expectedLifeCycle+Mon_9_expectedLifeCycle+Mon_10_expectedLifeCycle+Mon_11_expectedLifeCycle+Mon_12_expectedLifeCycle,0)
Mon_df['usedLifeCycle'] = np.round(Mon_1_ulcAverage+Mon_2_ulcAverage+Mon_3_ulcAverage+Mon_4_ulcAverage+Mon_5_ulcAverage+Mon_6_ulcAverage+Mon_7_ulcAverage+Mon_8_ulcAverage+Mon_9_ulcAverage+Mon_10_ulcAverage+Mon_11_ulcAverage+Mon_12_ulcAverage,0)
Mon_df['expectedLifeMonth'] = np.round(Mon_1_expectedLifeMonth+Mon_2_expectedLifeMonth+Mon_3_expectedLifeMonth+Mon_4_expectedLifeMonth+Mon_5_expectedLifeMonth+Mon_6_expectedLifeMonth+Mon_7_expectedLifeMonth+Mon_8_expectedLifeMonth+Mon_9_expectedLifeMonth+Mon_10_expectedLifeMonth+Mon_11_expectedLifeMonth+Mon_12_expectedLifeMonth,0)
Mon_df['usedLifeMonth'] = np.round(Mon_1_usedLifeMonth+Mon_2_usedLifeMonth+Mon_3_usedLifeMonth+Mon_4_usedLifeMonth+Mon_5_usedLifeMonth+Mon_6_usedLifeMonth+Mon_7_usedLifeMonth+Mon_8_usedLifeMonth+Mon_9_usedLifeMonth+Mon_10_usedLifeMonth+Mon_11_usedLifeMonth+Mon_12_usedLifeMonth,0)
Mon_df['expectedLifePercent'] = np.round(Mon_1_expectedLifePercent+Mon_2_expectedLifePercent+Mon_3_expectedLifePercent+Mon_4_expectedLifePercent+Mon_5_expectedLifePercent+Mon_6_expectedLifePercent+Mon_7_expectedLifePercent+Mon_8_expectedLifePercent+Mon_9_expectedLifePercent+Mon_10_expectedLifePercent+Mon_11_expectedLifePercent+Mon_12_expectedLifePercent,0)
Mon_df['usedLifePercent'] = np.round(Mon_1_usedLifePercent+Mon_2_usedLifePercent+Mon_3_usedLifePercent+Mon_4_usedLifePercent+Mon_5_usedLifePercent+Mon_6_usedLifePercent+Mon_7_usedLifePercent+Mon_8_usedLifePercent+Mon_9_usedLifePercent+Mon_10_usedLifePercent+Mon_11_usedLifePercent+Mon_12_usedLifePercent,0)

Mon_df['year'] = pd.to_numeric(Mon_df['year'])
Mon_df['month'] = pd.to_numeric(Mon_df['month']) #년,월 숫자 형식으로 변환 

In [76]:
Mon_df.to_csv(path_list[0] + 'Month_result.csv',index=False, encoding="utf-8-sig",mode = 'w') # 월 평균 데이터 저장

month to year ▼

In [54]:
Mdf4 = pd.read_csv(path_list[0] + 'Month_result.csv',encoding = 'utf-8-sig')

In [55]:

Year_21 = []
Year_22 = [] #년도 별 리스트 생성 22년 이후에 추가 생성 


In [56]:
for i in range(len(Mdf4)):
    if (Mdf4['year'][i] == 2021):
        Year_21.append(Mdf4.loc[i])
    elif (Mdf4['year'][i] == 2022):
        Year_22.append(Mdf4.loc[i])
    else :
        pass

In [58]:
Year_21_chargedTime_list = []
Year_21_dischargedTime_list = []
Year_21_elcAverage_list = []
Year_21_ulcAverage_list = []
Year_21_batteryState_list = []
Year_21_overChargeCount_list = []
Year_21_overdisChargeCount_list = []
Year_21_abnormalTemperatureCount_list = []
Year_21_lackFaceValueCount_list = []
Year_21_insufficientChargeCount_list = []
Year_21_voltaicCharged_list = []
Year_21_voltaicDischarged_list = []
Year_21_voltageCharged_list = []
Year_21_voltageDischarged_list = []
Year_21_temperatureCharged_list = []
Year_21_temperatureDischarged_list = []
Year_21_faceValue_list = []
Year_21_dod_list = [] #계산식 저장 리스트
Year_21_chargedTimeTotal_list = []
Year_21_dischargedTimeTotal_list = []
Year_21_overChargeTimeTotal_list = []
Year_21_overdisChargeTimeTotal_list =[]
Year_21_abnormalTemperatureTimeTotal_list = []
Year_21_insufficientChargeTimeTotal_list = []
Year_21_lackFaceValueTimeTotal_list = []
Year_21_insufficientCharge_list = []
Year_21_meancycle_list = []
Year_21_usedLifeMonth_list = []

for i in range(len(Year_21)): # 월 데이터 모으기
    Year_21_chargedTime_list.append(Year_21[i]['chargedTime'])
    Year_21_dischargedTime_list.append(Year_21[i]['dischargedTime'])
    Year_21_elcAverage_list.append(Year_21[i]['expectedLifeCycle'])
    Year_21_ulcAverage_list.append(Year_21[i]['usedLifeCycle'])
    Year_21_batteryState_list.append(Year_21[i]['batteryState'])
    Year_21_overChargeCount_list.append(Year_21[i]['overChargeCount'])
    Year_21_overdisChargeCount_list.append(Year_21[i]['overdisChargeCount'])
    Year_21_abnormalTemperatureCount_list.append(Year_21[i]['abnormalTemperatureCount'])
    Year_21_lackFaceValueCount_list.append(Year_21[i]['lackFaceValueCount'])
    Year_21_insufficientChargeCount_list.append(Year_21[i]['insufficientChargeCount'])
    Year_21_voltaicCharged_list.append(Year_21[i]['voltaicCharged'])
    Year_21_voltaicDischarged_list.append(Year_21[i]['voltaicDischarged'])
    Year_21_voltageCharged_list.append(Year_21[i]['voltageCharged'])
    Year_21_voltageDischarged_list.append(Year_21[i]['voltageDischarged'])
    Year_21_temperatureCharged_list.append(Year_21[i]['temperatureCharged'])
    Year_21_temperatureDischarged_list.append(Year_21[i]['temperatureDischarged'])
    Year_21_faceValue_list.append(Year_21[i]['faceValue'])
    Year_21_dod_list.append(Year_21[i]['dod']) 
    Year_21_chargedTimeTotal_list.append(Year_21[i]['chargedTimeTotal'])
    Year_21_dischargedTimeTotal_list.append(Year_21[i]['dischargedTimeTotal'])
    Year_21_overChargeTimeTotal_list.append(Year_21[i]['overChargeTimeTotal'])
    Year_21_overdisChargeTimeTotal_list.append(Year_21[i]['overdisChargeTimeTotal'])
    Year_21_abnormalTemperatureTimeTotal_list.append(Year_21[i]['abnormalTemperatureTimeTotal'])
    Year_21_lackFaceValueTimeTotal_list.append(Year_21[i]['lackFaceValueTimeTotal'])
    Year_21_insufficientChargeTimeTotal_list.append(Year_21[i]['insufficientChargeTimeTotal'])
    Year_21_insufficientCharge_list.append(Year_21[i]['insufficientCharge'])
#     Year_21_meancycle_list.append(Year_21[i]['meancycle'])
    Year_21_usedLifeMonth_list.append(Year_21[i]['usedLifeMonth'])
    
Year_21_chargedTime = []
Year_21_dischargedTime = []
Year_21_elcAverage = []
Year_21_ulcAverage = []
Year_21_batteryState = []
Year_21_overChargeCount = []
Year_21_overdisChargeCount = []
Year_21_abnormalTemperatureCount = []
Year_21_lackFaceValueCount = []
Year_21_insufficientChargeCount = []
Year_21_voltaicCharged = []
Year_21_voltaicDischarged = []
Year_21_voltageCharged = []
Year_21_voltageDischarged = []
Year_21_temperatureCharged = []
Year_21_temperatureDischarged = []
Year_21_faceValue = []
Year_21_dod = [] # 월 데이터 리스트
Year_21_year = []
Year_21_month = []
Year_21_sensorID = []
Year_21_chargedTimeTotal = []
Year_21_dischargedTimeTotal = []
Year_21_overChargeTimeTotal = []
Year_21_overdisChargeTimeTotal =[]
Year_21_abnormalTemperatureTimeTotal = []
Year_21_lackFaceValueTimeTotal = []
Year_21_insufficientChargeTimeTotal = []
Year_21_insufficientCharge = []
# Year_21_meancycle = []
Year_21_expectedLifeMonth = []
Year_21_expectedLifeCycle = []
Year_21_usedLifeMonth = []
Year_21_expectedLifePercent = []
Year_21_usedLifePercent = []

try :
    Year_21_chargedTime.append(sum(Year_21_chargedTime_list)/len(Year_21_chargedTime_list))
    Year_21_dischargedTime.append(sum(Year_21_dischargedTime_list)/len(Year_21_dischargedTime_list))
    Year_21_elcAverage.append(sum(Year_21_elcAverage_list)/len(Year_21_elcAverage_list))
    Year_21_ulcAverage.append(sum(Year_21_ulcAverage_list)/len(Year_21_ulcAverage_list))
    Year_21_batteryState.append(Year_21[-1]['batteryState'])
    Year_21_overChargeCount.append(sum(Year_21_overChargeCount_list)/len(Year_21_overChargeCount_list))
    Year_21_overdisChargeCount.append(sum(Year_21_overdisChargeCount_list)/len(Year_21_overdisChargeCount_list))
    Year_21_abnormalTemperatureCount.append(sum(Year_21_abnormalTemperatureCount_list)/len(Year_21_abnormalTemperatureCount_list))
    Year_21_lackFaceValueCount.append(sum(Year_21_lackFaceValueCount_list)/len(Year_21_lackFaceValueCount_list))
    Year_21_insufficientChargeCount.append(sum(Year_21_insufficientChargeCount_list)/len(Year_21_insufficientChargeCount_list))
    Year_21_voltaicCharged.append(sum(Year_21_voltaicCharged_list)/len(Year_21_voltaicCharged_list))
    Year_21_voltaicDischarged.append(sum(Year_21_voltaicDischarged_list)/len(Year_21_voltaicDischarged_list))
    Year_21_voltageCharged.append(sum(Year_21_voltageCharged_list)/len(Year_21_voltageCharged_list))
    Year_21_voltageDischarged.append(sum(Year_21_voltageDischarged_list)/len(Year_21_voltageDischarged_list))
    Year_21_temperatureCharged.append(sum(Year_21_temperatureCharged_list)/len(Year_21_temperatureCharged_list))
    Year_21_temperatureDischarged.append(sum(Year_21_temperatureDischarged_list)/len(Year_21_temperatureDischarged_list))
    Year_21_faceValue.append(sum(Year_21_faceValue_list)/len(Year_21_faceValue_list))
    Year_21_dod.append(sum(Year_21_dod_list)/len(Year_21_dod_list)) 
    Year_21_year.append(Year_21[0]['year'])
    Year_21_sensorID.append(Year_21[0]['sensorID'])
    Year_21_chargedTimeTotal.append(sum(Year_21_chargedTimeTotal_list))
    Year_21_dischargedTimeTotal.append(sum(Year_21_dischargedTimeTotal_list))
    Year_21_overChargeTimeTotal.append(sum(Year_21_overChargeTimeTotal_list)/60)
    Year_21_overdisChargeTimeTotal.append(sum(Year_21_overdisChargeTimeTotal_list)/60)
    Year_21_abnormalTemperatureTimeTotal.append(sum(Year_21_abnormalTemperatureTimeTotal_list)/60) 
    Year_21_lackFaceValueTimeTotal.append(sum(Year_21_lackFaceValueTimeTotal_list)/60)
    Year_21_insufficientChargeTimeTotal.append(sum(Year_21_insufficientChargeTimeTotal_list)/60)   
    Year_21_insufficientCharge.append(sum(Year_21_insufficientCharge_list)/len(Year_21_insufficientCharge_list))
#     Year_21_meancycle.append(math.floor(sum(Year_21_meancycle_list)/len(Year_21_meancycle_list)))
    Year_21_usedLifeMonth.append(round(sum(Year_21_usedLifeMonth_list)/len(Year_21_usedLifeMonth_list),1))
    
except ZeroDivisionError:
    pass

# if len(Year_21) > 0:
#     if Year_21_dod[0] < 40 :
#         Dod_c = 1.5
#     elif Year_21_dod[0] >= 40 and Year_21_dod[0] < 50:
#         Dod_c = 1.42
#     elif Year_21_dod[0] >= 50 and Year_21_dod[0] < 60:
#         Dod_c = 1.25
#     elif Year_21_dod[0] >= 60 and Year_21_dod[0] < 70:
#         Dod_c = 1.08
#     elif Year_21_dod[0] >= 70 and Year_21_dod[0] < 80:
#         Dod_c = 0.92
#     elif Year_21_dod[0] >= 80 and Year_21_dod[0] < 90:
#         Dod_c = 0.75
#     elif Year_21_dod[0] >= 90 : # and DD <= 100 <-DD값이 낮아지면 다시 삽입
#         Dod_c = 0.58
#     else:
#         pass

#     if Year_21_temperatureDischarged[0] < 30 :
#         Year_21_average_temperatureDischarged = 30
#     else :
#         Year_21_average_temperatureDischarged = Year_21_temperatureDischarged[0]

#     Year_21_voltaicDischarged_ln = round(ln((Year_21_voltaicDischarged[0]/800)*5),1)

#     if pd.isna(Year_21_voltaicDischarged_ln) == True :
#         Year_21_voltaicDischarged_ln = 0
#     else :
#         pass

#     Year_21_expectedLifeMonth.append(round(((Dod_c*1200)*30*12/(Year_21_average_temperatureDischarged*(1+(7/30)*Year_21_voltaicDischarged_ln)*Year_21_meancycle[0]*220))*1*0.85*1,1))
#     Year_21_expectedLifeCycle.append(round(((Dod_c*1200)*30/(Year_21_average_temperatureDischarged*(1+(7/30)*Year_21_voltaicDischarged_ln)))*1*0.85*1,1))

#     if round((Year_21_expectedLifeCycle[0] * 100) / 1200,1) > 100 :
#         Year_21_expectedLifePercent.append(100)
#     else :
#         Year_21_expectedLifePercent.append(round((Year_21_expectedLifeCycle[0] * 100) / 1200,1))

#     Year_21_usedLifePercent.append(round((Year_21_ulcAverage[0] * 100) / 1200,1))


In [60]:
Year_22_chargedTime_list = []
Year_22_dischargedTime_list = []
Year_22_elcAverage_list = []
Year_22_ulcAverage_list = []
Year_22_batteryState_list = []
Year_22_overChargeCount_list = []
Year_22_overdisChargeCount_list = []
Year_22_abnormalTemperatureCount_list = []
Year_22_lackFaceValueCount_list = []
Year_22_insufficientChargeCount_list = []
Year_22_voltaicCharged_list = []
Year_22_voltaicDischarged_list = []
Year_22_voltageCharged_list = []
Year_22_voltageDischarged_list = []
Year_22_temperatureCharged_list = []
Year_22_temperatureDischarged_list = []
Year_22_faceValue_list = []
Year_22_dod_list = [] #계산식 저장 리스트
Year_22_chargedTimeTotal_list = []
Year_22_dischargedTimeTotal_list = []
Year_22_overChargeTimeTotal_list = []
Year_22_overdisChargeTimeTotal_list =[]
Year_22_abnormalTemperatureTimeTotal_list = []
Year_22_insufficientChargeTimeTotal_list = []
Year_22_lackFaceValueTimeTotal_list = []
Year_22_insufficientCharge_list = []
Year_22_meancycle_list = []
Year_22_usedLifeMonth_list = []

for i in range(len(Year_22)): # 월 데이터 모으기
    Year_22_chargedTime_list.append(Year_22[i]['chargedTime'])
    Year_22_dischargedTime_list.append(Year_22[i]['dischargedTime'])
    Year_22_elcAverage_list.append(Year_22[i]['expectedLifeCycle'])
    Year_22_ulcAverage_list.append(Year_22[i]['usedLifeCycle'])
    Year_22_batteryState_list.append(Year_22[i]['batteryState'])
    Year_22_overChargeCount_list.append(Year_22[i]['overChargeCount'])
    Year_22_overdisChargeCount_list.append(Year_22[i]['overdisChargeCount'])
    Year_22_abnormalTemperatureCount_list.append(Year_22[i]['abnormalTemperatureCount'])
    Year_22_lackFaceValueCount_list.append(Year_22[i]['lackFaceValueCount'])
    Year_22_insufficientChargeCount_list.append(Year_22[i]['insufficientChargeCount'])
    Year_22_voltaicCharged_list.append(Year_22[i]['voltaicCharged'])
    Year_22_voltaicDischarged_list.append(Year_22[i]['voltaicDischarged'])
    Year_22_voltageCharged_list.append(Year_22[i]['voltageCharged'])
    Year_22_voltageDischarged_list.append(Year_22[i]['voltageDischarged'])
    Year_22_temperatureCharged_list.append(Year_22[i]['temperatureCharged'])
    Year_22_temperatureDischarged_list.append(Year_22[i]['temperatureDischarged'])
    Year_22_faceValue_list.append(Year_22[i]['faceValue'])
    Year_22_dod_list.append(Year_22[i]['dod']) 
    Year_22_chargedTimeTotal_list.append(Year_22[i]['chargedTimeTotal'])
    Year_22_dischargedTimeTotal_list.append(Year_22[i]['dischargedTimeTotal'])
    Year_22_overChargeTimeTotal_list.append(Year_22[i]['overChargeTimeTotal'])
    Year_22_overdisChargeTimeTotal_list.append(Year_22[i]['overdisChargeTimeTotal'])
    Year_22_abnormalTemperatureTimeTotal_list.append(Year_22[i]['abnormalTemperatureTimeTotal'])
    Year_22_lackFaceValueTimeTotal_list.append(Year_22[i]['lackFaceValueTimeTotal'])
    Year_22_insufficientChargeTimeTotal_list.append(Year_22[i]['insufficientChargeTimeTotal'])
    Year_22_insufficientCharge_list.append(Year_22[i]['insufficientCharge'])
#     Year_22_meancycle_list.append(Year_22[i]['meancycle'])
    Year_22_usedLifeMonth_list.append(Year_22[i]['usedLifeMonth'])
    
Year_22_chargedTime = []
Year_22_dischargedTime = []
Year_22_elcAverage = []
Year_22_ulcAverage = []
Year_22_batteryState = []
Year_22_overChargeCount = []
Year_22_overdisChargeCount = []
Year_22_abnormalTemperatureCount = []
Year_22_lackFaceValueCount = []
Year_22_insufficientChargeCount = []
Year_22_voltaicCharged = []
Year_22_voltaicDischarged = []
Year_22_voltageCharged = []
Year_22_voltageDischarged = []
Year_22_temperatureCharged = []
Year_22_temperatureDischarged = []
Year_22_faceValue = []
Year_22_dod = [] # 월 데이터 리스트
Year_22_year = []
Year_22_month = []
Year_22_sensorID = []
Year_22_chargedTimeTotal = []
Year_22_dischargedTimeTotal = []
Year_22_overChargeTimeTotal = []
Year_22_overdisChargeTimeTotal =[]
Year_22_abnormalTemperatureTimeTotal = []
Year_22_lackFaceValueTimeTotal = []
Year_22_insufficientChargeTimeTotal = []
Year_22_insufficientCharge = []
# Year_22_meancycle = []
Year_22_expectedLifeMonth = []
Year_22_expectedLifeCycle = []
Year_22_usedLifeMonth = []
Year_22_expectedLifePercent = []
Year_22_usedLifePercent = []

try :
    Year_22_chargedTime.append(sum(Year_22_chargedTime_list)/len(Year_22_chargedTime_list))
    Year_22_dischargedTime.append(sum(Year_22_dischargedTime_list)/len(Year_22_dischargedTime_list))
    Year_22_elcAverage.append(sum(Year_22_elcAverage_list)/len(Year_22_elcAverage_list))
    Year_22_ulcAverage.append(sum(Year_22_ulcAverage_list)/len(Year_22_ulcAverage_list))
    Year_22_batteryState.append(Year_22[-1]['batteryState'])
    Year_22_overChargeCount.append(sum(Year_22_overChargeCount_list)/len(Year_22_overChargeCount_list))
    Year_22_overdisChargeCount.append(sum(Year_22_overdisChargeCount_list)/len(Year_22_overdisChargeCount_list))
    Year_22_abnormalTemperatureCount.append(sum(Year_22_abnormalTemperatureCount_list)/len(Year_22_abnormalTemperatureCount_list))
    Year_22_lackFaceValueCount.append(sum(Year_22_lackFaceValueCount_list)/len(Year_22_lackFaceValueCount_list))
    Year_22_insufficientChargeCount.append(sum(Year_22_insufficientChargeCount_list)/len(Year_22_insufficientChargeCount_list))
    Year_22_voltaicCharged.append(sum(Year_22_voltaicCharged_list)/len(Year_22_voltaicCharged_list))
    Year_22_voltaicDischarged.append(sum(Year_22_voltaicDischarged_list)/len(Year_22_voltaicDischarged_list))
    Year_22_voltageCharged.append(sum(Year_22_voltageCharged_list)/len(Year_22_voltageCharged_list))
    Year_22_voltageDischarged.append(sum(Year_22_voltageDischarged_list)/len(Year_22_voltageDischarged_list))
    Year_22_temperatureCharged.append(sum(Year_22_temperatureCharged_list)/len(Year_22_temperatureCharged_list))
    Year_22_temperatureDischarged.append(sum(Year_22_temperatureDischarged_list)/len(Year_22_temperatureDischarged_list))
    Year_22_faceValue.append(sum(Year_22_faceValue_list)/len(Year_22_faceValue_list))
    Year_22_dod.append(sum(Year_22_dod_list)/len(Year_22_dod_list)) 
    Year_22_year.append(Year_22[0]['year'])
    Year_22_sensorID.append(Year_22[0]['sensorID'])
    Year_22_chargedTimeTotal.append(sum(Year_22_chargedTimeTotal_list))
    Year_22_dischargedTimeTotal.append(sum(Year_22_dischargedTimeTotal_list))
    Year_22_overChargeTimeTotal.append(sum(Year_22_overChargeTimeTotal_list)/60)
    Year_22_overdisChargeTimeTotal.append(sum(Year_22_overdisChargeTimeTotal_list)/60)
    Year_22_abnormalTemperatureTimeTotal.append(sum(Year_22_abnormalTemperatureTimeTotal_list)/60) 
    Year_22_lackFaceValueTimeTotal.append(sum(Year_22_lackFaceValueTimeTotal_list)/60)
    Year_22_insufficientChargeTimeTotal.append(sum(Year_22_insufficientChargeTimeTotal_list)/60)   
    Year_22_insufficientCharge.append(sum(Year_22_insufficientCharge_list)/len(Year_22_insufficientCharge_list))
#     Year_22_meancycle.append(math.floor(sum(Year_22_meancycle_list)/len(Year_22_meancycle_list)))
    Year_22_usedLifeMonth.append(round(sum(Year_22_usedLifeMonth_list)/len(Year_22_usedLifeMonth_list),1))
    
except ZeroDivisionError:
    pass

# if len(Year_22) > 0:
#     if Year_22_dod[0] < 40 :
#         Dod_c = 1.5
#     elif Year_22_dod[0] >= 40 and Year_22_dod[0] < 50:
#         Dod_c = 1.42
#     elif Year_22_dod[0] >= 50 and Year_22_dod[0] < 60:
#         Dod_c = 1.25
#     elif Year_22_dod[0] >= 60 and Year_22_dod[0] < 70:
#         Dod_c = 1.08
#     elif Year_22_dod[0] >= 70 and Year_22_dod[0] < 80:
#         Dod_c = 0.92
#     elif Year_22_dod[0] >= 80 and Year_22_dod[0] < 90:
#         Dod_c = 0.75
#     elif Year_22_dod[0] >= 90 : # and DD <= 100 <-DD값이 낮아지면 다시 삽입
#         Dod_c = 0.58
#     else:
#         pass

#     if Year_22_temperatureDischarged[0] < 30 :
#         Year_22_average_temperatureDischarged = 30
#     else :
#         Year_22_average_temperatureDischarged = Year_22_temperatureDischarged[0]

#     Year_22_voltaicDischarged_ln = round(ln((Year_22_voltaicDischarged[0]/800)*5),1)

#     if pd.isna(Year_22_voltaicDischarged_ln) == True :
#         Year_22_voltaicDischarged_ln = 0
#     else :
#         pass

#     Year_22_expectedLifeMonth.append(round(((Dod_c*1200)*30*12/(Year_22_average_temperatureDischarged*(1+(7/30)*Year_22_voltaicDischarged_ln)*Year_22_meancycle[0]*220))*1*0.85*1,1))
#     Year_22_expectedLifeCycle.append(round(((Dod_c*1200)*30/(Year_22_average_temperatureDischarged*(1+(7/30)*Year_22_voltaicDischarged_ln)))*1*0.85*1,1))

#     if round((Year_22_expectedLifeCycle[0] * 100) / 1200,1) > 100 :
#         Year_22_expectedLifePercent.append(100)
#     else :
#         Year_22_expectedLifePercent.append(round((Year_22_expectedLifeCycle[0] * 100) / 1200,1))

#     Year_22_usedLifePercent.append(round((Year_22_ulcAverage[0] * 100) / 1200,1))


In [61]:
col_name = ['sensorID'] # 년 월 배터리 상태 추가

sensorID_list = Year_21_sensorID+Year_22_sensorID

Year_df = pd.DataFrame(sensorID_list, columns = col_name)


In [62]:
Year_df['year'] = Year_21_year+Year_22_year
Year_df['chargedTime'] = np.round(Year_21_chargedTime+Year_22_chargedTime,1)
Year_df['dischargedTime'] = np.round(Year_21_dischargedTime+Year_22_dischargedTime,1)
Year_df['elcAverage'] = np.round(Year_21_elcAverage+Year_22_elcAverage,0)
Year_df['ulcAverage'] = np.round(Year_21_ulcAverage+Year_22_ulcAverage,0)
Year_df['batteryState'] = Year_21_batteryState+Year_22_batteryState
Year_df['overChargeCount'] = np.round(Year_21_overChargeCount+Year_22_overChargeCount,1)
Year_df['overdisChargeCount'] = np.round(Year_21_overdisChargeCount+Year_22_overdisChargeCount,1)
Year_df['abnormalTemperatureCount'] = np.round(Year_21_abnormalTemperatureCount+Year_22_abnormalTemperatureCount,1)
Year_df['lackFaceValueCount'] = np.round(Year_21_lackFaceValueCount+Year_22_lackFaceValueCount,1)
Year_df['voltaicCharged'] = np.round(Year_21_voltaicCharged+Year_22_voltaicCharged,1)
Year_df['voltaicDischarged'] = np.round(Year_21_voltaicDischarged+Year_22_voltaicDischarged,1)
Year_df['voltageCharged'] = np.round(Year_21_voltageCharged+Year_22_voltageCharged,1)
Year_df['voltageDischarged'] = np.round(Year_21_voltageDischarged+Year_22_voltageDischarged,1)
Year_df['temperatureCharged'] = np.round(Year_21_temperatureCharged+Year_22_temperatureCharged,1)
Year_df['temperatureDischarged'] = np.round(Year_21_temperatureDischarged+Year_22_temperatureDischarged,1)
Year_df['faceValue'] = np.round(Year_21_faceValue+Year_22_faceValue,1)
Year_df['dod'] = np.round(Year_21_dod+Year_22_dod,1)
Year_df['chargedTimeTotal'] = np.round(Year_21_chargedTimeTotal+Year_22_chargedTimeTotal,1)
Year_df['dischargedTimeTotal'] = np.round(Year_21_dischargedTimeTotal+Year_22_dischargedTimeTotal,1)
Year_df['overChargeTimeTotal'] = np.round(Year_21_overChargeTimeTotal+Year_22_overChargeTimeTotal,1)
Year_df['overdisChargeTimeTotal'] = np.round(Year_21_overdisChargeTimeTotal+Year_22_overdisChargeTimeTotal,1)
Year_df['abnormalTemperatureTimeTotal'] = np.round(Year_21_abnormalTemperatureTimeTotal+Year_22_abnormalTemperatureTimeTotal,1)
Year_df['lackFaceValueTimeTotal'] = np.round(Year_21_lackFaceValueTimeTotal+Year_22_lackFaceValueTimeTotal,1)
Year_df['year'] = pd.to_numeric(Year_df['year']) #년 숫자 형식 변환


In [63]:
Year_df.to_csv(path_list[0] + 'Year_result.csv',index=False, encoding="utf-8-sig",mode = 'w') # 월 평균 데이터 저장

csv to josn /  json to server ▽

In [4]:
for i in range(1):
    output_file_name = "Hour_result.json"
    with open(path_list[0] + 'Hourresult.csv', "r", encoding='utf-8-sig', newline="") as input_file, \
        open(path_list[0] + output_file_name, "w", encoding='utf-8-sig', newline="") as output_file:

        reader = csv.reader(input_file)
        # 첫 줄은 col_names 리스트로 읽어 놓고
        col_names = next(reader)
        # 그 다음 줄부터 zip으로 묶어서 json으로 dumps
        output_file.write('[')
        for cols in reader:
            doc = {col_name: col for col_name, col in zip(col_names, cols)}
            print(json.dumps(doc, ensure_ascii=False,indent = 4), file=output_file)
            output_file.write(',')
        output_file.write(']') #리스트로 묶기

    with open(path_list[0] +'Hour_result.json', 'r',encoding='utf-8-sig') as f:
        n_list = (f.readlines())
        n_list[len(n_list)-1] = ']' #불필요한 마지막값 제거 

    with open(path_list[0] +'Hour_result.json','w',encoding='utf-8-sig') as e:
        e.writelines(n_list)
        
# 일 평균 csv json으로 변환

In [5]:
for i in range(1):
    output_file_name = "Dayresult.json"
    with open(path_list[0] + 'Dayresult.csv', "r", encoding='utf-8-sig', newline="") as input_file, \
        open(path_list[0] + output_file_name, "w", encoding='utf-8-sig', newline="") as output_file:

        reader = csv.reader(input_file)
        # 첫 줄은 col_names 리스트로 읽어 놓고
        col_names = next(reader)
        # 그 다음 줄부터 zip으로 묶어서 json으로 dumps
        output_file.write('[')
        for cols in reader:
            doc = {col_name: col for col_name, col in zip(col_names, cols)}
            print(json.dumps(doc, ensure_ascii=False,indent = 4), file=output_file)
            output_file.write(',')
        output_file.write(']') #리스트로 묶기

    with open(path_list[0] +'Dayresult.json', 'r',encoding='utf-8-sig') as f:
        n_list = (f.readlines())
        n_list[len(n_list)-1] = ']' #불필요한 마지막값 제거 

    with open(path_list[0] +'Dayresult.json','w',encoding='utf-8-sig') as e:
        e.writelines(n_list)
        
# 일 평균 csv json으로 변환

In [79]:
for i in range(1):
    output_file_name = "Month_result.json"
    with open(path_list[0] + 'Month_result.csv', "r", encoding='utf-8-sig', newline="") as input_file, \
        open(path_list[0] + output_file_name, "w", encoding='utf-8-sig', newline="") as output_file:

        reader = csv.reader(input_file)
        # 첫 줄은 col_names 리스트로 읽어 놓고
        col_names = next(reader)
        # 그 다음 줄부터 zip으로 묶어서 json으로 dumps
        output_file.write('[')
        for cols in reader:
            doc = {col_name: col for col_name, col in zip(col_names, cols)}
            print(json.dumps(doc, ensure_ascii=False,indent = 4), file=output_file)
            output_file.write(',')
        output_file.write(']') #리스트로 묶기

    with open(path_list[0] + 'Month_result.json', 'r',encoding='utf-8-sig') as f:
        n_list = (f.readlines())
        n_list[len(n_list)-1] = ']' #불필요한 마지막값 제거 

    with open(path_list[0] + 'Month_result.json','w',encoding='utf-8-sig') as e:
        e.writelines(n_list)

# 월 평균 csv json으로 변환

In [ ]:
for i in range(1):
    output_file_name = "Year_result.json"
    with open(path_list[0] + 'Year_result.csv', "r", encoding='utf-8-sig', newline="") as input_file, \
        open(path_list[0] + output_file_name, "w", encoding='utf-8-sig', newline="") as output_file:

        reader = csv.reader(input_file)
        # 첫 줄은 col_names 리스트로 읽어 놓고
        col_names = next(reader)
        # 그 다음 줄부터 zip으로 묶어서 json으로 dumps
        output_file.write('[')
        for cols in reader:
            doc = {col_name: col for col_name, col in zip(col_names, cols)}
            print(json.dumps(doc, ensure_ascii=False,indent = 4), file=output_file)
            output_file.write(',')
        output_file.write(']') #리스트로 묶기

    with open(path_list[0] + 'Year_result.json', 'r',encoding='utf-8-sig') as f:
        n_list = (f.readlines())
        n_list[len(n_list)-1] = ']' #불필요한 마지막값 제거 

    with open(path_list[0] + 'Year_result.json','w',encoding='utf-8-sig') as e:
        e.writelines(n_list)

# 년 평균 csv json으로 변환

 ▼ SOFTMILE(Json to Server)

In [6]:
headers={
    'Authorization': '' ,
    'Content-Type': 'application/json'
}

with open(path_list[0] + 'Hour_result.json','r',encoding='utf-8-sig') as f:

    json_data = json.load(f) #파일 JSON형식으로 읽기

jd = json_data
    
cut_point = 0 
while True:
    data=json.dumps(jd[cut_point:cut_point+10],indent = 4)
    print(requests.post("http://3.34.245.33/api/sensor/analysis/hour", headers=headers, data=data)) #data json형식으로 맞춰서 데이터 전송
    cut_point += 10
    if cut_point > len(jd): #10개씩 나눠서 보내기
        break
#일 평균 데이터 전송

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

In [7]:
headers={
    'Authorization': '' ,
    'Content-Type': 'application/json'
}

with open(path_list[0] + 'Dayresult.json','r',encoding='utf-8-sig') as f:

    json_data = json.load(f) #파일 JSON형식으로 읽기

jd = json_data
    
cut_point = 0 
while True:
    data=json.dumps(jd[cut_point:cut_point+10],indent = 4)
    print(requests.post("http://3.34.245.33/api/sensor/analysis/day", headers=headers, data=data)) #data json형식으로 맞춰서 데이터 전송
    cut_point += 10
    if cut_point > len(jd): #10개씩 나눠서 보내기
        break
#일 평균 데이터 전송

<Response [200]>
<Response [200]>
<Response [200]>


In [80]:
headers={
    'Authorization': '' ,
    'Content-Type': 'application/json'
}

with open(path_list[0] + 'Month_result.json','r',encoding='utf-8-sig') as f:

    json_data = json.load(f) #파일 JSON형식으로 읽기

jd = json_data
    
cut_point = 0 
while True:
    data=json.dumps(jd[cut_point:cut_point+10],indent = 4)
    print(requests.post("http://3.34.245.33/api/sensor/analysis/month", headers=headers, data=data)) #data json형식으로 맞춰서 데이터 전송
    cut_point += 10
    if cut_point > len(jd): #10개씩 나눠서 보내기
        break
#월 평균 데이터 전송

<Response [200]>


In [ ]:
headers={
    'Authorization': '' ,
    'Content-Type': 'application/json'
}

with open(path_list[0] + 'Year_result.json','r',encoding='utf-8-sig') as f:

    json_data = json.load(f) #파일 JSON형식으로 읽기

jd = json_data
    
cut_point = 0 
while True:
    data=json.dumps(jd[cut_point:cut_point+10],indent = 4)
    print(requests.post("http://3.34.245.33/api/sensor/analysis/year", headers=headers, data=data)) #data json형식으로 맞춰서 데이터 전송
    cut_point += 10
    if cut_point > len(jd): #10개씩 나눠서 보내기
        break
#월 평균 데이터 전송